In [1]:
import pandas as pd
import numpy as np
from pytimekr import pytimekr
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style='whitegrid', font_scale=1.5)
sns.set_palette('Set2', n_colors=10)
plt.rc('font', family='malgun gothic')
plt.rc('axes', unicode_minus=False)

In [2]:
# Data Load
train = pd.read_csv('data/tmk_bda_train.csv').iloc[:,1:]
test = pd.read_csv('data/tmk_bda_test.csv').iloc[:,1:]

# Data Preprocessing

#### 자료형 변환

In [3]:
# train
train['age_grp'] = train['age_grp'].astype('object')
train['order_date'] = pd.to_datetime(train['order_date'],format='%Y%m%d')

# test
test['age_grp'] = test['age_grp'].astype('object')
test['order_date'] = pd.to_datetime(test['order_date'],format='%Y%m%d')

#### Prime_yn Encoding

In [4]:
train['prime_yn'] = train['prime_yn'].apply(lambda x : 1 if x == 'Y' else 0)

#### 임직원과 비임직원 데이터 분할

In [5]:
# train
employee_tr = train.loc[train.employee_yn == 'Y'].drop('employee_yn',axis=1)
non_employee_tr = train.loc[train.employee_yn == 'N'].drop('employee_yn',axis=1)

# test
employee_te = test.loc[test.employee_yn == 'Y'].drop('employee_yn',axis=1)
non_employee_te = test.loc[test.employee_yn == 'N'].drop('employee_yn',axis=1)

#### net_order_qty의 이상치를 적정값으로 변환

In [6]:
# train
employee_tr['net_order_qty'] = employee_tr['net_order_qty'].apply(lambda x : 50 if x >= 60 else x)
non_employee_tr['net_order_qty'] = non_employee_tr['net_order_qty'].apply(lambda x : 50 if x >= 55 else x)

# test
employee_te['net_order_qty'] = employee_te['net_order_qty'].apply(lambda x : 50 if x >= 60 else x)
non_employee_te['net_order_qty'] = non_employee_te['net_order_qty'].apply(lambda x : 50 if x >= 55 else x)

# Feature Generation

#### 원본 데이터 유지를 위해 복사본 사용

In [7]:
# train
emp_tr = employee_tr.copy()
nemp_tr = non_employee_tr.copy()

# test
emp_te = employee_te.copy()
nemp_te = non_employee_te.copy()

#### '밥'이 들어간 상품 구매여부

In [8]:
# train
emp_tr['whether_rice'] = emp_tr['product_name'].apply(lambda x : 1 if '밥' in x else 0)
nemp_tr['whether_rice'] = nemp_tr['product_name'].apply(lambda x : 1 if '밥' in x else 0)

# test
emp_te['whether_rice'] = emp_te['product_name'].apply(lambda x : 1 if '밥' in x else 0)
nemp_te['whether_rice'] = nemp_te['product_name'].apply(lambda x : 1 if '밥' in x else 0)

#### 나이대별 주문수량 - 단순연산

In [9]:
# train
f_tr = emp_tr.groupby('age_grp')['net_order_qty'].agg([('age_qty_sum', np.sum), ('age_qty_mean', np.mean),
                                                       ('age_qty_max', np.max), ('age_qty_std', np.std),
                                                       ('age_qty_lam', lambda x : np.std(x)/np.mean(x))]).reset_index()

nf_tr = nemp_tr.groupby('age_grp')['net_order_qty'].agg([('age_qty_sum', np.sum), ('age_qty_mean', np.mean),
                                                         ('age_qty_max', np.max), ('age_qty_std', np.std),
                                                         ('age_qty_lam', lambda x : np.std(x)/np.mean(x))]).reset_index()

#test
f_te = emp_te.groupby('age_grp')['net_order_qty'].agg([('age_qty_sum', np.sum), ('age_qty_mean', np.mean),
                                                       ('age_qty_max', np.max), ('age_qty_std', np.std),
                                                       ('age_qty_lam', lambda x : np.std(x)/np.mean(x))]).reset_index()

nf_te = nemp_te.groupby('age_grp')['net_order_qty'].agg([('age_qty_sum', np.sum), ('age_qty_mean', np.mean),
                                                         ('age_qty_max', np.max), ('age_qty_std', np.std),
                                                         ('age_qty_lam', lambda x : np.std(x)/np.mean(x))]).reset_index()

emp_tr = pd.merge(emp_tr, f_tr, on = 'age_grp', how='left')
nemp_tr = pd.merge(nemp_tr, nf_tr, on = 'age_grp', how='left')
emp_te = pd.merge(emp_te, f_te, on = 'age_grp', how='left')
nemp_te = pd.merge(nemp_te, nf_te, on = 'age_grp', how='left')

display(f_tr, nf_tr, f_te, nf_te)

,age_grp,age_qty_sum,age_qty_mean,age_qty_max,age_qty_std,age_qty_lam
0,1,14,1.555556,4,1.130388,0.685119
1,2,5661,1.802866,50,2.538582,1.407857
2,3,14016,1.799692,50,2.541034,1.411837
3,4,9878,1.676795,30,1.802310,1.074763
4,5,5196,1.903297,50,2.667371,1.401191
5,6,394,1.595142,25,1.895131,1.185657


,age_grp,age_qty_sum,age_qty_mean,age_qty_max,age_qty_std,age_qty_lam
0,1,31,1.148148,2,0.362014,0.309408
1,2,2022,1.720851,36,2.606577,1.514057
2,3,10109,1.651528,50,2.573803,1.558311
3,4,20945,1.597270,50,2.077366,1.300523
4,5,7745,1.619615,50,2.338848,1.443925
5,6,1550,1.819249,50,2.837586,1.558842


,age_grp,age_qty_sum,age_qty_mean,age_qty_max,age_qty_std,age_qty_lam
0,2,2362,1.846755,20,2.161689,1.170076
1,3,6299,1.923946,50,2.718489,1.412760
2,4,4344,1.830594,50,2.601230,1.420677
3,5,2051,1.993197,50,2.695823,1.351855
4,6,160,1.758242,13,1.922618,1.087464


,age_grp,age_qty_sum,age_qty_mean,age_qty_max,age_qty_std,age_qty_lam
0,1,11,1.833333,5,1.602082,0.797724
1,2,1006,1.668325,18,1.969732,1.179685
2,3,5034,1.651033,50,2.465424,1.493016
3,4,9748,1.674914,50,2.125668,1.269011
4,5,3209,1.748774,50,2.496583,1.427230
5,6,593,1.970100,20,2.200554,1.115119


#### 나이대별 주문금액 - 단순연산

In [10]:
# train
f_tr = emp_tr.groupby('age_grp')['net_order_amt'].agg([('age_amt_sum', np.sum), ('age_amt_mean', np.mean),
                                                       ('age_amt_max', np.max), ('age_amt_min', np.min), ('age_amt_std', np.std), 
                                                       ('age_amt_lam', lambda x : np.std(x)/np.mean(x))]).reset_index()

nf_tr = nemp_tr.groupby('age_grp')['net_order_amt'].agg([('age_amt_sum', np.sum), ('age_amt_mean', np.mean),
                                                       ('age_amt_max', np.max), ('age_amt_min', np.min), ('age_amt_std', np.std), 
                                                       ('age_amt_lam', lambda x : np.std(x)/np.mean(x))]).reset_index()

#test
f_te = emp_te.groupby('age_grp')['net_order_amt'].agg([('age_amt_sum', np.sum), ('age_amt_mean', np.mean),
                                                       ('age_amt_max', np.max), ('age_amt_min', np.min), ('age_amt_std', np.std), 
                                                       ('age_amt_lam', lambda x : np.std(x)/np.mean(x))]).reset_index()

nf_te = nemp_te.groupby('age_grp')['net_order_amt'].agg([('age_amt_sum', np.sum), ('age_amt_mean', np.mean),
                                                       ('age_amt_max', np.max), ('age_amt_min', np.min), ('age_amt_std', np.std), 
                                                       ('age_amt_lam', lambda x : np.std(x)/np.mean(x))]).reset_index()

emp_tr = pd.merge(emp_tr, f_tr, on = 'age_grp', how='left')
nemp_tr = pd.merge(nemp_tr, nf_tr, on = 'age_grp', how='left')
emp_te = pd.merge(emp_te, f_te, on = 'age_grp', how='left')
nemp_te = pd.merge(nemp_te, nf_te, on = 'age_grp', how='left')

display(f_tr, nf_tr, f_te, nf_te)

,age_grp,age_amt_sum,age_amt_mean,age_amt_max,age_amt_min,age_amt_std,age_amt_lam
0,1,82.406257,9.156251,11.346883,7.486053,1.113599,0.114666
1,2,29443.654827,9.376960,14.263333,6.529419,1.062394,0.113280
2,3,72784.833843,9.345767,14.960234,6.398595,1.055813,0.112965
3,4,55081.236431,9.350066,13.787356,6.398595,1.033456,0.110520
4,5,25894.559524,9.485187,14.367981,6.529419,1.087832,0.114666
5,6,2278.447731,9.224485,12.640556,6.850126,1.024210,0.110807


,age_grp,age_amt_sum,age_amt_mean,age_amt_max,age_amt_min,age_amt_std,age_amt_lam
0,1,229.639227,8.505157,10.113627,7.362011,0.696935,0.080411
1,2,10520.071711,8.953253,14.429067,6.388561,0.991499,0.110695
2,3,54632.412902,8.925406,15.224838,6.315358,0.858737,0.096205
3,4,117222.001841,8.939373,15.647347,6.505784,0.821749,0.091921
4,5,43167.956688,9.027176,14.703951,6.368187,0.901548,0.099860
5,6,7796.033936,9.150275,13.846193,6.565265,0.932694,0.101871


,age_grp,age_amt_sum,age_amt_mean,age_amt_max,age_amt_min,age_amt_std,age_amt_lam
0,2,12006.393148,9.387328,13.102701,6.746412,0.981570,0.104522
1,3,30620.795909,9.352717,14.085958,6.398595,1.032606,0.110390
2,4,22361.197902,9.423177,14.010500,6.447306,1.019646,0.108183
3,5,9771.299956,9.495918,13.945003,7.077498,1.056864,0.111243
4,6,850.795981,9.349406,11.726027,7.711101,0.927965,0.098707


,age_grp,age_amt_sum,age_amt_mean,age_amt_max,age_amt_min,age_amt_std,age_amt_lam
0,1,51.223448,8.537241,10.450481,7.353082,1.333545,0.142593
1,2,5329.893357,8.838961,12.568140,6.552508,0.881066,0.099597
2,3,26753.191378,8.774415,14.646290,6.493754,0.803936,0.091608
3,4,51387.235130,8.829422,14.635427,6.552508,0.810374,0.091773
4,5,16465.229934,8.972877,14.674198,6.634633,0.869077,0.096830
5,6,2740.702801,9.105325,13.358109,6.610696,0.959518,0.105205


#### 구매일별 주문수량 - 단순연산

In [11]:
# train
f_tr = emp_tr.groupby('order_date')['net_order_qty'].agg([('order_qty_sum', np.sum), ('order_qty_mean', np.mean),
                                                          ('order_qty_max', np.max), ('order_qty_std', np.std),
                                                          ('order_qty_lam', lambda x : np.std(x)/np.mean(x))]).reset_index()

nf_tr = nemp_tr.groupby('order_date')['net_order_qty'].agg([('order_qty_sum', np.sum), ('order_qty_mean', np.mean),
                                                            ('order_qty_max', np.max), ('order_qty_std', np.std),
                                                            ('order_qty_lam', lambda x : np.std(x)/np.mean(x))]).reset_index()

#test
f_te = emp_te.groupby('order_date')['net_order_qty'].agg([('order_qty_sum', np.sum), ('order_qty_mean', np.mean),
                                                          ('order_qty_max', np.max), ('order_qty_std', np.std),
                                                          ('order_qty_lam', lambda x : np.std(x)/np.mean(x))]).reset_index()

nf_te = nemp_te.groupby('order_date')['net_order_qty'].agg([('order_qty_sum', np.sum), ('order_qty_mean', np.mean),
                                                            ('order_qty_max', np.max), ('order_qty_std', np.std),
                                                            ('order_qty_lam', lambda x : np.std(x)/np.mean(x))]).reset_index()

emp_tr = pd.merge(emp_tr, f_tr, on = 'order_date', how='left')
nemp_tr = pd.merge(nemp_tr, nf_tr, on = 'order_date', how='left')
emp_te = pd.merge(emp_te, f_te, on = 'order_date', how='left')
nemp_te = pd.merge(nemp_te, nf_te, on = 'order_date', how='left')

display(f_tr, nf_tr, f_te, nf_te)

,order_date,order_qty_sum,order_qty_mean,order_qty_max,order_qty_std,order_qty_lam
0,2023-01-01,5039,1.950832,35,2.370647,1.214962
1,2023-01-02,2608,2.265856,50,3.273962,1.444284
2,2023-01-03,699,2.144172,50,3.847166,1.791490
3,2023-01-04,139,2.074627,25,3.134913,1.499754
4,2023-01-05,43,1.535714,6,1.373887,0.878504
5,2023-01-06,98,2.041667,29,4.047370,1.961627
6,2023-01-07,8,1.600000,2,0.547723,0.306186
7,2023-01-08,100,5.000000,50,11.206201,2.184491
8,2023-01-09,57,1.500000,5,0.796954,0.524265
9,2023-01-10,256,5.120000,50,11.322075,2.189118


,order_date,order_qty_sum,order_qty_mean,order_qty_max,order_qty_std,order_qty_lam
0,2023-01-01,6800,1.810919,50,2.378074,1.313011
1,2023-01-02,2749,1.938646,50,3.337271,1.720837
2,2023-01-03,1085,2.047170,49,3.411817,1.665029
3,2023-01-04,317,2.756522,20,3.812701,1.377130
4,2023-01-05,17,3.400000,13,5.366563,1.411765
5,2023-01-06,11,1.000000,1,0.000000,0.000000
6,2023-01-07,3,1.000000,1,0.000000,0.000000
7,2023-01-08,13,1.444444,3,0.726483,0.474186
8,2023-01-09,91,3.956522,50,10.119640,2.501491
9,2023-01-10,207,3.980769,50,8.110730,2.017792


,order_date,order_qty_sum,order_qty_mean,order_qty_max,order_qty_std,order_qty_lam
0,2023-01-01,3356,1.936526,50,2.586050,1.335021
1,2023-01-02,3233,2.153897,50,3.199318,1.484867
2,2023-01-03,1482,2.524702,50,3.945099,1.561268
3,2023-01-04,197,1.876190,35,3.418545,1.813370
4,2023-01-05,59,1.638889,5,1.046157,0.629405
5,2023-01-09,5,2.500000,4,2.121320,0.600000
6,2023-01-10,23,1.150000,2,0.366348,0.310497
7,2023-01-11,53,1.827586,6,1.255530,0.675039
8,2023-01-12,295,2.251908,16,2.351247,1.040120
9,2023-01-13,705,2.182663,15,2.215293,1.013378


,order_date,order_qty_sum,order_qty_mean,order_qty_max,order_qty_std,order_qty_lam
0,2023-01-01,5739,1.703979,50,1.952196,1.145499
1,2023-01-02,4134,1.868052,50,2.486807,1.330929
2,2023-01-03,1477,2.315047,49,3.794087,1.637596
3,2023-01-04,197,2.525641,42,4.884974,1.921714
4,2023-01-05,11,2.200000,5,1.643168,0.668043
5,2023-01-07,4,2.000000,3,1.414214,0.500000
6,2023-01-09,38,2.375000,5,1.454877,0.593128
7,2023-01-10,61,3.388889,31,7.080536,2.030472
8,2023-01-11,39,3.545455,22,6.186496,1.663705
9,2023-01-12,319,3.067308,50,7.352781,2.385592


#### 구매일별 주문금액 - 단순연산

In [12]:
# train
f_tr = emp_tr.groupby('order_date')['net_order_amt'].agg([('order_amt_sum',np.sum),('order_amt_mean',np.mean),
                                                          ('order_amt_max',np.max),('order_amt_min',np.min),('order_amt_std',np.std),
                                                          ('order_amt_lam', lambda x : np.std(x)/np.mean(x))]).reset_index()

nf_tr = nemp_tr.groupby('order_date')['net_order_amt'].agg([('order_amt_sum',np.sum),('order_amt_mean',np.mean),
                                                          ('order_amt_max',np.max),('order_amt_min',np.min),('order_amt_std',np.std),
                                                          ('order_amt_lam', lambda x : np.std(x)/np.mean(x))]).reset_index()
#test
f_te = emp_te.groupby('order_date')['net_order_amt'].agg([('order_amt_sum',np.sum),('order_amt_mean',np.mean),
                                                          ('order_amt_max',np.max),('order_amt_min',np.min),('order_amt_std',np.std),
                                                          ('order_amt_lam', lambda x : np.std(x)/np.mean(x))]).reset_index()

nf_te = nemp_te.groupby('order_date')['net_order_amt'].agg([('order_amt_sum',np.sum),('order_amt_mean',np.mean),
                                                          ('order_amt_max',np.max),('order_amt_min',np.min),('order_amt_std',np.std),
                                                          ('order_amt_lam', lambda x : np.std(x)/np.mean(x))]).reset_index()

emp_tr = pd.merge(emp_tr, f_tr, on = 'order_date', how='left')
nemp_tr = pd.merge(nemp_tr, nf_tr, on = 'order_date', how='left')
emp_te = pd.merge(emp_te, f_te, on = 'order_date', how='left')
nemp_te = pd.merge(nemp_te, nf_te, on = 'order_date', how='left')

display(f_tr, nf_tr, f_te, nf_te)

,order_date,order_amt_sum,order_amt_mean,order_amt_max,order_amt_min,order_amt_std,order_amt_lam
0,2023-01-01,24152.389374,9.350519,14.475993,6.398595,1.059660,0.113304
1,2023-01-02,11173.333082,9.707501,13.927475,6.803505,1.199277,0.123488
2,2023-01-03,3073.021241,9.426446,14.765809,7.199678,1.120984,0.118737
3,2023-01-04,631.248804,9.421624,12.629381,7.247793,1.108639,0.116788
4,2023-01-05,236.630354,8.451084,10.397360,7.487174,0.506712,0.058878
5,2023-01-06,442.765694,9.224285,13.688244,7.177019,1.222785,0.131173
6,2023-01-07,47.450623,9.490125,10.005006,8.831566,0.429083,0.040440
7,2023-01-08,189.582180,9.479109,14.263333,7.894691,1.771625,0.182166
8,2023-01-09,347.645625,9.148569,12.186885,7.701652,0.930330,0.100344
9,2023-01-10,497.839378,9.956788,14.960234,7.892452,1.572851,0.156380


,order_date,order_amt_sum,order_amt_mean,order_amt_max,order_amt_min,order_amt_std,order_amt_lam
0,2023-01-01,33186.169784,8.837861,13.908243,6.368187,0.798415,0.090328
1,2023-01-02,12606.200804,8.890128,14.674198,6.705639,0.926187,0.104145
2,2023-01-03,4635.100267,8.745472,13.695212,6.634633,0.839763,0.095932
3,2023-01-04,1029.639623,8.953388,11.271325,7.157735,0.898592,0.099926
4,2023-01-05,57.193914,11.438783,14.279256,10.400650,1.682081,0.131526
5,2023-01-06,99.125219,9.011384,10.516969,8.289288,0.919711,0.097311
6,2023-01-07,25.507394,8.502465,8.519590,8.468213,0.029663,0.002849
7,2023-01-08,80.695505,8.966167,9.912893,8.119101,0.672358,0.070700
8,2023-01-09,210.480764,9.151338,15.647347,8.022569,1.604174,0.171441
9,2023-01-10,469.264662,9.024320,14.703951,7.402452,1.590649,0.174559


,order_date,order_amt_sum,order_amt_mean,order_amt_max,order_amt_min,order_amt_std,order_amt_lam
0,2023-01-01,16094.917815,9.287316,13.756107,6.447306,1.014246,0.109176
1,2023-01-02,14131.069924,9.414437,14.085958,6.937314,1.105816,0.117420
2,2023-01-03,5675.815179,9.669191,14.010500,7.199678,1.208723,0.124901
3,2023-01-04,1025.077910,9.762647,13.115290,7.287561,1.249554,0.127382
4,2023-01-05,322.335542,8.953765,11.097425,6.975414,0.977459,0.107641
5,2023-01-09,21.590711,10.795355,11.560238,10.030473,1.081708,0.070853
6,2023-01-10,190.953623,9.547681,11.735229,7.955074,1.308046,0.133532
7,2023-01-11,269.003641,9.275988,11.552002,7.266129,0.858277,0.090917
8,2023-01-12,1220.086932,9.313641,12.839817,6.956545,1.191175,0.127407
9,2023-01-13,3136.357686,9.710086,13.591097,7.228388,1.196998,0.123083


,order_date,order_amt_sum,order_amt_mean,order_amt_max,order_amt_min,order_amt_std,order_amt_lam
0,2023-01-01,29464.822040,8.748463,13.975836,6.771936,0.710914,0.081249
1,2023-01-02,19388.016129,8.760965,14.674198,6.552508,0.834438,0.095224
2,2023-01-03,5745.199804,9.005015,14.646290,6.643790,1.159895,0.128704
3,2023-01-04,716.689642,9.188329,13.094994,7.742402,1.033834,0.111792
4,2023-01-05,46.549257,9.309851,10.693784,7.976939,1.218683,0.117083
5,2023-01-07,19.019713,9.509857,9.667005,9.352708,0.222242,0.016525
6,2023-01-09,143.060616,8.941288,11.699496,7.600402,1.133802,0.122779
7,2023-01-10,166.028703,9.223817,14.135050,7.920083,1.675335,0.176514
8,2023-01-11,102.435055,9.312278,13.233225,7.078342,1.968395,0.201539
9,2023-01-12,894.389268,8.599897,14.635427,6.901737,1.603341,0.185539


#### 주문수량 x 주문금액

In [13]:
# train
emp_tr['qty_x_amt'] = emp_tr['net_order_qty'] * emp_tr['net_order_amt']
nemp_tr['qty_x_amt'] = nemp_tr['net_order_qty'] * nemp_tr['net_order_amt']

#test
emp_te['qty_x_amt'] = emp_te['net_order_qty'] * emp_te['net_order_amt']
nemp_te['qty_x_amt'] = nemp_te['net_order_qty'] * nemp_te['net_order_amt']

display(emp_tr, nemp_tr, emp_te, nemp_te)

,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,order_qty_max,order_qty_std,order_qty_lam,order_amt_sum,order_amt_mean,order_amt_max,order_amt_min,order_amt_std,order_amt_lam,qty_x_amt
0,20230124153976,잔칫집 식혜 240ml 30입,1,9.803170,F,2,2023-01-24,0,0,5661,...,10,1.245392,0.846292,4666.655298,9.296126,12.641294,7.160846,0.840252,0.090297,9.803170
1,20230124155563,백설 한입쏙 비엔나 120g*2,1,8.256607,M,3,2023-01-24,0,0,14016,...,10,1.245392,0.846292,4666.655298,9.296126,12.641294,7.160846,0.840252,0.090297,8.256607
2,20230125160921,맥스봉 체다치즈어랏400g (20g*20EA),1,8.974998,F,2,2023-01-25,0,0,5661,...,24,1.663238,1.106552,9813.672541,9.302059,12.583337,7.273786,0.787251,0.084592,8.974998
3,20230125157569,고메 바삭쫄깃한 사천탕수육 450g,1,8.938007,M,4,2023-01-25,1,0,9878,...,24,1.663238,1.106552,9813.672541,9.302059,12.583337,7.273786,0.787251,0.084592,8.938007
4,20230125156655,고메 콘크림수프 180g*4입,1,8.997518,F,3,2023-01-25,0,0,14016,...,24,1.663238,1.106552,9813.672541,9.302059,12.583337,7.273786,0.787251,0.084592,8.997518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19800,20230102975115,[2023설사원선물신청] 흑삼정 골드클래스 100gx2,1,11.597276,M,5,2023-01-02,0,0,5196,...,50,3.273962,1.444284,11173.333082,9.707501,13.927475,6.803505,1.199277,0.123488,11.597276
19801,20230102972321,고메 바삭쫄깃한 탕수육 450g,1,8.702510,M,5,2023-01-02,1,0,5196,...,50,3.273962,1.444284,11173.333082,9.707501,13.927475,6.803505,1.199277,0.123488,8.702510
19802,20230101969615,리턴업 전립소 쏘팔메토 골드 1000mgX60캡슐(2개월)X2개,1,11.364344,F,2,2023-01-01,1,0,5661,...,35,2.370647,1.214962,24152.389374,9.350519,14.475993,6.398595,1.059660,0.113304,11.364344
19803,20230102972321,고메 거멍 모짜체다핫도그 340g,1,8.648397,M,5,2023-01-02,1,0,5196,...,50,3.273962,1.444284,11173.333082,9.707501,13.927475,6.803505,1.199277,0.123488,8.648397


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,order_qty_max,order_qty_std,order_qty_lam,order_amt_sum,order_amt_mean,order_amt_max,order_amt_min,order_amt_std,order_amt_lam,qty_x_amt
0,20230125158386,비비고 왕교자 1.05kg,1,9.348449,F,4,2023-01-25,0,0,20945,...,20,1.702446,1.091935,7739.167204,8.834666,11.713218,6.388561,0.763573,0.086380,9.348449
1,20230126164638,고메 바삭쫄깃한 탕수육 900g,1,9.667259,F,4,2023-01-26,1,0,20945,...,23,0.983240,0.798073,52784.248999,9.114876,12.026773,6.455199,0.609696,0.066884,9.667259
2,20230125159705,햇반 매일잡곡밥210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,20,1.702446,1.091935,7739.167204,8.834666,11.713218,6.388561,0.763573,0.086380,199.893056
3,20230126167696,[UPCYCLE]햇반 흑미밥 210gx24개(1box),1,10.125110,F,3,2023-01-26,0,1,10109,...,23,0.983240,0.798073,52784.248999,9.114876,12.026773,6.455199,0.609696,0.066884,10.125110
4,20230125159705,햇반 매일찰잡곡밥 210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,20,1.702446,1.091935,7739.167204,8.834666,11.713218,6.388561,0.763573,0.086380,199.893056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26065,20230101966635,고메 순살크리스피 치킨 고추크런치 450g,1,8.659560,M,3,2023-01-01,0,0,10109,...,50,2.378074,1.313011,33186.169784,8.837861,13.908243,6.368187,0.798415,0.090328,8.659560
26066,20230102974273,비비고 사골순댓국 460g,10,10.518646,M,4,2023-01-02,1,0,20945,...,50,3.337271,1.720837,12606.200804,8.890128,14.674198,6.705639,0.926187,0.104145,105.186462
26067,20230102972720,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,3,2023-01-02,0,0,10109,...,50,3.337271,1.720837,12606.200804,8.890128,14.674198,6.705639,0.926187,0.104145,9.639327
26068,20230101964953,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,4,2023-01-01,0,0,20945,...,50,2.378074,1.313011,33186.169784,8.837861,13.908243,6.368187,0.798415,0.090328,9.639327


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,order_qty_max,order_qty_std,order_qty_lam,order_amt_sum,order_amt_mean,order_amt_max,order_amt_min,order_amt_std,order_amt_lam,qty_x_amt
0,20230101970142,삼호 생선살어묵 야채 200g,1,8.098947,M,4,2023-01-01,NaN,0,4344,...,50,2.586050,1.335021,16094.917815,9.287316,13.756107,6.447306,1.014246,0.109176,8.098947
1,20230101964512,[앱전용특가] 쁘띠첼밀감90gX6개X3개,1,9.076809,M,4,2023-01-01,NaN,0,4344,...,50,2.586050,1.335021,16094.917815,9.287316,13.756107,6.447306,1.014246,0.109176,9.076809
2,20230102975046,빕스 볶음밥 1인분 X 6종_행사,1,9.928522,F,3,2023-01-02,NaN,1,6299,...,50,3.199318,1.484867,14131.069924,9.414437,14.085958,6.937314,1.105816,0.117420,9.928522
3,20230102973138,백설 진한참기름 500ml,2,10.060534,M,4,2023-01-02,NaN,0,4344,...,50,3.199318,1.484867,14131.069924,9.414437,14.085958,6.937314,1.105816,0.117420,20.121068
4,20230101969192,삼호 가쓰오모둠어묵탕413g,2,9.796181,M,4,2023-01-01,NaN,0,4344,...,50,2.586050,1.335021,16094.917815,9.287316,13.756107,6.447306,1.014246,0.109176,19.592361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8041,20230103994630,고메 콰트로포르마지 피자 310g,1,8.736650,M,5,2023-01-03,NaN,0,2051,...,50,3.945099,1.561268,5675.815179,9.669191,14.010500,7.199678,1.208723,0.124901,8.736650
8042,20230103994130,햇반솥반 전복내장영양밥 200g,2,9.196748,M,3,2023-01-03,NaN,1,6299,...,50,3.945099,1.561268,5675.815179,9.669191,14.010500,7.199678,1.208723,0.124901,18.393497
8043,20230103986821,비비고 찐만두168g,1,7.930566,M,5,2023-01-03,NaN,0,2051,...,50,3.945099,1.561268,5675.815179,9.669191,14.010500,7.199678,1.208723,0.124901,7.930566
8044,20230102979023,사골곰탕 한 그릇 100g (1-2인분X5개입),1,8.477412,F,3,2023-01-02,NaN,0,6299,...,50,3.199318,1.484867,14131.069924,9.414437,14.085958,6.937314,1.105816,0.117420,8.477412


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,order_qty_max,order_qty_std,order_qty_lam,order_amt_sum,order_amt_mean,order_amt_max,order_amt_min,order_amt_std,order_amt_lam,qty_x_amt
0,20230101964282,비비고 스팸부대찌개 460g,1,8.161946,M,4,2023-01-01,NaN,0,9748,...,50,1.952196,1.145499,29464.822040,8.748463,13.975836,6.771936,0.710914,0.081249,8.161946
1,20230101965237,크레잇 블랙페퍼 폭찹 스테이크,1,8.277412,F,4,2023-01-01,NaN,0,9748,...,50,1.952196,1.145499,29464.822040,8.748463,13.975836,6.771936,0.710914,0.081249,8.277412
2,20230102973798,비비고 소고기 미역국 500g,3,9.145375,F,3,2023-01-02,NaN,0,5034,...,50,2.486807,1.330929,19388.016129,8.760965,14.674198,6.552508,0.834438,0.095224,27.436125
3,20230101965633,비비고 소고기 미역국 500g,2,8.613230,F,3,2023-01-01,NaN,0,5034,...,50,1.952196,1.145499,29464.822040,8.748463,13.975836,6.771936,0.710914,0.081249,17.226461
4,20230101963753,고메 치즈크리스피핫도그 340g,1,8.540714,F,4,2023-01-01,NaN,0,9748,...,50,1.952196,1.145499,29464.822040,8.748463,13.975836,6.771936,0.710914,0.081249,8.540714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11609,20230102980326,고메 케이준프라이즈 피자 145g,2,8.671115,F,4,2023-01-02,NaN,0,9748,...,50,2.486807,1.330929,19388.016129,8.760965,14.674198,6.552508,0.834438,0.095224,17.342231
11610,20230102983068,고메 빅크리스피핫도그520g,1,8.641179,F,3,2023-01-02,NaN,0,5034,...,50,2.486807,1.330929,19388.016129,8.760965,14.674198,6.552508,0.834438,0.095224,8.641179
11611,20230103992835,비비고 메추리알장조림 170g,7,9.467692,F,4,2023-01-03,NaN,0,9748,...,49,3.794087,1.637596,5745.199804,9.005015,14.646290,6.643790,1.159895,0.128704,66.273845
11612,20230104999357,비비고 언양식 바싹불고기 460g,1,8.594895,M,4,2023-01-04,NaN,0,9748,...,42,4.884974,1.921714,716.689642,9.188329,13.094994,7.742402,1.033834,0.111792,8.594895


#### 연령대별 평균 주문수량 x 구매일별 평균 주문수량

In [14]:
# train
emp_tr['age_qty_mean_x_order_qty_mean'] = emp_tr['age_qty_mean'] * emp_tr['order_qty_mean']
nemp_tr['age_qty_mean_x_order_qty_mean'] = nemp_tr['age_qty_mean'] * nemp_tr['order_qty_mean']

#test
emp_te['age_qty_mean_x_order_qty_mean'] = emp_te['age_qty_mean'] * emp_te['order_qty_mean']
nemp_te['age_qty_mean_x_order_qty_mean'] = nemp_te['age_qty_mean'] * nemp_te['order_qty_mean']

display(emp_tr, nemp_tr, emp_te, nemp_te)

,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,order_qty_std,order_qty_lam,order_amt_sum,order_amt_mean,order_amt_max,order_amt_min,order_amt_std,order_amt_lam,qty_x_amt,age_qty_mean_x_order_qty_mean
0,20230124153976,잔칫집 식혜 240ml 30입,1,9.803170,F,2,2023-01-24,0,0,5661,...,1.245392,0.846292,4666.655298,9.296126,12.641294,7.160846,0.840252,0.090297,9.803170,2.650429
1,20230124155563,백설 한입쏙 비엔나 120g*2,1,8.256607,M,3,2023-01-24,0,0,14016,...,1.245392,0.846292,4666.655298,9.296126,12.641294,7.160846,0.840252,0.090297,8.256607,2.645762
2,20230125160921,맥스봉 체다치즈어랏400g (20g*20EA),1,8.974998,F,2,2023-01-25,0,0,5661,...,1.663238,1.106552,9813.672541,9.302059,12.583337,7.273786,0.787251,0.084592,8.974998,2.708572
3,20230125157569,고메 바삭쫄깃한 사천탕수육 450g,1,8.938007,M,4,2023-01-25,1,0,9878,...,1.663238,1.106552,9813.672541,9.302059,12.583337,7.273786,0.787251,0.084592,8.938007,2.519166
4,20230125156655,고메 콘크림수프 180g*4입,1,8.997518,F,3,2023-01-25,0,0,14016,...,1.663238,1.106552,9813.672541,9.302059,12.583337,7.273786,0.787251,0.084592,8.997518,2.703802
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19800,20230102975115,[2023설사원선물신청] 흑삼정 골드클래스 100gx2,1,11.597276,M,5,2023-01-02,0,0,5196,...,3.273962,1.444284,11173.333082,9.707501,13.927475,6.803505,1.199277,0.123488,11.597276,4.312596
19801,20230102972321,고메 바삭쫄깃한 탕수육 450g,1,8.702510,M,5,2023-01-02,1,0,5196,...,3.273962,1.444284,11173.333082,9.707501,13.927475,6.803505,1.199277,0.123488,8.702510,4.312596
19802,20230101969615,리턴업 전립소 쏘팔메토 골드 1000mgX60캡슐(2개월)X2개,1,11.364344,F,2,2023-01-01,1,0,5661,...,2.370647,1.214962,24152.389374,9.350519,14.475993,6.398595,1.059660,0.113304,11.364344,3.517090
19803,20230102972321,고메 거멍 모짜체다핫도그 340g,1,8.648397,M,5,2023-01-02,1,0,5196,...,3.273962,1.444284,11173.333082,9.707501,13.927475,6.803505,1.199277,0.123488,8.648397,4.312596


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,order_qty_std,order_qty_lam,order_amt_sum,order_amt_mean,order_amt_max,order_amt_min,order_amt_std,order_amt_lam,qty_x_amt,age_qty_mean_x_order_qty_mean
0,20230125158386,비비고 왕교자 1.05kg,1,9.348449,F,4,2023-01-25,0,0,20945,...,1.702446,1.091935,7739.167204,8.834666,11.713218,6.388561,0.763573,0.086380,9.348449,2.488897
1,20230126164638,고메 바삭쫄깃한 탕수육 900g,1,9.667259,F,4,2023-01-26,1,0,20945,...,0.983240,0.798073,52784.248999,9.114876,12.026773,6.455199,0.609696,0.066884,9.667259,1.967695
2,20230125159705,햇반 매일잡곡밥210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,1.702446,1.091935,7739.167204,8.834666,11.713218,6.388561,0.763573,0.086380,199.893056,2.488897
3,20230126167696,[UPCYCLE]햇반 흑미밥 210gx24개(1box),1,10.125110,F,3,2023-01-26,0,1,10109,...,0.983240,0.798073,52784.248999,9.114876,12.026773,6.455199,0.609696,0.066884,10.125110,2.034536
4,20230125159705,햇반 매일찰잡곡밥 210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,1.702446,1.091935,7739.167204,8.834666,11.713218,6.388561,0.763573,0.086380,199.893056,2.488897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26065,20230101966635,고메 순살크리스피 치킨 고추크런치 450g,1,8.659560,M,3,2023-01-01,0,0,10109,...,2.378074,1.313011,33186.169784,8.837861,13.908243,6.368187,0.798415,0.090328,8.659560,2.990782
26066,20230102974273,비비고 사골순댓국 460g,10,10.518646,M,4,2023-01-02,1,0,20945,...,3.337271,1.720837,12606.200804,8.890128,14.674198,6.705639,0.926187,0.104145,105.186462,3.096541
26067,20230102972720,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,3,2023-01-02,0,0,10109,...,3.337271,1.720837,12606.200804,8.890128,14.674198,6.705639,0.926187,0.104145,9.639327,3.201727
26068,20230101964953,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,4,2023-01-01,0,0,20945,...,2.378074,1.313011,33186.169784,8.837861,13.908243,6.368187,0.798415,0.090328,9.639327,2.892526


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,order_qty_std,order_qty_lam,order_amt_sum,order_amt_mean,order_amt_max,order_amt_min,order_amt_std,order_amt_lam,qty_x_amt,age_qty_mean_x_order_qty_mean
0,20230101970142,삼호 생선살어묵 야채 200g,1,8.098947,M,4,2023-01-01,NaN,0,4344,...,2.586050,1.335021,16094.917815,9.287316,13.756107,6.447306,1.014246,0.109176,8.098947,3.544994
1,20230101964512,[앱전용특가] 쁘띠첼밀감90gX6개X3개,1,9.076809,M,4,2023-01-01,NaN,0,4344,...,2.586050,1.335021,16094.917815,9.287316,13.756107,6.447306,1.014246,0.109176,9.076809,3.544994
2,20230102975046,빕스 볶음밥 1인분 X 6종_행사,1,9.928522,F,3,2023-01-02,NaN,1,6299,...,3.199318,1.484867,14131.069924,9.414437,14.085958,6.937314,1.105816,0.117420,9.928522,4.143983
3,20230102973138,백설 진한참기름 500ml,2,10.060534,M,4,2023-01-02,NaN,0,4344,...,3.199318,1.484867,14131.069924,9.414437,14.085958,6.937314,1.105816,0.117420,20.121068,3.942912
4,20230101969192,삼호 가쓰오모둠어묵탕413g,2,9.796181,M,4,2023-01-01,NaN,0,4344,...,2.586050,1.335021,16094.917815,9.287316,13.756107,6.447306,1.014246,0.109176,19.592361,3.544994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8041,20230103994630,고메 콰트로포르마지 피자 310g,1,8.736650,M,5,2023-01-03,NaN,0,2051,...,3.945099,1.561268,5675.815179,9.669191,14.010500,7.199678,1.208723,0.124901,8.736650,5.032229
8042,20230103994130,햇반솥반 전복내장영양밥 200g,2,9.196748,M,3,2023-01-03,NaN,1,6299,...,3.945099,1.561268,5675.815179,9.669191,14.010500,7.199678,1.208723,0.124901,18.393497,4.857391
8043,20230103986821,비비고 찐만두168g,1,7.930566,M,5,2023-01-03,NaN,0,2051,...,3.945099,1.561268,5675.815179,9.669191,14.010500,7.199678,1.208723,0.124901,7.930566,5.032229
8044,20230102979023,사골곰탕 한 그릇 100g (1-2인분X5개입),1,8.477412,F,3,2023-01-02,NaN,0,6299,...,3.199318,1.484867,14131.069924,9.414437,14.085958,6.937314,1.105816,0.117420,8.477412,4.143983


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,order_qty_std,order_qty_lam,order_amt_sum,order_amt_mean,order_amt_max,order_amt_min,order_amt_std,order_amt_lam,qty_x_amt,age_qty_mean_x_order_qty_mean
0,20230101964282,비비고 스팸부대찌개 460g,1,8.161946,M,4,2023-01-01,NaN,0,9748,...,1.952196,1.145499,29464.822040,8.748463,13.975836,6.771936,0.710914,0.081249,8.161946,2.854018
1,20230101965237,크레잇 블랙페퍼 폭찹 스테이크,1,8.277412,F,4,2023-01-01,NaN,0,9748,...,1.952196,1.145499,29464.822040,8.748463,13.975836,6.771936,0.710914,0.081249,8.277412,2.854018
2,20230102973798,비비고 소고기 미역국 500g,3,9.145375,F,3,2023-01-02,NaN,0,5034,...,2.486807,1.330929,19388.016129,8.760965,14.674198,6.552508,0.834438,0.095224,27.436125,3.084216
3,20230101965633,비비고 소고기 미역국 500g,2,8.613230,F,3,2023-01-01,NaN,0,5034,...,1.952196,1.145499,29464.822040,8.748463,13.975836,6.771936,0.710914,0.081249,17.226461,2.813325
4,20230101963753,고메 치즈크리스피핫도그 340g,1,8.540714,F,4,2023-01-01,NaN,0,9748,...,1.952196,1.145499,29464.822040,8.748463,13.975836,6.771936,0.710914,0.081249,8.540714,2.854018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11609,20230102980326,고메 케이준프라이즈 피자 145g,2,8.671115,F,4,2023-01-02,NaN,0,9748,...,2.486807,1.330929,19388.016129,8.760965,14.674198,6.552508,0.834438,0.095224,17.342231,3.128827
11610,20230102983068,고메 빅크리스피핫도그520g,1,8.641179,F,3,2023-01-02,NaN,0,5034,...,2.486807,1.330929,19388.016129,8.760965,14.674198,6.552508,0.834438,0.095224,8.641179,3.084216
11611,20230103992835,비비고 메추리알장조림 170g,7,9.467692,F,4,2023-01-03,NaN,0,9748,...,3.794087,1.637596,5745.199804,9.005015,14.646290,6.643790,1.159895,0.128704,66.273845,3.877505
11612,20230104999357,비비고 언양식 바싹불고기 460g,1,8.594895,M,4,2023-01-04,NaN,0,9748,...,4.884974,1.921714,716.689642,9.188329,13.094994,7.742402,1.033834,0.111792,8.594895,4.230232


#### 연령대별 최대 주문수량 x 구매일별 최대 주문수량

In [15]:
# train
emp_tr['order_amt_max_x_order_qty_max'] = emp_tr['order_amt_max'] * emp_tr['order_qty_max']
nemp_tr['order_amt_max_x_order_qty_max'] = nemp_tr['order_amt_max'] * nemp_tr['order_qty_max']

#test
emp_te['order_amt_max_x_order_qty_max'] = emp_te['order_amt_max'] * emp_te['order_qty_max']
nemp_te['order_amt_max_x_order_qty_max'] = nemp_te['order_amt_max'] * nemp_te['order_qty_max']

display(emp_tr, nemp_tr, emp_te, nemp_te)

,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,order_qty_lam,order_amt_sum,order_amt_mean,order_amt_max,order_amt_min,order_amt_std,order_amt_lam,qty_x_amt,age_qty_mean_x_order_qty_mean,order_amt_max_x_order_qty_max
0,20230124153976,잔칫집 식혜 240ml 30입,1,9.803170,F,2,2023-01-24,0,0,5661,...,0.846292,4666.655298,9.296126,12.641294,7.160846,0.840252,0.090297,9.803170,2.650429,126.412939
1,20230124155563,백설 한입쏙 비엔나 120g*2,1,8.256607,M,3,2023-01-24,0,0,14016,...,0.846292,4666.655298,9.296126,12.641294,7.160846,0.840252,0.090297,8.256607,2.645762,126.412939
2,20230125160921,맥스봉 체다치즈어랏400g (20g*20EA),1,8.974998,F,2,2023-01-25,0,0,5661,...,1.106552,9813.672541,9.302059,12.583337,7.273786,0.787251,0.084592,8.974998,2.708572,302.000092
3,20230125157569,고메 바삭쫄깃한 사천탕수육 450g,1,8.938007,M,4,2023-01-25,1,0,9878,...,1.106552,9813.672541,9.302059,12.583337,7.273786,0.787251,0.084592,8.938007,2.519166,302.000092
4,20230125156655,고메 콘크림수프 180g*4입,1,8.997518,F,3,2023-01-25,0,0,14016,...,1.106552,9813.672541,9.302059,12.583337,7.273786,0.787251,0.084592,8.997518,2.703802,302.000092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19800,20230102975115,[2023설사원선물신청] 흑삼정 골드클래스 100gx2,1,11.597276,M,5,2023-01-02,0,0,5196,...,1.444284,11173.333082,9.707501,13.927475,6.803505,1.199277,0.123488,11.597276,4.312596,696.373746
19801,20230102972321,고메 바삭쫄깃한 탕수육 450g,1,8.702510,M,5,2023-01-02,1,0,5196,...,1.444284,11173.333082,9.707501,13.927475,6.803505,1.199277,0.123488,8.702510,4.312596,696.373746
19802,20230101969615,리턴업 전립소 쏘팔메토 골드 1000mgX60캡슐(2개월)X2개,1,11.364344,F,2,2023-01-01,1,0,5661,...,1.214962,24152.389374,9.350519,14.475993,6.398595,1.059660,0.113304,11.364344,3.517090,506.659755
19803,20230102972321,고메 거멍 모짜체다핫도그 340g,1,8.648397,M,5,2023-01-02,1,0,5196,...,1.444284,11173.333082,9.707501,13.927475,6.803505,1.199277,0.123488,8.648397,4.312596,696.373746


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,order_qty_lam,order_amt_sum,order_amt_mean,order_amt_max,order_amt_min,order_amt_std,order_amt_lam,qty_x_amt,age_qty_mean_x_order_qty_mean,order_amt_max_x_order_qty_max
0,20230125158386,비비고 왕교자 1.05kg,1,9.348449,F,4,2023-01-25,0,0,20945,...,1.091935,7739.167204,8.834666,11.713218,6.388561,0.763573,0.086380,9.348449,2.488897,234.264358
1,20230126164638,고메 바삭쫄깃한 탕수육 900g,1,9.667259,F,4,2023-01-26,1,0,20945,...,0.798073,52784.248999,9.114876,12.026773,6.455199,0.609696,0.066884,9.667259,1.967695,276.615768
2,20230125159705,햇반 매일잡곡밥210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,1.091935,7739.167204,8.834666,11.713218,6.388561,0.763573,0.086380,199.893056,2.488897,234.264358
3,20230126167696,[UPCYCLE]햇반 흑미밥 210gx24개(1box),1,10.125110,F,3,2023-01-26,0,1,10109,...,0.798073,52784.248999,9.114876,12.026773,6.455199,0.609696,0.066884,10.125110,2.034536,276.615768
4,20230125159705,햇반 매일찰잡곡밥 210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,1.091935,7739.167204,8.834666,11.713218,6.388561,0.763573,0.086380,199.893056,2.488897,234.264358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26065,20230101966635,고메 순살크리스피 치킨 고추크런치 450g,1,8.659560,M,3,2023-01-01,0,0,10109,...,1.313011,33186.169784,8.837861,13.908243,6.368187,0.798415,0.090328,8.659560,2.990782,695.412144
26066,20230102974273,비비고 사골순댓국 460g,10,10.518646,M,4,2023-01-02,1,0,20945,...,1.720837,12606.200804,8.890128,14.674198,6.705639,0.926187,0.104145,105.186462,3.096541,733.709901
26067,20230102972720,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,3,2023-01-02,0,0,10109,...,1.720837,12606.200804,8.890128,14.674198,6.705639,0.926187,0.104145,9.639327,3.201727,733.709901
26068,20230101964953,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,4,2023-01-01,0,0,20945,...,1.313011,33186.169784,8.837861,13.908243,6.368187,0.798415,0.090328,9.639327,2.892526,695.412144


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,order_qty_lam,order_amt_sum,order_amt_mean,order_amt_max,order_amt_min,order_amt_std,order_amt_lam,qty_x_amt,age_qty_mean_x_order_qty_mean,order_amt_max_x_order_qty_max
0,20230101970142,삼호 생선살어묵 야채 200g,1,8.098947,M,4,2023-01-01,NaN,0,4344,...,1.335021,16094.917815,9.287316,13.756107,6.447306,1.014246,0.109176,8.098947,3.544994,687.805328
1,20230101964512,[앱전용특가] 쁘띠첼밀감90gX6개X3개,1,9.076809,M,4,2023-01-01,NaN,0,4344,...,1.335021,16094.917815,9.287316,13.756107,6.447306,1.014246,0.109176,9.076809,3.544994,687.805328
2,20230102975046,빕스 볶음밥 1인분 X 6종_행사,1,9.928522,F,3,2023-01-02,NaN,1,6299,...,1.484867,14131.069924,9.414437,14.085958,6.937314,1.105816,0.117420,9.928522,4.143983,704.297911
3,20230102973138,백설 진한참기름 500ml,2,10.060534,M,4,2023-01-02,NaN,0,4344,...,1.484867,14131.069924,9.414437,14.085958,6.937314,1.105816,0.117420,20.121068,3.942912,704.297911
4,20230101969192,삼호 가쓰오모둠어묵탕413g,2,9.796181,M,4,2023-01-01,NaN,0,4344,...,1.335021,16094.917815,9.287316,13.756107,6.447306,1.014246,0.109176,19.592361,3.544994,687.805328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8041,20230103994630,고메 콰트로포르마지 피자 310g,1,8.736650,M,5,2023-01-03,NaN,0,2051,...,1.561268,5675.815179,9.669191,14.010500,7.199678,1.208723,0.124901,8.736650,5.032229,700.524994
8042,20230103994130,햇반솥반 전복내장영양밥 200g,2,9.196748,M,3,2023-01-03,NaN,1,6299,...,1.561268,5675.815179,9.669191,14.010500,7.199678,1.208723,0.124901,18.393497,4.857391,700.524994
8043,20230103986821,비비고 찐만두168g,1,7.930566,M,5,2023-01-03,NaN,0,2051,...,1.561268,5675.815179,9.669191,14.010500,7.199678,1.208723,0.124901,7.930566,5.032229,700.524994
8044,20230102979023,사골곰탕 한 그릇 100g (1-2인분X5개입),1,8.477412,F,3,2023-01-02,NaN,0,6299,...,1.484867,14131.069924,9.414437,14.085958,6.937314,1.105816,0.117420,8.477412,4.143983,704.297911


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,order_qty_lam,order_amt_sum,order_amt_mean,order_amt_max,order_amt_min,order_amt_std,order_amt_lam,qty_x_amt,age_qty_mean_x_order_qty_mean,order_amt_max_x_order_qty_max
0,20230101964282,비비고 스팸부대찌개 460g,1,8.161946,M,4,2023-01-01,NaN,0,9748,...,1.145499,29464.822040,8.748463,13.975836,6.771936,0.710914,0.081249,8.161946,2.854018,698.791807
1,20230101965237,크레잇 블랙페퍼 폭찹 스테이크,1,8.277412,F,4,2023-01-01,NaN,0,9748,...,1.145499,29464.822040,8.748463,13.975836,6.771936,0.710914,0.081249,8.277412,2.854018,698.791807
2,20230102973798,비비고 소고기 미역국 500g,3,9.145375,F,3,2023-01-02,NaN,0,5034,...,1.330929,19388.016129,8.760965,14.674198,6.552508,0.834438,0.095224,27.436125,3.084216,733.709901
3,20230101965633,비비고 소고기 미역국 500g,2,8.613230,F,3,2023-01-01,NaN,0,5034,...,1.145499,29464.822040,8.748463,13.975836,6.771936,0.710914,0.081249,17.226461,2.813325,698.791807
4,20230101963753,고메 치즈크리스피핫도그 340g,1,8.540714,F,4,2023-01-01,NaN,0,9748,...,1.145499,29464.822040,8.748463,13.975836,6.771936,0.710914,0.081249,8.540714,2.854018,698.791807
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11609,20230102980326,고메 케이준프라이즈 피자 145g,2,8.671115,F,4,2023-01-02,NaN,0,9748,...,1.330929,19388.016129,8.760965,14.674198,6.552508,0.834438,0.095224,17.342231,3.128827,733.709901
11610,20230102983068,고메 빅크리스피핫도그520g,1,8.641179,F,3,2023-01-02,NaN,0,5034,...,1.330929,19388.016129,8.760965,14.674198,6.552508,0.834438,0.095224,8.641179,3.084216,733.709901
11611,20230103992835,비비고 메추리알장조림 170g,7,9.467692,F,4,2023-01-03,NaN,0,9748,...,1.637596,5745.199804,9.005015,14.646290,6.643790,1.159895,0.128704,66.273845,3.877505,717.668190
11612,20230104999357,비비고 언양식 바싹불고기 460g,1,8.594895,M,4,2023-01-04,NaN,0,9748,...,1.921714,716.689642,9.188329,13.094994,7.742402,1.033834,0.111792,8.594895,4.230232,549.989758


#### 주문일자에서 day만 추출

In [16]:
# train
emp_tr['day']= emp_tr['order_date'].dt.day
nemp_tr['day']= nemp_tr['order_date'].dt.day

#test
emp_te['day']= emp_te['order_date'].dt.day
nemp_te['day']= nemp_te['order_date'].dt.day

display(emp_tr, nemp_tr, emp_te, nemp_te)

,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,order_amt_sum,order_amt_mean,order_amt_max,order_amt_min,order_amt_std,order_amt_lam,qty_x_amt,age_qty_mean_x_order_qty_mean,order_amt_max_x_order_qty_max,day
0,20230124153976,잔칫집 식혜 240ml 30입,1,9.803170,F,2,2023-01-24,0,0,5661,...,4666.655298,9.296126,12.641294,7.160846,0.840252,0.090297,9.803170,2.650429,126.412939,24
1,20230124155563,백설 한입쏙 비엔나 120g*2,1,8.256607,M,3,2023-01-24,0,0,14016,...,4666.655298,9.296126,12.641294,7.160846,0.840252,0.090297,8.256607,2.645762,126.412939,24
2,20230125160921,맥스봉 체다치즈어랏400g (20g*20EA),1,8.974998,F,2,2023-01-25,0,0,5661,...,9813.672541,9.302059,12.583337,7.273786,0.787251,0.084592,8.974998,2.708572,302.000092,25
3,20230125157569,고메 바삭쫄깃한 사천탕수육 450g,1,8.938007,M,4,2023-01-25,1,0,9878,...,9813.672541,9.302059,12.583337,7.273786,0.787251,0.084592,8.938007,2.519166,302.000092,25
4,20230125156655,고메 콘크림수프 180g*4입,1,8.997518,F,3,2023-01-25,0,0,14016,...,9813.672541,9.302059,12.583337,7.273786,0.787251,0.084592,8.997518,2.703802,302.000092,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19800,20230102975115,[2023설사원선물신청] 흑삼정 골드클래스 100gx2,1,11.597276,M,5,2023-01-02,0,0,5196,...,11173.333082,9.707501,13.927475,6.803505,1.199277,0.123488,11.597276,4.312596,696.373746,2
19801,20230102972321,고메 바삭쫄깃한 탕수육 450g,1,8.702510,M,5,2023-01-02,1,0,5196,...,11173.333082,9.707501,13.927475,6.803505,1.199277,0.123488,8.702510,4.312596,696.373746,2
19802,20230101969615,리턴업 전립소 쏘팔메토 골드 1000mgX60캡슐(2개월)X2개,1,11.364344,F,2,2023-01-01,1,0,5661,...,24152.389374,9.350519,14.475993,6.398595,1.059660,0.113304,11.364344,3.517090,506.659755,1
19803,20230102972321,고메 거멍 모짜체다핫도그 340g,1,8.648397,M,5,2023-01-02,1,0,5196,...,11173.333082,9.707501,13.927475,6.803505,1.199277,0.123488,8.648397,4.312596,696.373746,2


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,order_amt_sum,order_amt_mean,order_amt_max,order_amt_min,order_amt_std,order_amt_lam,qty_x_amt,age_qty_mean_x_order_qty_mean,order_amt_max_x_order_qty_max,day
0,20230125158386,비비고 왕교자 1.05kg,1,9.348449,F,4,2023-01-25,0,0,20945,...,7739.167204,8.834666,11.713218,6.388561,0.763573,0.086380,9.348449,2.488897,234.264358,25
1,20230126164638,고메 바삭쫄깃한 탕수육 900g,1,9.667259,F,4,2023-01-26,1,0,20945,...,52784.248999,9.114876,12.026773,6.455199,0.609696,0.066884,9.667259,1.967695,276.615768,26
2,20230125159705,햇반 매일잡곡밥210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,7739.167204,8.834666,11.713218,6.388561,0.763573,0.086380,199.893056,2.488897,234.264358,25
3,20230126167696,[UPCYCLE]햇반 흑미밥 210gx24개(1box),1,10.125110,F,3,2023-01-26,0,1,10109,...,52784.248999,9.114876,12.026773,6.455199,0.609696,0.066884,10.125110,2.034536,276.615768,26
4,20230125159705,햇반 매일찰잡곡밥 210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,7739.167204,8.834666,11.713218,6.388561,0.763573,0.086380,199.893056,2.488897,234.264358,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26065,20230101966635,고메 순살크리스피 치킨 고추크런치 450g,1,8.659560,M,3,2023-01-01,0,0,10109,...,33186.169784,8.837861,13.908243,6.368187,0.798415,0.090328,8.659560,2.990782,695.412144,1
26066,20230102974273,비비고 사골순댓국 460g,10,10.518646,M,4,2023-01-02,1,0,20945,...,12606.200804,8.890128,14.674198,6.705639,0.926187,0.104145,105.186462,3.096541,733.709901,2
26067,20230102972720,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,3,2023-01-02,0,0,10109,...,12606.200804,8.890128,14.674198,6.705639,0.926187,0.104145,9.639327,3.201727,733.709901,2
26068,20230101964953,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,4,2023-01-01,0,0,20945,...,33186.169784,8.837861,13.908243,6.368187,0.798415,0.090328,9.639327,2.892526,695.412144,1


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,order_amt_sum,order_amt_mean,order_amt_max,order_amt_min,order_amt_std,order_amt_lam,qty_x_amt,age_qty_mean_x_order_qty_mean,order_amt_max_x_order_qty_max,day
0,20230101970142,삼호 생선살어묵 야채 200g,1,8.098947,M,4,2023-01-01,NaN,0,4344,...,16094.917815,9.287316,13.756107,6.447306,1.014246,0.109176,8.098947,3.544994,687.805328,1
1,20230101964512,[앱전용특가] 쁘띠첼밀감90gX6개X3개,1,9.076809,M,4,2023-01-01,NaN,0,4344,...,16094.917815,9.287316,13.756107,6.447306,1.014246,0.109176,9.076809,3.544994,687.805328,1
2,20230102975046,빕스 볶음밥 1인분 X 6종_행사,1,9.928522,F,3,2023-01-02,NaN,1,6299,...,14131.069924,9.414437,14.085958,6.937314,1.105816,0.117420,9.928522,4.143983,704.297911,2
3,20230102973138,백설 진한참기름 500ml,2,10.060534,M,4,2023-01-02,NaN,0,4344,...,14131.069924,9.414437,14.085958,6.937314,1.105816,0.117420,20.121068,3.942912,704.297911,2
4,20230101969192,삼호 가쓰오모둠어묵탕413g,2,9.796181,M,4,2023-01-01,NaN,0,4344,...,16094.917815,9.287316,13.756107,6.447306,1.014246,0.109176,19.592361,3.544994,687.805328,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8041,20230103994630,고메 콰트로포르마지 피자 310g,1,8.736650,M,5,2023-01-03,NaN,0,2051,...,5675.815179,9.669191,14.010500,7.199678,1.208723,0.124901,8.736650,5.032229,700.524994,3
8042,20230103994130,햇반솥반 전복내장영양밥 200g,2,9.196748,M,3,2023-01-03,NaN,1,6299,...,5675.815179,9.669191,14.010500,7.199678,1.208723,0.124901,18.393497,4.857391,700.524994,3
8043,20230103986821,비비고 찐만두168g,1,7.930566,M,5,2023-01-03,NaN,0,2051,...,5675.815179,9.669191,14.010500,7.199678,1.208723,0.124901,7.930566,5.032229,700.524994,3
8044,20230102979023,사골곰탕 한 그릇 100g (1-2인분X5개입),1,8.477412,F,3,2023-01-02,NaN,0,6299,...,14131.069924,9.414437,14.085958,6.937314,1.105816,0.117420,8.477412,4.143983,704.297911,2


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,order_amt_sum,order_amt_mean,order_amt_max,order_amt_min,order_amt_std,order_amt_lam,qty_x_amt,age_qty_mean_x_order_qty_mean,order_amt_max_x_order_qty_max,day
0,20230101964282,비비고 스팸부대찌개 460g,1,8.161946,M,4,2023-01-01,NaN,0,9748,...,29464.822040,8.748463,13.975836,6.771936,0.710914,0.081249,8.161946,2.854018,698.791807,1
1,20230101965237,크레잇 블랙페퍼 폭찹 스테이크,1,8.277412,F,4,2023-01-01,NaN,0,9748,...,29464.822040,8.748463,13.975836,6.771936,0.710914,0.081249,8.277412,2.854018,698.791807,1
2,20230102973798,비비고 소고기 미역국 500g,3,9.145375,F,3,2023-01-02,NaN,0,5034,...,19388.016129,8.760965,14.674198,6.552508,0.834438,0.095224,27.436125,3.084216,733.709901,2
3,20230101965633,비비고 소고기 미역국 500g,2,8.613230,F,3,2023-01-01,NaN,0,5034,...,29464.822040,8.748463,13.975836,6.771936,0.710914,0.081249,17.226461,2.813325,698.791807,1
4,20230101963753,고메 치즈크리스피핫도그 340g,1,8.540714,F,4,2023-01-01,NaN,0,9748,...,29464.822040,8.748463,13.975836,6.771936,0.710914,0.081249,8.540714,2.854018,698.791807,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11609,20230102980326,고메 케이준프라이즈 피자 145g,2,8.671115,F,4,2023-01-02,NaN,0,9748,...,19388.016129,8.760965,14.674198,6.552508,0.834438,0.095224,17.342231,3.128827,733.709901,2
11610,20230102983068,고메 빅크리스피핫도그520g,1,8.641179,F,3,2023-01-02,NaN,0,5034,...,19388.016129,8.760965,14.674198,6.552508,0.834438,0.095224,8.641179,3.084216,733.709901,2
11611,20230103992835,비비고 메추리알장조림 170g,7,9.467692,F,4,2023-01-03,NaN,0,9748,...,5745.199804,9.005015,14.646290,6.643790,1.159895,0.128704,66.273845,3.877505,717.668190,3
11612,20230104999357,비비고 언양식 바싹불고기 460g,1,8.594895,M,4,2023-01-04,NaN,0,9748,...,716.689642,9.188329,13.094994,7.742402,1.033834,0.111792,8.594895,4.230232,549.989758,4


#### 구매일의 월별 주차에 따른 Label Encoding

In [17]:
# train
emp_tr['day_label'] = emp_tr.day.apply(lambda x : 1 if x < 8 else (2 if x < 15 else (3 if x < 22 else (4 if x < 29 else 5))))
nemp_tr['day_label'] = nemp_tr.day.apply(lambda x : 1 if x < 8 else (2 if x < 15 else (3 if x < 22 else (4 if x < 29 else 5))))

#test
emp_te['day_label'] = emp_te.day.apply(lambda x : 1 if x < 8 else (2 if x < 15 else (3 if x < 22 else (4 if x < 29 else 5))))
nemp_te['day_label'] = nemp_te.day.apply(lambda x : 1 if x < 8 else (2 if x < 15 else (3 if x < 22 else (4 if x < 29 else 5))))

display(emp_tr, nemp_tr, emp_te, nemp_te)

,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,order_amt_mean,order_amt_max,order_amt_min,order_amt_std,order_amt_lam,qty_x_amt,age_qty_mean_x_order_qty_mean,order_amt_max_x_order_qty_max,day,day_label
0,20230124153976,잔칫집 식혜 240ml 30입,1,9.803170,F,2,2023-01-24,0,0,5661,...,9.296126,12.641294,7.160846,0.840252,0.090297,9.803170,2.650429,126.412939,24,4
1,20230124155563,백설 한입쏙 비엔나 120g*2,1,8.256607,M,3,2023-01-24,0,0,14016,...,9.296126,12.641294,7.160846,0.840252,0.090297,8.256607,2.645762,126.412939,24,4
2,20230125160921,맥스봉 체다치즈어랏400g (20g*20EA),1,8.974998,F,2,2023-01-25,0,0,5661,...,9.302059,12.583337,7.273786,0.787251,0.084592,8.974998,2.708572,302.000092,25,4
3,20230125157569,고메 바삭쫄깃한 사천탕수육 450g,1,8.938007,M,4,2023-01-25,1,0,9878,...,9.302059,12.583337,7.273786,0.787251,0.084592,8.938007,2.519166,302.000092,25,4
4,20230125156655,고메 콘크림수프 180g*4입,1,8.997518,F,3,2023-01-25,0,0,14016,...,9.302059,12.583337,7.273786,0.787251,0.084592,8.997518,2.703802,302.000092,25,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19800,20230102975115,[2023설사원선물신청] 흑삼정 골드클래스 100gx2,1,11.597276,M,5,2023-01-02,0,0,5196,...,9.707501,13.927475,6.803505,1.199277,0.123488,11.597276,4.312596,696.373746,2,1
19801,20230102972321,고메 바삭쫄깃한 탕수육 450g,1,8.702510,M,5,2023-01-02,1,0,5196,...,9.707501,13.927475,6.803505,1.199277,0.123488,8.702510,4.312596,696.373746,2,1
19802,20230101969615,리턴업 전립소 쏘팔메토 골드 1000mgX60캡슐(2개월)X2개,1,11.364344,F,2,2023-01-01,1,0,5661,...,9.350519,14.475993,6.398595,1.059660,0.113304,11.364344,3.517090,506.659755,1,1
19803,20230102972321,고메 거멍 모짜체다핫도그 340g,1,8.648397,M,5,2023-01-02,1,0,5196,...,9.707501,13.927475,6.803505,1.199277,0.123488,8.648397,4.312596,696.373746,2,1


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,order_amt_mean,order_amt_max,order_amt_min,order_amt_std,order_amt_lam,qty_x_amt,age_qty_mean_x_order_qty_mean,order_amt_max_x_order_qty_max,day,day_label
0,20230125158386,비비고 왕교자 1.05kg,1,9.348449,F,4,2023-01-25,0,0,20945,...,8.834666,11.713218,6.388561,0.763573,0.086380,9.348449,2.488897,234.264358,25,4
1,20230126164638,고메 바삭쫄깃한 탕수육 900g,1,9.667259,F,4,2023-01-26,1,0,20945,...,9.114876,12.026773,6.455199,0.609696,0.066884,9.667259,1.967695,276.615768,26,4
2,20230125159705,햇반 매일잡곡밥210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,8.834666,11.713218,6.388561,0.763573,0.086380,199.893056,2.488897,234.264358,25,4
3,20230126167696,[UPCYCLE]햇반 흑미밥 210gx24개(1box),1,10.125110,F,3,2023-01-26,0,1,10109,...,9.114876,12.026773,6.455199,0.609696,0.066884,10.125110,2.034536,276.615768,26,4
4,20230125159705,햇반 매일찰잡곡밥 210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,8.834666,11.713218,6.388561,0.763573,0.086380,199.893056,2.488897,234.264358,25,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26065,20230101966635,고메 순살크리스피 치킨 고추크런치 450g,1,8.659560,M,3,2023-01-01,0,0,10109,...,8.837861,13.908243,6.368187,0.798415,0.090328,8.659560,2.990782,695.412144,1,1
26066,20230102974273,비비고 사골순댓국 460g,10,10.518646,M,4,2023-01-02,1,0,20945,...,8.890128,14.674198,6.705639,0.926187,0.104145,105.186462,3.096541,733.709901,2,1
26067,20230102972720,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,3,2023-01-02,0,0,10109,...,8.890128,14.674198,6.705639,0.926187,0.104145,9.639327,3.201727,733.709901,2,1
26068,20230101964953,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,4,2023-01-01,0,0,20945,...,8.837861,13.908243,6.368187,0.798415,0.090328,9.639327,2.892526,695.412144,1,1


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,order_amt_mean,order_amt_max,order_amt_min,order_amt_std,order_amt_lam,qty_x_amt,age_qty_mean_x_order_qty_mean,order_amt_max_x_order_qty_max,day,day_label
0,20230101970142,삼호 생선살어묵 야채 200g,1,8.098947,M,4,2023-01-01,NaN,0,4344,...,9.287316,13.756107,6.447306,1.014246,0.109176,8.098947,3.544994,687.805328,1,1
1,20230101964512,[앱전용특가] 쁘띠첼밀감90gX6개X3개,1,9.076809,M,4,2023-01-01,NaN,0,4344,...,9.287316,13.756107,6.447306,1.014246,0.109176,9.076809,3.544994,687.805328,1,1
2,20230102975046,빕스 볶음밥 1인분 X 6종_행사,1,9.928522,F,3,2023-01-02,NaN,1,6299,...,9.414437,14.085958,6.937314,1.105816,0.117420,9.928522,4.143983,704.297911,2,1
3,20230102973138,백설 진한참기름 500ml,2,10.060534,M,4,2023-01-02,NaN,0,4344,...,9.414437,14.085958,6.937314,1.105816,0.117420,20.121068,3.942912,704.297911,2,1
4,20230101969192,삼호 가쓰오모둠어묵탕413g,2,9.796181,M,4,2023-01-01,NaN,0,4344,...,9.287316,13.756107,6.447306,1.014246,0.109176,19.592361,3.544994,687.805328,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8041,20230103994630,고메 콰트로포르마지 피자 310g,1,8.736650,M,5,2023-01-03,NaN,0,2051,...,9.669191,14.010500,7.199678,1.208723,0.124901,8.736650,5.032229,700.524994,3,1
8042,20230103994130,햇반솥반 전복내장영양밥 200g,2,9.196748,M,3,2023-01-03,NaN,1,6299,...,9.669191,14.010500,7.199678,1.208723,0.124901,18.393497,4.857391,700.524994,3,1
8043,20230103986821,비비고 찐만두168g,1,7.930566,M,5,2023-01-03,NaN,0,2051,...,9.669191,14.010500,7.199678,1.208723,0.124901,7.930566,5.032229,700.524994,3,1
8044,20230102979023,사골곰탕 한 그릇 100g (1-2인분X5개입),1,8.477412,F,3,2023-01-02,NaN,0,6299,...,9.414437,14.085958,6.937314,1.105816,0.117420,8.477412,4.143983,704.297911,2,1


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,order_amt_mean,order_amt_max,order_amt_min,order_amt_std,order_amt_lam,qty_x_amt,age_qty_mean_x_order_qty_mean,order_amt_max_x_order_qty_max,day,day_label
0,20230101964282,비비고 스팸부대찌개 460g,1,8.161946,M,4,2023-01-01,NaN,0,9748,...,8.748463,13.975836,6.771936,0.710914,0.081249,8.161946,2.854018,698.791807,1,1
1,20230101965237,크레잇 블랙페퍼 폭찹 스테이크,1,8.277412,F,4,2023-01-01,NaN,0,9748,...,8.748463,13.975836,6.771936,0.710914,0.081249,8.277412,2.854018,698.791807,1,1
2,20230102973798,비비고 소고기 미역국 500g,3,9.145375,F,3,2023-01-02,NaN,0,5034,...,8.760965,14.674198,6.552508,0.834438,0.095224,27.436125,3.084216,733.709901,2,1
3,20230101965633,비비고 소고기 미역국 500g,2,8.613230,F,3,2023-01-01,NaN,0,5034,...,8.748463,13.975836,6.771936,0.710914,0.081249,17.226461,2.813325,698.791807,1,1
4,20230101963753,고메 치즈크리스피핫도그 340g,1,8.540714,F,4,2023-01-01,NaN,0,9748,...,8.748463,13.975836,6.771936,0.710914,0.081249,8.540714,2.854018,698.791807,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11609,20230102980326,고메 케이준프라이즈 피자 145g,2,8.671115,F,4,2023-01-02,NaN,0,9748,...,8.760965,14.674198,6.552508,0.834438,0.095224,17.342231,3.128827,733.709901,2,1
11610,20230102983068,고메 빅크리스피핫도그520g,1,8.641179,F,3,2023-01-02,NaN,0,5034,...,8.760965,14.674198,6.552508,0.834438,0.095224,8.641179,3.084216,733.709901,2,1
11611,20230103992835,비비고 메추리알장조림 170g,7,9.467692,F,4,2023-01-03,NaN,0,9748,...,9.005015,14.646290,6.643790,1.159895,0.128704,66.273845,3.877505,717.668190,3,1
11612,20230104999357,비비고 언양식 바싹불고기 460g,1,8.594895,M,4,2023-01-04,NaN,0,9748,...,9.188329,13.094994,7.742402,1.033834,0.111792,8.594895,4.230232,549.989758,4,1


#### 상품별 평균 구매금액

In [18]:
# train
f_tr = emp_tr.groupby('product_name')['net_order_amt'].agg([('product_amt_mean',np.mean)]).reset_index()
nf_tr = nemp_tr.groupby('product_name')['net_order_amt'].agg([('product_amt_mean',np.mean)]).reset_index()

emp_tr = pd.merge(emp_tr, f_tr, on = 'product_name', how = 'left')
nemp_tr = pd.merge(nemp_tr, nf_tr, on = 'product_name', how = 'left')

# test
f_te = emp_te.groupby('product_name')['net_order_amt'].agg([('product_amt_mean',np.mean)]).reset_index()
nf_te = nemp_te.groupby('product_name')['net_order_amt'].agg([('product_amt_mean',np.mean)]).reset_index()

emp_te = pd.merge(emp_te, f_te, on = 'product_name', how = 'left')
nemp_te = pd.merge(nemp_te, nf_te, on = 'product_name', how = 'left')

display(f_tr, nf_tr, f_te, nf_te)

,product_name,product_amt_mean
0,(냉동) 비비고 테이블 특 선물세트 (특양지곰탕700gx2개+특설렁탕700gx1개),10.291666
1,(냉동) 비비고 테이블 특설렁탕 700g,9.686073
2,(냉동) 비비고 테이블 특설렁탕 700gX2개,9.798905
3,(냉동) 비비고 테이블 특설렁탕 700gx2개,9.875140
4,(냉동) 비비고 테이블 특설렁탕 700x2개+(냉동) 비비고 테이블 특양지곰탕 70...,10.545719
...,...,...
2548,행복한콩 폭신폭신 두부볼 750gX2개,9.909420
2549,헬씨누리 침향환 환심 10환,9.538567
2550,헬씨누리 침향환 환심 10환X6입(1BOX)_패밀리데이,11.091499
2551,헬씨누리 침향환 환심 10환X6입(1BOX)_행사,11.043706


,product_name,product_amt_mean
0,(냉동) 비비고 테이블 특 선물세트 (특양지곰탕700gx2개+특설렁탕700gx1개),10.268687
1,(냉동) 비비고 테이블 특설렁탕 700g,9.261695
2,(냉동) 비비고 테이블 특설렁탕 700x2개+(냉동) 비비고 테이블 특양지곰탕 70...,10.568083
3,(냉동) 비비고 테이블 특설렁탕 700x2개+(냉동) 비비고 테이블 특양지곰탕 70...,10.543893
4,(냉동) 비비고 테이블 특설렁탕 700x3개+(냉동) 비비고 테이블 특양지곰탕 70...,11.184435
...,...,...
1873,행복한콩 촌두부 380g,8.299537
1874,행복한콩 콩비지 320g,7.728863
1875,행복한콩 큰두부520g,7.831108
1876,행복한콩 폭신폭신 두부볼 750g,9.277608


,product_name,product_amt_mean
0,(냉동) 비비고 테이블 특 선물세트 (특양지곰탕700gx2개+특설렁탕700gx1개),10.243329
1,(냉동) 비비고 테이블 특설렁탕 700g,9.399631
2,(냉동) 비비고 테이블 특설렁탕 700x3개+(냉동) 비비고 테이블 특양지곰탕 70...,10.836739
3,(냉동) 비비고 테이블 특양지곰탕 700g,9.243566
4,(냉동) 비비고 테이블 특양지곰탕 700gx2개,9.877139
...,...,...
1896,행복한콩 콩비지 320g,8.045832
1897,행복한콩 큰두부520g,8.024232
1898,행복한콩 폭신폭신 두부볼 750g,9.634831
1899,헬씨누리 침향환 환심 10환X6입(1BOX)_행사,11.062823


,product_name,product_amt_mean
0,(냉동) 비비고 테이블 특설렁탕 700g,9.088013
1,(냉동) 비비고 테이블 특설렁탕 700gx2개,9.850772
2,(냉동) 비비고 테이블 특설렁탕 700x2개+(냉동) 비비고 테이블 특양지곰탕 70...,10.560541
3,(냉동) 비비고 테이블 특설렁탕 700x3개+(냉동) 비비고 테이블 특양지곰탕 70...,10.817849
4,(냉동) 비비고 테이블 특양지곰탕 700g,9.209254
...,...,...
1468,행복한콩 촌두부 380g,8.173457
1469,행복한콩 콩비지 320g,7.865188
1470,행복한콩 큰두부520g,7.649563
1471,행복한콩 폭신폭신 두부볼 750g,9.058824


#### 상품별 최대 구매수량

In [19]:
# train
f_tr = emp_tr.groupby('product_name')['net_order_qty'].agg([('product_qty_max',np.max)]).reset_index()
nf_tr = nemp_tr.groupby('product_name')['net_order_qty'].agg([('product_qty_max',np.max)]).reset_index()

emp_tr = pd.merge(emp_tr, f_tr, on = 'product_name', how = 'left')
nemp_tr = pd.merge(nemp_tr, nf_tr, on = 'product_name', how = 'left')

# test
f_te = emp_te.groupby('product_name')['net_order_qty'].agg([('product_qty_max',np.max)]).reset_index()
nf_te = nemp_te.groupby('product_name')['net_order_qty'].agg([('product_qty_max',np.max)]).reset_index()

emp_te = pd.merge(emp_te, f_te, on = 'product_name', how = 'left')
nemp_te = pd.merge(nemp_te, nf_te, on = 'product_name', how = 'left')

display(f_tr, nf_tr, f_te, nf_te)

,product_name,product_qty_max
0,(냉동) 비비고 테이블 특 선물세트 (특양지곰탕700gx2개+특설렁탕700gx1개),2
1,(냉동) 비비고 테이블 특설렁탕 700g,12
2,(냉동) 비비고 테이블 특설렁탕 700gX2개,1
3,(냉동) 비비고 테이블 특설렁탕 700gx2개,1
4,(냉동) 비비고 테이블 특설렁탕 700x2개+(냉동) 비비고 테이블 특양지곰탕 70...,1
...,...,...
2548,행복한콩 폭신폭신 두부볼 750gX2개,1
2549,헬씨누리 침향환 환심 10환,1
2550,헬씨누리 침향환 환심 10환X6입(1BOX)_패밀리데이,1
2551,헬씨누리 침향환 환심 10환X6입(1BOX)_행사,1


,product_name,product_qty_max
0,(냉동) 비비고 테이블 특 선물세트 (특양지곰탕700gx2개+특설렁탕700gx1개),1
1,(냉동) 비비고 테이블 특설렁탕 700g,5
2,(냉동) 비비고 테이블 특설렁탕 700x2개+(냉동) 비비고 테이블 특양지곰탕 70...,1
3,(냉동) 비비고 테이블 특설렁탕 700x2개+(냉동) 비비고 테이블 특양지곰탕 70...,1
4,(냉동) 비비고 테이블 특설렁탕 700x3개+(냉동) 비비고 테이블 특양지곰탕 70...,1
...,...,...
1873,행복한콩 촌두부 380g,3
1874,행복한콩 콩비지 320g,5
1875,행복한콩 큰두부520g,20
1876,행복한콩 폭신폭신 두부볼 750g,3


,product_name,product_qty_max
0,(냉동) 비비고 테이블 특 선물세트 (특양지곰탕700gx2개+특설렁탕700gx1개),1
1,(냉동) 비비고 테이블 특설렁탕 700g,2
2,(냉동) 비비고 테이블 특설렁탕 700x3개+(냉동) 비비고 테이블 특양지곰탕 70...,1
3,(냉동) 비비고 테이블 특양지곰탕 700g,4
4,(냉동) 비비고 테이블 특양지곰탕 700gx2개,1
...,...,...
1896,행복한콩 콩비지 320g,3
1897,행복한콩 큰두부520g,2
1898,행복한콩 폭신폭신 두부볼 750g,6
1899,헬씨누리 침향환 환심 10환X6입(1BOX)_행사,1


,product_name,product_qty_max
0,(냉동) 비비고 테이블 특설렁탕 700g,6
1,(냉동) 비비고 테이블 특설렁탕 700gx2개,1
2,(냉동) 비비고 테이블 특설렁탕 700x2개+(냉동) 비비고 테이블 특양지곰탕 70...,1
3,(냉동) 비비고 테이블 특설렁탕 700x3개+(냉동) 비비고 테이블 특양지곰탕 70...,1
4,(냉동) 비비고 테이블 특양지곰탕 700g,6
...,...,...
1468,행복한콩 촌두부 380g,3
1469,행복한콩 콩비지 320g,1
1470,행복한콩 큰두부520g,2
1471,행복한콩 폭신폭신 두부볼 750g,2


In [20]:
# train
f_tr = emp_tr.groupby('product_name')['net_order_amt'].agg([('net_order_amt_max-min',
                                                                       lambda x : np.max(x) - np.min(x))]).reset_index()
nf_tr = nemp_tr.groupby('product_name')['net_order_amt'].agg([('net_order_amt_max-min',
                                                                         lambda x : np.max(x) - np.min(x))]).reset_index()

emp_tr = pd.merge(emp_tr, f_tr, on= 'product_name', how= 'left')
nemp_tr = pd.merge(nemp_tr, nf_tr, on= 'product_name', how= 'left')

#test
f_te = emp_te.groupby('product_name')['net_order_amt'].agg([('net_order_amt_max-min',
                                                                       lambda x : np.max(x) - np.min(x))]).reset_index()
nf_te = nemp_te.groupby('product_name')['net_order_amt'].agg([('net_order_amt_max-min',
                                                                         lambda x : np.max(x) - np.min(x))]).reset_index()

emp_te = pd.merge(emp_te, f_te, on= 'product_name', how= 'left')
nemp_te = pd.merge(nemp_te, nf_te, on= 'product_name', how= 'left')

display(f_tr, nf_tr, f_te, nf_te)

,product_name,net_order_amt_max-min
0,(냉동) 비비고 테이블 특 선물세트 (특양지곰탕700gx2개+특설렁탕700gx1개),0.709533
1,(냉동) 비비고 테이블 특설렁탕 700g,2.484884
2,(냉동) 비비고 테이블 특설렁탕 700gX2개,0.000000
3,(냉동) 비비고 테이블 특설렁탕 700gx2개,0.000000
4,(냉동) 비비고 테이블 특설렁탕 700x2개+(냉동) 비비고 테이블 특양지곰탕 70...,0.045084
...,...,...
2548,행복한콩 폭신폭신 두부볼 750gX2개,0.000000
2549,헬씨누리 침향환 환심 10환,0.047817
2550,헬씨누리 침향환 환심 10환X6입(1BOX)_패밀리데이,0.000000
2551,헬씨누리 침향환 환심 10환X6입(1BOX)_행사,0.000000


,product_name,net_order_amt_max-min
0,(냉동) 비비고 테이블 특 선물세트 (특양지곰탕700gx2개+특설렁탕700gx1개),0.072583
1,(냉동) 비비고 테이블 특설렁탕 700g,1.696338
2,(냉동) 비비고 테이블 특설렁탕 700x2개+(냉동) 비비고 테이블 특양지곰탕 70...,0.072569
3,(냉동) 비비고 테이블 특설렁탕 700x2개+(냉동) 비비고 테이블 특양지곰탕 70...,0.000000
4,(냉동) 비비고 테이블 특설렁탕 700x3개+(냉동) 비비고 테이블 특양지곰탕 70...,0.000000
...,...,...
1873,행복한콩 촌두부 380g,1.381768
1874,행복한콩 콩비지 320g,1.696462
1875,행복한콩 큰두부520g,3.082675
1876,행복한콩 폭신폭신 두부볼 750g,1.405034


,product_name,net_order_amt_max-min
0,(냉동) 비비고 테이블 특 선물세트 (특양지곰탕700gx2개+특설렁탕700gx1개),0.044667
1,(냉동) 비비고 테이블 특설렁탕 700g,1.073695
2,(냉동) 비비고 테이블 특설렁탕 700x3개+(냉동) 비비고 테이블 특양지곰탕 70...,0.027933
3,(냉동) 비비고 테이블 특양지곰탕 700g,1.386294
4,(냉동) 비비고 테이블 특양지곰탕 700gx2개,0.150439
...,...,...
1896,행복한콩 콩비지 320g,0.930115
1897,행복한콩 큰두부520g,0.740694
1898,행복한콩 폭신폭신 두부볼 750g,1.583477
1899,헬씨누리 침향환 환심 10환X6입(1BOX)_행사,0.047793


,product_name,net_order_amt_max-min
0,(냉동) 비비고 테이블 특설렁탕 700g,2.140034
1,(냉동) 비비고 테이블 특설렁탕 700gx2개,0.000000
2,(냉동) 비비고 테이블 특설렁탕 700x2개+(냉동) 비비고 테이블 특양지곰탕 70...,0.000000
3,(냉동) 비비고 테이블 특설렁탕 700x3개+(냉동) 비비고 테이블 특양지곰탕 70...,0.123880
4,(냉동) 비비고 테이블 특양지곰탕 700g,2.005288
...,...,...
1468,행복한콩 촌두부 380g,1.203704
1469,행복한콩 콩비지 320g,0.000000
1470,행복한콩 큰두부520g,0.671527
1471,행복한콩 폭신폭신 두부볼 750g,0.765756


#### 매출 하위 5가지 제품 구매여부

In [21]:
emp_tr_bottom5 = list(emp_tr.product_name.value_counts().tail().reset_index()['index'])
nemp_tr_bottom5 = list(nemp_tr.product_name.value_counts().tail().reset_index()['index'])

In [22]:
# train
emp_tr['bottom5'] = emp_tr['product_name'].apply(lambda x: 1 if x in emp_tr_bottom5 else 0)
nemp_tr['bottom5'] = nemp_tr['product_name'].apply(lambda x: 1 if x in nemp_tr_bottom5 else 0)

#test
emp_te['bottom5'] = emp_te['product_name'].apply(lambda x: 1 if x in emp_tr_bottom5 else 0)
nemp_te['bottom5'] = nemp_te['product_name'].apply(lambda x: 1 if x in nemp_tr_bottom5 else 0)

display(emp_tr, nemp_tr, emp_te, nemp_te)

,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,order_amt_lam,qty_x_amt,age_qty_mean_x_order_qty_mean,order_amt_max_x_order_qty_max,day,day_label,product_amt_mean,product_qty_max,net_order_amt_max-min,bottom5
0,20230124153976,잔칫집 식혜 240ml 30입,1,9.803170,F,2,2023-01-24,0,0,5661,...,0.090297,9.803170,2.650429,126.412939,24,4,9.868019,2,0.693118,0
1,20230124155563,백설 한입쏙 비엔나 120g*2,1,8.256607,M,3,2023-01-24,0,0,14016,...,0.090297,8.256607,2.645762,126.412939,24,4,8.451926,10,2.592984,0
2,20230125160921,맥스봉 체다치즈어랏400g (20g*20EA),1,8.974998,F,2,2023-01-25,0,0,5661,...,0.084592,8.974998,2.708572,302.000092,25,4,9.045646,2,0.705940,0
3,20230125157569,고메 바삭쫄깃한 사천탕수육 450g,1,8.938007,M,4,2023-01-25,1,0,9878,...,0.084592,8.938007,2.519166,302.000092,25,4,9.163596,3,1.501889,0
4,20230125156655,고메 콘크림수프 180g*4입,1,8.997518,F,3,2023-01-25,0,0,14016,...,0.084592,8.997518,2.703802,302.000092,25,4,8.996939,4,1.423942,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19800,20230102975115,[2023설사원선물신청] 흑삼정 골드클래스 100gx2,1,11.597276,M,5,2023-01-02,0,0,5196,...,0.123488,11.597276,4.312596,696.373746,2,1,11.853054,9,2.215853,0
19801,20230102972321,고메 바삭쫄깃한 탕수육 450g,1,8.702510,M,5,2023-01-02,1,0,5196,...,0.123488,8.702510,4.312596,696.373746,2,1,9.120909,5,1.674637,0
19802,20230101969615,리턴업 전립소 쏘팔메토 골드 1000mgX60캡슐(2개월)X2개,1,11.364344,F,2,2023-01-01,1,0,5661,...,0.113304,11.364344,3.517090,506.659755,1,1,11.479985,3,1.016713,0
19803,20230102972321,고메 거멍 모짜체다핫도그 340g,1,8.648397,M,5,2023-01-02,1,0,5196,...,0.123488,8.648397,4.312596,696.373746,2,1,8.977414,2,1.042207,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,order_amt_lam,qty_x_amt,age_qty_mean_x_order_qty_mean,order_amt_max_x_order_qty_max,day,day_label,product_amt_mean,product_qty_max,net_order_amt_max-min,bottom5
0,20230125158386,비비고 왕교자 1.05kg,1,9.348449,F,4,2023-01-25,0,0,20945,...,0.086380,9.348449,2.488897,234.264358,25,4,9.273739,12,2.914180,0
1,20230126164638,고메 바삭쫄깃한 탕수육 900g,1,9.667259,F,4,2023-01-26,1,0,20945,...,0.066884,9.667259,1.967695,276.615768,26,4,9.410137,4,1.511256,0
2,20230125159705,햇반 매일잡곡밥210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,0.086380,199.893056,2.488897,234.264358,25,4,10.036068,20,0.082830,0
3,20230126167696,[UPCYCLE]햇반 흑미밥 210gx24개(1box),1,10.125110,F,3,2023-01-26,0,1,10109,...,0.066884,10.125110,2.034536,276.615768,26,4,10.177054,5,1.747883,0
4,20230125159705,햇반 매일찰잡곡밥 210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,0.086380,199.893056,2.488897,234.264358,25,4,8.976090,20,2.037125,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26065,20230101966635,고메 순살크리스피 치킨 고추크런치 450g,1,8.659560,M,3,2023-01-01,0,0,10109,...,0.090328,8.659560,2.990782,695.412144,1,1,8.953284,7,2.108972,0
26066,20230102974273,비비고 사골순댓국 460g,10,10.518646,M,4,2023-01-02,1,0,20945,...,0.104145,105.186462,3.096541,733.709901,2,1,8.759052,20,3.074324,0
26067,20230102972720,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,3,2023-01-02,0,0,10109,...,0.104145,9.639327,3.201727,733.709901,2,1,9.713009,3,1.098566,0
26068,20230101964953,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,4,2023-01-01,0,0,20945,...,0.090328,9.639327,2.892526,695.412144,1,1,9.713009,3,1.098566,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,order_amt_lam,qty_x_amt,age_qty_mean_x_order_qty_mean,order_amt_max_x_order_qty_max,day,day_label,product_amt_mean,product_qty_max,net_order_amt_max-min,bottom5
0,20230101970142,삼호 생선살어묵 야채 200g,1,8.098947,M,4,2023-01-01,NaN,0,4344,...,0.109176,8.098947,3.544994,687.805328,1,1,8.143229,1,0.109817,0
1,20230101964512,[앱전용특가] 쁘띠첼밀감90gX6개X3개,1,9.076809,M,4,2023-01-01,NaN,0,4344,...,0.109176,9.076809,3.544994,687.805328,1,1,9.105797,1,0.086964,0
2,20230102975046,빕스 볶음밥 1인분 X 6종_행사,1,9.928522,F,3,2023-01-02,NaN,1,6299,...,0.117420,9.928522,4.143983,704.297911,2,1,10.125702,2,0.740922,0
3,20230102973138,백설 진한참기름 500ml,2,10.060534,M,4,2023-01-02,NaN,0,4344,...,0.117420,20.121068,3.942912,704.297911,2,1,9.552229,2,1.068972,0
4,20230101969192,삼호 가쓰오모둠어묵탕413g,2,9.796181,M,4,2023-01-01,NaN,0,4344,...,0.109176,19.592361,3.544994,687.805328,1,1,9.169970,3,1.291451,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8041,20230103994630,고메 콰트로포르마지 피자 310g,1,8.736650,M,5,2023-01-03,NaN,0,2051,...,0.124901,8.736650,5.032229,700.524994,3,1,9.050980,2,1.096499,0
8042,20230103994130,햇반솥반 전복내장영양밥 200g,2,9.196748,M,3,2023-01-03,NaN,1,6299,...,0.124901,18.393497,4.857391,700.524994,3,1,8.900761,10,2.493781,0
8043,20230103986821,비비고 찐만두168g,1,7.930566,M,5,2023-01-03,NaN,0,2051,...,0.124901,7.930566,5.032229,700.524994,3,1,8.521369,4,1.357569,0
8044,20230102979023,사골곰탕 한 그릇 100g (1-2인분X5개입),1,8.477412,F,3,2023-01-02,NaN,0,6299,...,0.117420,8.477412,4.143983,704.297911,2,1,8.674647,2,0.693043,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,order_amt_lam,qty_x_amt,age_qty_mean_x_order_qty_mean,order_amt_max_x_order_qty_max,day,day_label,product_amt_mean,product_qty_max,net_order_amt_max-min,bottom5
0,20230101964282,비비고 스팸부대찌개 460g,1,8.161946,M,4,2023-01-01,NaN,0,9748,...,0.081249,8.161946,2.854018,698.791807,1,1,8.625450,10,2.316721,0
1,20230101965237,크레잇 블랙페퍼 폭찹 스테이크,1,8.277412,F,4,2023-01-01,NaN,0,9748,...,0.081249,8.277412,2.854018,698.791807,1,1,8.724876,6,1.845584,0
2,20230102973798,비비고 소고기 미역국 500g,3,9.145375,F,3,2023-01-02,NaN,0,5034,...,0.095224,27.436125,3.084216,733.709901,2,1,8.886063,10,2.422193,0
3,20230101965633,비비고 소고기 미역국 500g,2,8.613230,F,3,2023-01-01,NaN,0,5034,...,0.081249,17.226461,2.813325,698.791807,1,1,8.886063,10,2.422193,0
4,20230101963753,고메 치즈크리스피핫도그 340g,1,8.540714,F,4,2023-01-01,NaN,0,9748,...,0.081249,8.540714,2.854018,698.791807,1,1,8.766554,6,1.969910,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11609,20230102980326,고메 케이준프라이즈 피자 145g,2,8.671115,F,4,2023-01-02,NaN,0,9748,...,0.095224,17.342231,3.128827,733.709901,2,1,8.505393,6,1.878765,0
11610,20230102983068,고메 빅크리스피핫도그520g,1,8.641179,F,3,2023-01-02,NaN,0,5034,...,0.095224,8.641179,3.084216,733.709901,2,1,8.920214,4,1.599858,0
11611,20230103992835,비비고 메추리알장조림 170g,7,9.467692,F,4,2023-01-03,NaN,0,9748,...,0.128704,66.273845,3.877505,717.668190,3,1,8.157156,9,2.197164,0
11612,20230104999357,비비고 언양식 바싹불고기 460g,1,8.594895,M,4,2023-01-04,NaN,0,9748,...,0.111792,8.594895,4.230232,549.989758,4,1,8.762952,4,1.473311,0


#### 매출 상위 5가지 제품 구매여부

In [23]:
emp_tr_top5 = list(emp_tr.product_name.value_counts().head().reset_index()['index'])
nemp_tr_top5 = list(nemp_tr.product_name.value_counts().head().reset_index()['index'])

In [24]:
# train
emp_tr['top5'] = emp_tr['product_name'].apply(lambda x: 1 if x in emp_tr_top5 else 0)
nemp_tr['top5'] = nemp_tr['product_name'].apply(lambda x: 1 if x in nemp_tr_top5 else 0)

#test
emp_te['top5'] = emp_te['product_name'].apply(lambda x: 1 if x in emp_tr_top5 else 0)
nemp_te['top5'] = nemp_te['product_name'].apply(lambda x: 1 if x in nemp_tr_top5 else 0)

display(emp_tr, nemp_tr, emp_te, nemp_te)

,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,qty_x_amt,age_qty_mean_x_order_qty_mean,order_amt_max_x_order_qty_max,day,day_label,product_amt_mean,product_qty_max,net_order_amt_max-min,bottom5,top5
0,20230124153976,잔칫집 식혜 240ml 30입,1,9.803170,F,2,2023-01-24,0,0,5661,...,9.803170,2.650429,126.412939,24,4,9.868019,2,0.693118,0,0
1,20230124155563,백설 한입쏙 비엔나 120g*2,1,8.256607,M,3,2023-01-24,0,0,14016,...,8.256607,2.645762,126.412939,24,4,8.451926,10,2.592984,0,0
2,20230125160921,맥스봉 체다치즈어랏400g (20g*20EA),1,8.974998,F,2,2023-01-25,0,0,5661,...,8.974998,2.708572,302.000092,25,4,9.045646,2,0.705940,0,0
3,20230125157569,고메 바삭쫄깃한 사천탕수육 450g,1,8.938007,M,4,2023-01-25,1,0,9878,...,8.938007,2.519166,302.000092,25,4,9.163596,3,1.501889,0,0
4,20230125156655,고메 콘크림수프 180g*4입,1,8.997518,F,3,2023-01-25,0,0,14016,...,8.997518,2.703802,302.000092,25,4,8.996939,4,1.423942,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19800,20230102975115,[2023설사원선물신청] 흑삼정 골드클래스 100gx2,1,11.597276,M,5,2023-01-02,0,0,5196,...,11.597276,4.312596,696.373746,2,1,11.853054,9,2.215853,0,0
19801,20230102972321,고메 바삭쫄깃한 탕수육 450g,1,8.702510,M,5,2023-01-02,1,0,5196,...,8.702510,4.312596,696.373746,2,1,9.120909,5,1.674637,0,0
19802,20230101969615,리턴업 전립소 쏘팔메토 골드 1000mgX60캡슐(2개월)X2개,1,11.364344,F,2,2023-01-01,1,0,5661,...,11.364344,3.517090,506.659755,1,1,11.479985,3,1.016713,0,0
19803,20230102972321,고메 거멍 모짜체다핫도그 340g,1,8.648397,M,5,2023-01-02,1,0,5196,...,8.648397,4.312596,696.373746,2,1,8.977414,2,1.042207,0,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,qty_x_amt,age_qty_mean_x_order_qty_mean,order_amt_max_x_order_qty_max,day,day_label,product_amt_mean,product_qty_max,net_order_amt_max-min,bottom5,top5
0,20230125158386,비비고 왕교자 1.05kg,1,9.348449,F,4,2023-01-25,0,0,20945,...,9.348449,2.488897,234.264358,25,4,9.273739,12,2.914180,0,0
1,20230126164638,고메 바삭쫄깃한 탕수육 900g,1,9.667259,F,4,2023-01-26,1,0,20945,...,9.667259,1.967695,276.615768,26,4,9.410137,4,1.511256,0,0
2,20230125159705,햇반 매일잡곡밥210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,199.893056,2.488897,234.264358,25,4,10.036068,20,0.082830,0,0
3,20230126167696,[UPCYCLE]햇반 흑미밥 210gx24개(1box),1,10.125110,F,3,2023-01-26,0,1,10109,...,10.125110,2.034536,276.615768,26,4,10.177054,5,1.747883,0,0
4,20230125159705,햇반 매일찰잡곡밥 210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,199.893056,2.488897,234.264358,25,4,8.976090,20,2.037125,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26065,20230101966635,고메 순살크리스피 치킨 고추크런치 450g,1,8.659560,M,3,2023-01-01,0,0,10109,...,8.659560,2.990782,695.412144,1,1,8.953284,7,2.108972,0,0
26066,20230102974273,비비고 사골순댓국 460g,10,10.518646,M,4,2023-01-02,1,0,20945,...,105.186462,3.096541,733.709901,2,1,8.759052,20,3.074324,0,0
26067,20230102972720,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,3,2023-01-02,0,0,10109,...,9.639327,3.201727,733.709901,2,1,9.713009,3,1.098566,0,0
26068,20230101964953,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,4,2023-01-01,0,0,20945,...,9.639327,2.892526,695.412144,1,1,9.713009,3,1.098566,0,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,qty_x_amt,age_qty_mean_x_order_qty_mean,order_amt_max_x_order_qty_max,day,day_label,product_amt_mean,product_qty_max,net_order_amt_max-min,bottom5,top5
0,20230101970142,삼호 생선살어묵 야채 200g,1,8.098947,M,4,2023-01-01,NaN,0,4344,...,8.098947,3.544994,687.805328,1,1,8.143229,1,0.109817,0,0
1,20230101964512,[앱전용특가] 쁘띠첼밀감90gX6개X3개,1,9.076809,M,4,2023-01-01,NaN,0,4344,...,9.076809,3.544994,687.805328,1,1,9.105797,1,0.086964,0,0
2,20230102975046,빕스 볶음밥 1인분 X 6종_행사,1,9.928522,F,3,2023-01-02,NaN,1,6299,...,9.928522,4.143983,704.297911,2,1,10.125702,2,0.740922,0,0
3,20230102973138,백설 진한참기름 500ml,2,10.060534,M,4,2023-01-02,NaN,0,4344,...,20.121068,3.942912,704.297911,2,1,9.552229,2,1.068972,0,0
4,20230101969192,삼호 가쓰오모둠어묵탕413g,2,9.796181,M,4,2023-01-01,NaN,0,4344,...,19.592361,3.544994,687.805328,1,1,9.169970,3,1.291451,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8041,20230103994630,고메 콰트로포르마지 피자 310g,1,8.736650,M,5,2023-01-03,NaN,0,2051,...,8.736650,5.032229,700.524994,3,1,9.050980,2,1.096499,0,0
8042,20230103994130,햇반솥반 전복내장영양밥 200g,2,9.196748,M,3,2023-01-03,NaN,1,6299,...,18.393497,4.857391,700.524994,3,1,8.900761,10,2.493781,0,0
8043,20230103986821,비비고 찐만두168g,1,7.930566,M,5,2023-01-03,NaN,0,2051,...,7.930566,5.032229,700.524994,3,1,8.521369,4,1.357569,0,0
8044,20230102979023,사골곰탕 한 그릇 100g (1-2인분X5개입),1,8.477412,F,3,2023-01-02,NaN,0,6299,...,8.477412,4.143983,704.297911,2,1,8.674647,2,0.693043,0,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,qty_x_amt,age_qty_mean_x_order_qty_mean,order_amt_max_x_order_qty_max,day,day_label,product_amt_mean,product_qty_max,net_order_amt_max-min,bottom5,top5
0,20230101964282,비비고 스팸부대찌개 460g,1,8.161946,M,4,2023-01-01,NaN,0,9748,...,8.161946,2.854018,698.791807,1,1,8.625450,10,2.316721,0,0
1,20230101965237,크레잇 블랙페퍼 폭찹 스테이크,1,8.277412,F,4,2023-01-01,NaN,0,9748,...,8.277412,2.854018,698.791807,1,1,8.724876,6,1.845584,0,0
2,20230102973798,비비고 소고기 미역국 500g,3,9.145375,F,3,2023-01-02,NaN,0,5034,...,27.436125,3.084216,733.709901,2,1,8.886063,10,2.422193,0,0
3,20230101965633,비비고 소고기 미역국 500g,2,8.613230,F,3,2023-01-01,NaN,0,5034,...,17.226461,2.813325,698.791807,1,1,8.886063,10,2.422193,0,0
4,20230101963753,고메 치즈크리스피핫도그 340g,1,8.540714,F,4,2023-01-01,NaN,0,9748,...,8.540714,2.854018,698.791807,1,1,8.766554,6,1.969910,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11609,20230102980326,고메 케이준프라이즈 피자 145g,2,8.671115,F,4,2023-01-02,NaN,0,9748,...,17.342231,3.128827,733.709901,2,1,8.505393,6,1.878765,0,0
11610,20230102983068,고메 빅크리스피핫도그520g,1,8.641179,F,3,2023-01-02,NaN,0,5034,...,8.641179,3.084216,733.709901,2,1,8.920214,4,1.599858,0,0
11611,20230103992835,비비고 메추리알장조림 170g,7,9.467692,F,4,2023-01-03,NaN,0,9748,...,66.273845,3.877505,717.668190,3,1,8.157156,9,2.197164,0,0
11612,20230104999357,비비고 언양식 바싹불고기 460g,1,8.594895,M,4,2023-01-04,NaN,0,9748,...,8.594895,4.230232,549.989758,4,1,8.762952,4,1.473311,0,0


#### gender_grp와 age 병합

In [25]:
# train
emp_tr['gender+age'] = emp_tr['gender'] + emp_tr['age_grp'].apply(lambda x : str(x))
nemp_tr['gender+age'] = nemp_tr['gender'] + nemp_tr['age_grp'].apply(lambda x : str(x))

# test
emp_te['gender+age'] = emp_te['gender'] + emp_te['age_grp'].apply(lambda x : str(x))
nemp_te['gender+age'] = nemp_te['gender'] + nemp_te['age_grp'].apply(lambda x : str(x))

display(emp_tr, nemp_tr, emp_te, nemp_te)

,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,age_qty_mean_x_order_qty_mean,order_amt_max_x_order_qty_max,day,day_label,product_amt_mean,product_qty_max,net_order_amt_max-min,bottom5,top5,gender+age
0,20230124153976,잔칫집 식혜 240ml 30입,1,9.803170,F,2,2023-01-24,0,0,5661,...,2.650429,126.412939,24,4,9.868019,2,0.693118,0,0,F2
1,20230124155563,백설 한입쏙 비엔나 120g*2,1,8.256607,M,3,2023-01-24,0,0,14016,...,2.645762,126.412939,24,4,8.451926,10,2.592984,0,0,M3
2,20230125160921,맥스봉 체다치즈어랏400g (20g*20EA),1,8.974998,F,2,2023-01-25,0,0,5661,...,2.708572,302.000092,25,4,9.045646,2,0.705940,0,0,F2
3,20230125157569,고메 바삭쫄깃한 사천탕수육 450g,1,8.938007,M,4,2023-01-25,1,0,9878,...,2.519166,302.000092,25,4,9.163596,3,1.501889,0,0,M4
4,20230125156655,고메 콘크림수프 180g*4입,1,8.997518,F,3,2023-01-25,0,0,14016,...,2.703802,302.000092,25,4,8.996939,4,1.423942,0,0,F3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19800,20230102975115,[2023설사원선물신청] 흑삼정 골드클래스 100gx2,1,11.597276,M,5,2023-01-02,0,0,5196,...,4.312596,696.373746,2,1,11.853054,9,2.215853,0,0,M5
19801,20230102972321,고메 바삭쫄깃한 탕수육 450g,1,8.702510,M,5,2023-01-02,1,0,5196,...,4.312596,696.373746,2,1,9.120909,5,1.674637,0,0,M5
19802,20230101969615,리턴업 전립소 쏘팔메토 골드 1000mgX60캡슐(2개월)X2개,1,11.364344,F,2,2023-01-01,1,0,5661,...,3.517090,506.659755,1,1,11.479985,3,1.016713,0,0,F2
19803,20230102972321,고메 거멍 모짜체다핫도그 340g,1,8.648397,M,5,2023-01-02,1,0,5196,...,4.312596,696.373746,2,1,8.977414,2,1.042207,0,0,M5


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,age_qty_mean_x_order_qty_mean,order_amt_max_x_order_qty_max,day,day_label,product_amt_mean,product_qty_max,net_order_amt_max-min,bottom5,top5,gender+age
0,20230125158386,비비고 왕교자 1.05kg,1,9.348449,F,4,2023-01-25,0,0,20945,...,2.488897,234.264358,25,4,9.273739,12,2.914180,0,0,F4
1,20230126164638,고메 바삭쫄깃한 탕수육 900g,1,9.667259,F,4,2023-01-26,1,0,20945,...,1.967695,276.615768,26,4,9.410137,4,1.511256,0,0,F4
2,20230125159705,햇반 매일잡곡밥210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,2.488897,234.264358,25,4,10.036068,20,0.082830,0,0,M4
3,20230126167696,[UPCYCLE]햇반 흑미밥 210gx24개(1box),1,10.125110,F,3,2023-01-26,0,1,10109,...,2.034536,276.615768,26,4,10.177054,5,1.747883,0,0,F3
4,20230125159705,햇반 매일찰잡곡밥 210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,2.488897,234.264358,25,4,8.976090,20,2.037125,0,0,M4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26065,20230101966635,고메 순살크리스피 치킨 고추크런치 450g,1,8.659560,M,3,2023-01-01,0,0,10109,...,2.990782,695.412144,1,1,8.953284,7,2.108972,0,0,M3
26066,20230102974273,비비고 사골순댓국 460g,10,10.518646,M,4,2023-01-02,1,0,20945,...,3.096541,733.709901,2,1,8.759052,20,3.074324,0,0,M4
26067,20230102972720,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,3,2023-01-02,0,0,10109,...,3.201727,733.709901,2,1,9.713009,3,1.098566,0,0,F3
26068,20230101964953,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,4,2023-01-01,0,0,20945,...,2.892526,695.412144,1,1,9.713009,3,1.098566,0,0,F4


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,age_qty_mean_x_order_qty_mean,order_amt_max_x_order_qty_max,day,day_label,product_amt_mean,product_qty_max,net_order_amt_max-min,bottom5,top5,gender+age
0,20230101970142,삼호 생선살어묵 야채 200g,1,8.098947,M,4,2023-01-01,NaN,0,4344,...,3.544994,687.805328,1,1,8.143229,1,0.109817,0,0,M4
1,20230101964512,[앱전용특가] 쁘띠첼밀감90gX6개X3개,1,9.076809,M,4,2023-01-01,NaN,0,4344,...,3.544994,687.805328,1,1,9.105797,1,0.086964,0,0,M4
2,20230102975046,빕스 볶음밥 1인분 X 6종_행사,1,9.928522,F,3,2023-01-02,NaN,1,6299,...,4.143983,704.297911,2,1,10.125702,2,0.740922,0,0,F3
3,20230102973138,백설 진한참기름 500ml,2,10.060534,M,4,2023-01-02,NaN,0,4344,...,3.942912,704.297911,2,1,9.552229,2,1.068972,0,0,M4
4,20230101969192,삼호 가쓰오모둠어묵탕413g,2,9.796181,M,4,2023-01-01,NaN,0,4344,...,3.544994,687.805328,1,1,9.169970,3,1.291451,0,0,M4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8041,20230103994630,고메 콰트로포르마지 피자 310g,1,8.736650,M,5,2023-01-03,NaN,0,2051,...,5.032229,700.524994,3,1,9.050980,2,1.096499,0,0,M5
8042,20230103994130,햇반솥반 전복내장영양밥 200g,2,9.196748,M,3,2023-01-03,NaN,1,6299,...,4.857391,700.524994,3,1,8.900761,10,2.493781,0,0,M3
8043,20230103986821,비비고 찐만두168g,1,7.930566,M,5,2023-01-03,NaN,0,2051,...,5.032229,700.524994,3,1,8.521369,4,1.357569,0,0,M5
8044,20230102979023,사골곰탕 한 그릇 100g (1-2인분X5개입),1,8.477412,F,3,2023-01-02,NaN,0,6299,...,4.143983,704.297911,2,1,8.674647,2,0.693043,0,0,F3


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,age_qty_mean_x_order_qty_mean,order_amt_max_x_order_qty_max,day,day_label,product_amt_mean,product_qty_max,net_order_amt_max-min,bottom5,top5,gender+age
0,20230101964282,비비고 스팸부대찌개 460g,1,8.161946,M,4,2023-01-01,NaN,0,9748,...,2.854018,698.791807,1,1,8.625450,10,2.316721,0,0,M4
1,20230101965237,크레잇 블랙페퍼 폭찹 스테이크,1,8.277412,F,4,2023-01-01,NaN,0,9748,...,2.854018,698.791807,1,1,8.724876,6,1.845584,0,0,F4
2,20230102973798,비비고 소고기 미역국 500g,3,9.145375,F,3,2023-01-02,NaN,0,5034,...,3.084216,733.709901,2,1,8.886063,10,2.422193,0,0,F3
3,20230101965633,비비고 소고기 미역국 500g,2,8.613230,F,3,2023-01-01,NaN,0,5034,...,2.813325,698.791807,1,1,8.886063,10,2.422193,0,0,F3
4,20230101963753,고메 치즈크리스피핫도그 340g,1,8.540714,F,4,2023-01-01,NaN,0,9748,...,2.854018,698.791807,1,1,8.766554,6,1.969910,0,0,F4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11609,20230102980326,고메 케이준프라이즈 피자 145g,2,8.671115,F,4,2023-01-02,NaN,0,9748,...,3.128827,733.709901,2,1,8.505393,6,1.878765,0,0,F4
11610,20230102983068,고메 빅크리스피핫도그520g,1,8.641179,F,3,2023-01-02,NaN,0,5034,...,3.084216,733.709901,2,1,8.920214,4,1.599858,0,0,F3
11611,20230103992835,비비고 메추리알장조림 170g,7,9.467692,F,4,2023-01-03,NaN,0,9748,...,3.877505,717.668190,3,1,8.157156,9,2.197164,0,0,F4
11612,20230104999357,비비고 언양식 바싹불고기 460g,1,8.594895,M,4,2023-01-04,NaN,0,9748,...,4.230232,549.989758,4,1,8.762952,4,1.473311,0,0,M4


#### 주말 구매여부

In [26]:
# train
emp_tr['order_weekend'] = emp_tr.order_date.apply(lambda x : 1 if x.weekday() > 4 else 0)
nemp_tr['order_weekend'] = nemp_tr.order_date.apply(lambda x : 1 if x.weekday() > 4 else 0)

#test
emp_te['order_weekend'] = emp_te.order_date.apply(lambda x : 1 if x.weekday() > 4 else 0)
nemp_te['order_weekend'] = nemp_te.order_date.apply(lambda x : 1 if x.weekday() > 4 else 0)

display(emp_tr, nemp_tr, emp_te, nemp_te)

,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,order_amt_max_x_order_qty_max,day,day_label,product_amt_mean,product_qty_max,net_order_amt_max-min,bottom5,top5,gender+age,order_weekend
0,20230124153976,잔칫집 식혜 240ml 30입,1,9.803170,F,2,2023-01-24,0,0,5661,...,126.412939,24,4,9.868019,2,0.693118,0,0,F2,0
1,20230124155563,백설 한입쏙 비엔나 120g*2,1,8.256607,M,3,2023-01-24,0,0,14016,...,126.412939,24,4,8.451926,10,2.592984,0,0,M3,0
2,20230125160921,맥스봉 체다치즈어랏400g (20g*20EA),1,8.974998,F,2,2023-01-25,0,0,5661,...,302.000092,25,4,9.045646,2,0.705940,0,0,F2,0
3,20230125157569,고메 바삭쫄깃한 사천탕수육 450g,1,8.938007,M,4,2023-01-25,1,0,9878,...,302.000092,25,4,9.163596,3,1.501889,0,0,M4,0
4,20230125156655,고메 콘크림수프 180g*4입,1,8.997518,F,3,2023-01-25,0,0,14016,...,302.000092,25,4,8.996939,4,1.423942,0,0,F3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19800,20230102975115,[2023설사원선물신청] 흑삼정 골드클래스 100gx2,1,11.597276,M,5,2023-01-02,0,0,5196,...,696.373746,2,1,11.853054,9,2.215853,0,0,M5,0
19801,20230102972321,고메 바삭쫄깃한 탕수육 450g,1,8.702510,M,5,2023-01-02,1,0,5196,...,696.373746,2,1,9.120909,5,1.674637,0,0,M5,0
19802,20230101969615,리턴업 전립소 쏘팔메토 골드 1000mgX60캡슐(2개월)X2개,1,11.364344,F,2,2023-01-01,1,0,5661,...,506.659755,1,1,11.479985,3,1.016713,0,0,F2,1
19803,20230102972321,고메 거멍 모짜체다핫도그 340g,1,8.648397,M,5,2023-01-02,1,0,5196,...,696.373746,2,1,8.977414,2,1.042207,0,0,M5,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,order_amt_max_x_order_qty_max,day,day_label,product_amt_mean,product_qty_max,net_order_amt_max-min,bottom5,top5,gender+age,order_weekend
0,20230125158386,비비고 왕교자 1.05kg,1,9.348449,F,4,2023-01-25,0,0,20945,...,234.264358,25,4,9.273739,12,2.914180,0,0,F4,0
1,20230126164638,고메 바삭쫄깃한 탕수육 900g,1,9.667259,F,4,2023-01-26,1,0,20945,...,276.615768,26,4,9.410137,4,1.511256,0,0,F4,0
2,20230125159705,햇반 매일잡곡밥210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,234.264358,25,4,10.036068,20,0.082830,0,0,M4,0
3,20230126167696,[UPCYCLE]햇반 흑미밥 210gx24개(1box),1,10.125110,F,3,2023-01-26,0,1,10109,...,276.615768,26,4,10.177054,5,1.747883,0,0,F3,0
4,20230125159705,햇반 매일찰잡곡밥 210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,234.264358,25,4,8.976090,20,2.037125,0,0,M4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26065,20230101966635,고메 순살크리스피 치킨 고추크런치 450g,1,8.659560,M,3,2023-01-01,0,0,10109,...,695.412144,1,1,8.953284,7,2.108972,0,0,M3,1
26066,20230102974273,비비고 사골순댓국 460g,10,10.518646,M,4,2023-01-02,1,0,20945,...,733.709901,2,1,8.759052,20,3.074324,0,0,M4,0
26067,20230102972720,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,3,2023-01-02,0,0,10109,...,733.709901,2,1,9.713009,3,1.098566,0,0,F3,0
26068,20230101964953,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,4,2023-01-01,0,0,20945,...,695.412144,1,1,9.713009,3,1.098566,0,0,F4,1


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,order_amt_max_x_order_qty_max,day,day_label,product_amt_mean,product_qty_max,net_order_amt_max-min,bottom5,top5,gender+age,order_weekend
0,20230101970142,삼호 생선살어묵 야채 200g,1,8.098947,M,4,2023-01-01,NaN,0,4344,...,687.805328,1,1,8.143229,1,0.109817,0,0,M4,1
1,20230101964512,[앱전용특가] 쁘띠첼밀감90gX6개X3개,1,9.076809,M,4,2023-01-01,NaN,0,4344,...,687.805328,1,1,9.105797,1,0.086964,0,0,M4,1
2,20230102975046,빕스 볶음밥 1인분 X 6종_행사,1,9.928522,F,3,2023-01-02,NaN,1,6299,...,704.297911,2,1,10.125702,2,0.740922,0,0,F3,0
3,20230102973138,백설 진한참기름 500ml,2,10.060534,M,4,2023-01-02,NaN,0,4344,...,704.297911,2,1,9.552229,2,1.068972,0,0,M4,0
4,20230101969192,삼호 가쓰오모둠어묵탕413g,2,9.796181,M,4,2023-01-01,NaN,0,4344,...,687.805328,1,1,9.169970,3,1.291451,0,0,M4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8041,20230103994630,고메 콰트로포르마지 피자 310g,1,8.736650,M,5,2023-01-03,NaN,0,2051,...,700.524994,3,1,9.050980,2,1.096499,0,0,M5,0
8042,20230103994130,햇반솥반 전복내장영양밥 200g,2,9.196748,M,3,2023-01-03,NaN,1,6299,...,700.524994,3,1,8.900761,10,2.493781,0,0,M3,0
8043,20230103986821,비비고 찐만두168g,1,7.930566,M,5,2023-01-03,NaN,0,2051,...,700.524994,3,1,8.521369,4,1.357569,0,0,M5,0
8044,20230102979023,사골곰탕 한 그릇 100g (1-2인분X5개입),1,8.477412,F,3,2023-01-02,NaN,0,6299,...,704.297911,2,1,8.674647,2,0.693043,0,0,F3,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,order_amt_max_x_order_qty_max,day,day_label,product_amt_mean,product_qty_max,net_order_amt_max-min,bottom5,top5,gender+age,order_weekend
0,20230101964282,비비고 스팸부대찌개 460g,1,8.161946,M,4,2023-01-01,NaN,0,9748,...,698.791807,1,1,8.625450,10,2.316721,0,0,M4,1
1,20230101965237,크레잇 블랙페퍼 폭찹 스테이크,1,8.277412,F,4,2023-01-01,NaN,0,9748,...,698.791807,1,1,8.724876,6,1.845584,0,0,F4,1
2,20230102973798,비비고 소고기 미역국 500g,3,9.145375,F,3,2023-01-02,NaN,0,5034,...,733.709901,2,1,8.886063,10,2.422193,0,0,F3,0
3,20230101965633,비비고 소고기 미역국 500g,2,8.613230,F,3,2023-01-01,NaN,0,5034,...,698.791807,1,1,8.886063,10,2.422193,0,0,F3,1
4,20230101963753,고메 치즈크리스피핫도그 340g,1,8.540714,F,4,2023-01-01,NaN,0,9748,...,698.791807,1,1,8.766554,6,1.969910,0,0,F4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11609,20230102980326,고메 케이준프라이즈 피자 145g,2,8.671115,F,4,2023-01-02,NaN,0,9748,...,733.709901,2,1,8.505393,6,1.878765,0,0,F4,0
11610,20230102983068,고메 빅크리스피핫도그520g,1,8.641179,F,3,2023-01-02,NaN,0,5034,...,733.709901,2,1,8.920214,4,1.599858,0,0,F3,0
11611,20230103992835,비비고 메추리알장조림 170g,7,9.467692,F,4,2023-01-03,NaN,0,9748,...,717.668190,3,1,8.157156,9,2.197164,0,0,F4,0
11612,20230104999357,비비고 언양식 바싹불고기 460g,1,8.594895,M,4,2023-01-04,NaN,0,9748,...,549.989758,4,1,8.762952,4,1.473311,0,0,M4,0


#### 공휴일 구매여부

In [27]:
# train
emp_tr['order_holiday'] = emp_tr.order_date.apply(lambda x : int(x in pytimekr.holidays(year = 2023)))
nemp_tr['order_holiday'] = nemp_tr.order_date.apply(lambda x : int(x in pytimekr.holidays(year = 2023)))

#test
emp_te['order_holiday'] = emp_te.order_date.apply(lambda x : int(x in pytimekr.holidays(year = 2023)))
nemp_te['order_holiday'] = nemp_te.order_date.apply(lambda x : int(x in pytimekr.holidays(year = 2023)))

display(emp_tr, nemp_tr, emp_te, nemp_te)

C:\Users\82107\AppData\Local\Temp\ipykernel_26812\1044210079.py:2: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  emp_tr['order_holiday'] = emp_tr.order_date.apply(lambda x : int(x in pytimekr.holidays(year = 2023)))
C:\Users\82107\AppData\Local\Temp\ipykernel_26812\1044210079.py:3: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  nemp_tr['order_holiday'] = nemp_tr.order_date.apply(lambda x : int(x in pytimekr.holidays(year = 2023)))
C:\Users\82107\AppData\Local\Temp\ipykernel_26812\1044210079.py:6: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the sta

,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,day,day_label,product_amt_mean,product_qty_max,net_order_amt_max-min,bottom5,top5,gender+age,order_weekend,order_holiday
0,20230124153976,잔칫집 식혜 240ml 30입,1,9.803170,F,2,2023-01-24,0,0,5661,...,24,4,9.868019,2,0.693118,0,0,F2,0,0
1,20230124155563,백설 한입쏙 비엔나 120g*2,1,8.256607,M,3,2023-01-24,0,0,14016,...,24,4,8.451926,10,2.592984,0,0,M3,0,0
2,20230125160921,맥스봉 체다치즈어랏400g (20g*20EA),1,8.974998,F,2,2023-01-25,0,0,5661,...,25,4,9.045646,2,0.705940,0,0,F2,0,0
3,20230125157569,고메 바삭쫄깃한 사천탕수육 450g,1,8.938007,M,4,2023-01-25,1,0,9878,...,25,4,9.163596,3,1.501889,0,0,M4,0,0
4,20230125156655,고메 콘크림수프 180g*4입,1,8.997518,F,3,2023-01-25,0,0,14016,...,25,4,8.996939,4,1.423942,0,0,F3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19800,20230102975115,[2023설사원선물신청] 흑삼정 골드클래스 100gx2,1,11.597276,M,5,2023-01-02,0,0,5196,...,2,1,11.853054,9,2.215853,0,0,M5,0,0
19801,20230102972321,고메 바삭쫄깃한 탕수육 450g,1,8.702510,M,5,2023-01-02,1,0,5196,...,2,1,9.120909,5,1.674637,0,0,M5,0,0
19802,20230101969615,리턴업 전립소 쏘팔메토 골드 1000mgX60캡슐(2개월)X2개,1,11.364344,F,2,2023-01-01,1,0,5661,...,1,1,11.479985,3,1.016713,0,0,F2,1,1
19803,20230102972321,고메 거멍 모짜체다핫도그 340g,1,8.648397,M,5,2023-01-02,1,0,5196,...,2,1,8.977414,2,1.042207,0,0,M5,0,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,day,day_label,product_amt_mean,product_qty_max,net_order_amt_max-min,bottom5,top5,gender+age,order_weekend,order_holiday
0,20230125158386,비비고 왕교자 1.05kg,1,9.348449,F,4,2023-01-25,0,0,20945,...,25,4,9.273739,12,2.914180,0,0,F4,0,0
1,20230126164638,고메 바삭쫄깃한 탕수육 900g,1,9.667259,F,4,2023-01-26,1,0,20945,...,26,4,9.410137,4,1.511256,0,0,F4,0,0
2,20230125159705,햇반 매일잡곡밥210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,25,4,10.036068,20,0.082830,0,0,M4,0,0
3,20230126167696,[UPCYCLE]햇반 흑미밥 210gx24개(1box),1,10.125110,F,3,2023-01-26,0,1,10109,...,26,4,10.177054,5,1.747883,0,0,F3,0,0
4,20230125159705,햇반 매일찰잡곡밥 210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,25,4,8.976090,20,2.037125,0,0,M4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26065,20230101966635,고메 순살크리스피 치킨 고추크런치 450g,1,8.659560,M,3,2023-01-01,0,0,10109,...,1,1,8.953284,7,2.108972,0,0,M3,1,1
26066,20230102974273,비비고 사골순댓국 460g,10,10.518646,M,4,2023-01-02,1,0,20945,...,2,1,8.759052,20,3.074324,0,0,M4,0,0
26067,20230102972720,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,3,2023-01-02,0,0,10109,...,2,1,9.713009,3,1.098566,0,0,F3,0,0
26068,20230101964953,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,4,2023-01-01,0,0,20945,...,1,1,9.713009,3,1.098566,0,0,F4,1,1


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,day,day_label,product_amt_mean,product_qty_max,net_order_amt_max-min,bottom5,top5,gender+age,order_weekend,order_holiday
0,20230101970142,삼호 생선살어묵 야채 200g,1,8.098947,M,4,2023-01-01,NaN,0,4344,...,1,1,8.143229,1,0.109817,0,0,M4,1,1
1,20230101964512,[앱전용특가] 쁘띠첼밀감90gX6개X3개,1,9.076809,M,4,2023-01-01,NaN,0,4344,...,1,1,9.105797,1,0.086964,0,0,M4,1,1
2,20230102975046,빕스 볶음밥 1인분 X 6종_행사,1,9.928522,F,3,2023-01-02,NaN,1,6299,...,2,1,10.125702,2,0.740922,0,0,F3,0,0
3,20230102973138,백설 진한참기름 500ml,2,10.060534,M,4,2023-01-02,NaN,0,4344,...,2,1,9.552229,2,1.068972,0,0,M4,0,0
4,20230101969192,삼호 가쓰오모둠어묵탕413g,2,9.796181,M,4,2023-01-01,NaN,0,4344,...,1,1,9.169970,3,1.291451,0,0,M4,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8041,20230103994630,고메 콰트로포르마지 피자 310g,1,8.736650,M,5,2023-01-03,NaN,0,2051,...,3,1,9.050980,2,1.096499,0,0,M5,0,0
8042,20230103994130,햇반솥반 전복내장영양밥 200g,2,9.196748,M,3,2023-01-03,NaN,1,6299,...,3,1,8.900761,10,2.493781,0,0,M3,0,0
8043,20230103986821,비비고 찐만두168g,1,7.930566,M,5,2023-01-03,NaN,0,2051,...,3,1,8.521369,4,1.357569,0,0,M5,0,0
8044,20230102979023,사골곰탕 한 그릇 100g (1-2인분X5개입),1,8.477412,F,3,2023-01-02,NaN,0,6299,...,2,1,8.674647,2,0.693043,0,0,F3,0,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,day,day_label,product_amt_mean,product_qty_max,net_order_amt_max-min,bottom5,top5,gender+age,order_weekend,order_holiday
0,20230101964282,비비고 스팸부대찌개 460g,1,8.161946,M,4,2023-01-01,NaN,0,9748,...,1,1,8.625450,10,2.316721,0,0,M4,1,1
1,20230101965237,크레잇 블랙페퍼 폭찹 스테이크,1,8.277412,F,4,2023-01-01,NaN,0,9748,...,1,1,8.724876,6,1.845584,0,0,F4,1,1
2,20230102973798,비비고 소고기 미역국 500g,3,9.145375,F,3,2023-01-02,NaN,0,5034,...,2,1,8.886063,10,2.422193,0,0,F3,0,0
3,20230101965633,비비고 소고기 미역국 500g,2,8.613230,F,3,2023-01-01,NaN,0,5034,...,1,1,8.886063,10,2.422193,0,0,F3,1,1
4,20230101963753,고메 치즈크리스피핫도그 340g,1,8.540714,F,4,2023-01-01,NaN,0,9748,...,1,1,8.766554,6,1.969910,0,0,F4,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11609,20230102980326,고메 케이준프라이즈 피자 145g,2,8.671115,F,4,2023-01-02,NaN,0,9748,...,2,1,8.505393,6,1.878765,0,0,F4,0,0
11610,20230102983068,고메 빅크리스피핫도그520g,1,8.641179,F,3,2023-01-02,NaN,0,5034,...,2,1,8.920214,4,1.599858,0,0,F3,0,0
11611,20230103992835,비비고 메추리알장조림 170g,7,9.467692,F,4,2023-01-03,NaN,0,9748,...,3,1,8.157156,9,2.197164,0,0,F4,0,0
11612,20230104999357,비비고 언양식 바싹불고기 460g,1,8.594895,M,4,2023-01-04,NaN,0,9748,...,4,1,8.762952,4,1.473311,0,0,M4,0,0


#### 공유일 일주일 전 구매여부

In [28]:
# train
emp_tr['holiday_prev7d'] = emp_tr.order_date.apply(lambda x : int(x in [d - datetime.timedelta(days=7) for d in pytimekr.holidays(year = 2023)][:3]))
nemp_tr['holiday_prev7d'] = nemp_tr.order_date.apply(lambda x : int(x in [d - datetime.timedelta(days=7) for d in pytimekr.holidays(year = 2023)][:3]))

# test
# 공휴일 일주일 전 구매 여부
emp_te['holiday_prev7d'] = emp_te.order_date.apply(lambda x : int(x in [d - datetime.timedelta(days=7) for d in pytimekr.holidays(year = 2023)][:3]))
nemp_te['holiday_prev7d'] = nemp_te.order_date.apply(lambda x : int(x in [d - datetime.timedelta(days=7) for d in pytimekr.holidays(year = 2023)][:3]))

display(emp_tr, nemp_tr, emp_te, nemp_te)

C:\Users\82107\AppData\Local\Temp\ipykernel_26812\375322234.py:2: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  emp_tr['holiday_prev7d'] = emp_tr.order_date.apply(lambda x : int(x in [d - datetime.timedelta(days=7) for d in pytimekr.holidays(year = 2023)][:3]))
C:\Users\82107\AppData\Local\Temp\ipykernel_26812\375322234.py:3: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  nemp_tr['holiday_prev7d'] = nemp_tr.order_date.apply(lambda x : int(x in [d - datetime.timedelta(days=7) for d in pytimekr.holidays(year = 2023)][:3]))
C:\Users\82107\AppData\Local\Temp\ipykernel_26812\375322234.py:7: Future

,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,day_label,product_amt_mean,product_qty_max,net_order_amt_max-min,bottom5,top5,gender+age,order_weekend,order_holiday,holiday_prev7d
0,20230124153976,잔칫집 식혜 240ml 30입,1,9.803170,F,2,2023-01-24,0,0,5661,...,4,9.868019,2,0.693118,0,0,F2,0,0,0
1,20230124155563,백설 한입쏙 비엔나 120g*2,1,8.256607,M,3,2023-01-24,0,0,14016,...,4,8.451926,10,2.592984,0,0,M3,0,0,0
2,20230125160921,맥스봉 체다치즈어랏400g (20g*20EA),1,8.974998,F,2,2023-01-25,0,0,5661,...,4,9.045646,2,0.705940,0,0,F2,0,0,0
3,20230125157569,고메 바삭쫄깃한 사천탕수육 450g,1,8.938007,M,4,2023-01-25,1,0,9878,...,4,9.163596,3,1.501889,0,0,M4,0,0,0
4,20230125156655,고메 콘크림수프 180g*4입,1,8.997518,F,3,2023-01-25,0,0,14016,...,4,8.996939,4,1.423942,0,0,F3,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19800,20230102975115,[2023설사원선물신청] 흑삼정 골드클래스 100gx2,1,11.597276,M,5,2023-01-02,0,0,5196,...,1,11.853054,9,2.215853,0,0,M5,0,0,0
19801,20230102972321,고메 바삭쫄깃한 탕수육 450g,1,8.702510,M,5,2023-01-02,1,0,5196,...,1,9.120909,5,1.674637,0,0,M5,0,0,0
19802,20230101969615,리턴업 전립소 쏘팔메토 골드 1000mgX60캡슐(2개월)X2개,1,11.364344,F,2,2023-01-01,1,0,5661,...,1,11.479985,3,1.016713,0,0,F2,1,1,0
19803,20230102972321,고메 거멍 모짜체다핫도그 340g,1,8.648397,M,5,2023-01-02,1,0,5196,...,1,8.977414,2,1.042207,0,0,M5,0,0,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,day_label,product_amt_mean,product_qty_max,net_order_amt_max-min,bottom5,top5,gender+age,order_weekend,order_holiday,holiday_prev7d
0,20230125158386,비비고 왕교자 1.05kg,1,9.348449,F,4,2023-01-25,0,0,20945,...,4,9.273739,12,2.914180,0,0,F4,0,0,0
1,20230126164638,고메 바삭쫄깃한 탕수육 900g,1,9.667259,F,4,2023-01-26,1,0,20945,...,4,9.410137,4,1.511256,0,0,F4,0,0,0
2,20230125159705,햇반 매일잡곡밥210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,4,10.036068,20,0.082830,0,0,M4,0,0,0
3,20230126167696,[UPCYCLE]햇반 흑미밥 210gx24개(1box),1,10.125110,F,3,2023-01-26,0,1,10109,...,4,10.177054,5,1.747883,0,0,F3,0,0,0
4,20230125159705,햇반 매일찰잡곡밥 210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,4,8.976090,20,2.037125,0,0,M4,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26065,20230101966635,고메 순살크리스피 치킨 고추크런치 450g,1,8.659560,M,3,2023-01-01,0,0,10109,...,1,8.953284,7,2.108972,0,0,M3,1,1,0
26066,20230102974273,비비고 사골순댓국 460g,10,10.518646,M,4,2023-01-02,1,0,20945,...,1,8.759052,20,3.074324,0,0,M4,0,0,0
26067,20230102972720,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,3,2023-01-02,0,0,10109,...,1,9.713009,3,1.098566,0,0,F3,0,0,0
26068,20230101964953,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,4,2023-01-01,0,0,20945,...,1,9.713009,3,1.098566,0,0,F4,1,1,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,day_label,product_amt_mean,product_qty_max,net_order_amt_max-min,bottom5,top5,gender+age,order_weekend,order_holiday,holiday_prev7d
0,20230101970142,삼호 생선살어묵 야채 200g,1,8.098947,M,4,2023-01-01,NaN,0,4344,...,1,8.143229,1,0.109817,0,0,M4,1,1,0
1,20230101964512,[앱전용특가] 쁘띠첼밀감90gX6개X3개,1,9.076809,M,4,2023-01-01,NaN,0,4344,...,1,9.105797,1,0.086964,0,0,M4,1,1,0
2,20230102975046,빕스 볶음밥 1인분 X 6종_행사,1,9.928522,F,3,2023-01-02,NaN,1,6299,...,1,10.125702,2,0.740922,0,0,F3,0,0,0
3,20230102973138,백설 진한참기름 500ml,2,10.060534,M,4,2023-01-02,NaN,0,4344,...,1,9.552229,2,1.068972,0,0,M4,0,0,0
4,20230101969192,삼호 가쓰오모둠어묵탕413g,2,9.796181,M,4,2023-01-01,NaN,0,4344,...,1,9.169970,3,1.291451,0,0,M4,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8041,20230103994630,고메 콰트로포르마지 피자 310g,1,8.736650,M,5,2023-01-03,NaN,0,2051,...,1,9.050980,2,1.096499,0,0,M5,0,0,0
8042,20230103994130,햇반솥반 전복내장영양밥 200g,2,9.196748,M,3,2023-01-03,NaN,1,6299,...,1,8.900761,10,2.493781,0,0,M3,0,0,0
8043,20230103986821,비비고 찐만두168g,1,7.930566,M,5,2023-01-03,NaN,0,2051,...,1,8.521369,4,1.357569,0,0,M5,0,0,0
8044,20230102979023,사골곰탕 한 그릇 100g (1-2인분X5개입),1,8.477412,F,3,2023-01-02,NaN,0,6299,...,1,8.674647,2,0.693043,0,0,F3,0,0,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,day_label,product_amt_mean,product_qty_max,net_order_amt_max-min,bottom5,top5,gender+age,order_weekend,order_holiday,holiday_prev7d
0,20230101964282,비비고 스팸부대찌개 460g,1,8.161946,M,4,2023-01-01,NaN,0,9748,...,1,8.625450,10,2.316721,0,0,M4,1,1,0
1,20230101965237,크레잇 블랙페퍼 폭찹 스테이크,1,8.277412,F,4,2023-01-01,NaN,0,9748,...,1,8.724876,6,1.845584,0,0,F4,1,1,0
2,20230102973798,비비고 소고기 미역국 500g,3,9.145375,F,3,2023-01-02,NaN,0,5034,...,1,8.886063,10,2.422193,0,0,F3,0,0,0
3,20230101965633,비비고 소고기 미역국 500g,2,8.613230,F,3,2023-01-01,NaN,0,5034,...,1,8.886063,10,2.422193,0,0,F3,1,1,0
4,20230101963753,고메 치즈크리스피핫도그 340g,1,8.540714,F,4,2023-01-01,NaN,0,9748,...,1,8.766554,6,1.969910,0,0,F4,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11609,20230102980326,고메 케이준프라이즈 피자 145g,2,8.671115,F,4,2023-01-02,NaN,0,9748,...,1,8.505393,6,1.878765,0,0,F4,0,0,0
11610,20230102983068,고메 빅크리스피핫도그520g,1,8.641179,F,3,2023-01-02,NaN,0,5034,...,1,8.920214,4,1.599858,0,0,F3,0,0,0
11611,20230103992835,비비고 메추리알장조림 170g,7,9.467692,F,4,2023-01-03,NaN,0,9748,...,1,8.157156,9,2.197164,0,0,F4,0,0,0
11612,20230104999357,비비고 언양식 바싹불고기 460g,1,8.594895,M,4,2023-01-04,NaN,0,9748,...,1,8.762952,4,1.473311,0,0,M4,0,0,0


#### 월의 초,중,말의 구매여부 (초 : 1 ~ 10, 중 : 11 ~ 20, 말 : 21 ~ 31)

In [29]:
# train
emp_tr['week_for_10'] = emp_tr.order_date.apply(lambda x : 0 if x.day < 11 else 1 if x.day < 21 else 2 )
nemp_tr['week_for_10'] = nemp_tr.order_date.apply(lambda x : 0 if x.day < 11 else 1 if x.day < 21 else 2 )

#test
emp_te['week_for_10'] = emp_te.order_date.apply(lambda x : 0 if x.day < 11 else 1 if x.day < 21 else 2 )
nemp_te['week_for_10'] = nemp_te.order_date.apply(lambda x : 0 if x.day < 11 else 1 if x.day < 21 else 2 )

display(emp_tr, nemp_tr, emp_te, nemp_te)

,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,product_amt_mean,product_qty_max,net_order_amt_max-min,bottom5,top5,gender+age,order_weekend,order_holiday,holiday_prev7d,week_for_10
0,20230124153976,잔칫집 식혜 240ml 30입,1,9.803170,F,2,2023-01-24,0,0,5661,...,9.868019,2,0.693118,0,0,F2,0,0,0,2
1,20230124155563,백설 한입쏙 비엔나 120g*2,1,8.256607,M,3,2023-01-24,0,0,14016,...,8.451926,10,2.592984,0,0,M3,0,0,0,2
2,20230125160921,맥스봉 체다치즈어랏400g (20g*20EA),1,8.974998,F,2,2023-01-25,0,0,5661,...,9.045646,2,0.705940,0,0,F2,0,0,0,2
3,20230125157569,고메 바삭쫄깃한 사천탕수육 450g,1,8.938007,M,4,2023-01-25,1,0,9878,...,9.163596,3,1.501889,0,0,M4,0,0,0,2
4,20230125156655,고메 콘크림수프 180g*4입,1,8.997518,F,3,2023-01-25,0,0,14016,...,8.996939,4,1.423942,0,0,F3,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19800,20230102975115,[2023설사원선물신청] 흑삼정 골드클래스 100gx2,1,11.597276,M,5,2023-01-02,0,0,5196,...,11.853054,9,2.215853,0,0,M5,0,0,0,0
19801,20230102972321,고메 바삭쫄깃한 탕수육 450g,1,8.702510,M,5,2023-01-02,1,0,5196,...,9.120909,5,1.674637,0,0,M5,0,0,0,0
19802,20230101969615,리턴업 전립소 쏘팔메토 골드 1000mgX60캡슐(2개월)X2개,1,11.364344,F,2,2023-01-01,1,0,5661,...,11.479985,3,1.016713,0,0,F2,1,1,0,0
19803,20230102972321,고메 거멍 모짜체다핫도그 340g,1,8.648397,M,5,2023-01-02,1,0,5196,...,8.977414,2,1.042207,0,0,M5,0,0,0,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,product_amt_mean,product_qty_max,net_order_amt_max-min,bottom5,top5,gender+age,order_weekend,order_holiday,holiday_prev7d,week_for_10
0,20230125158386,비비고 왕교자 1.05kg,1,9.348449,F,4,2023-01-25,0,0,20945,...,9.273739,12,2.914180,0,0,F4,0,0,0,2
1,20230126164638,고메 바삭쫄깃한 탕수육 900g,1,9.667259,F,4,2023-01-26,1,0,20945,...,9.410137,4,1.511256,0,0,F4,0,0,0,2
2,20230125159705,햇반 매일잡곡밥210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,10.036068,20,0.082830,0,0,M4,0,0,0,2
3,20230126167696,[UPCYCLE]햇반 흑미밥 210gx24개(1box),1,10.125110,F,3,2023-01-26,0,1,10109,...,10.177054,5,1.747883,0,0,F3,0,0,0,2
4,20230125159705,햇반 매일찰잡곡밥 210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,8.976090,20,2.037125,0,0,M4,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26065,20230101966635,고메 순살크리스피 치킨 고추크런치 450g,1,8.659560,M,3,2023-01-01,0,0,10109,...,8.953284,7,2.108972,0,0,M3,1,1,0,0
26066,20230102974273,비비고 사골순댓국 460g,10,10.518646,M,4,2023-01-02,1,0,20945,...,8.759052,20,3.074324,0,0,M4,0,0,0,0
26067,20230102972720,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,3,2023-01-02,0,0,10109,...,9.713009,3,1.098566,0,0,F3,0,0,0,0
26068,20230101964953,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,4,2023-01-01,0,0,20945,...,9.713009,3,1.098566,0,0,F4,1,1,0,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,product_amt_mean,product_qty_max,net_order_amt_max-min,bottom5,top5,gender+age,order_weekend,order_holiday,holiday_prev7d,week_for_10
0,20230101970142,삼호 생선살어묵 야채 200g,1,8.098947,M,4,2023-01-01,NaN,0,4344,...,8.143229,1,0.109817,0,0,M4,1,1,0,0
1,20230101964512,[앱전용특가] 쁘띠첼밀감90gX6개X3개,1,9.076809,M,4,2023-01-01,NaN,0,4344,...,9.105797,1,0.086964,0,0,M4,1,1,0,0
2,20230102975046,빕스 볶음밥 1인분 X 6종_행사,1,9.928522,F,3,2023-01-02,NaN,1,6299,...,10.125702,2,0.740922,0,0,F3,0,0,0,0
3,20230102973138,백설 진한참기름 500ml,2,10.060534,M,4,2023-01-02,NaN,0,4344,...,9.552229,2,1.068972,0,0,M4,0,0,0,0
4,20230101969192,삼호 가쓰오모둠어묵탕413g,2,9.796181,M,4,2023-01-01,NaN,0,4344,...,9.169970,3,1.291451,0,0,M4,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8041,20230103994630,고메 콰트로포르마지 피자 310g,1,8.736650,M,5,2023-01-03,NaN,0,2051,...,9.050980,2,1.096499,0,0,M5,0,0,0,0
8042,20230103994130,햇반솥반 전복내장영양밥 200g,2,9.196748,M,3,2023-01-03,NaN,1,6299,...,8.900761,10,2.493781,0,0,M3,0,0,0,0
8043,20230103986821,비비고 찐만두168g,1,7.930566,M,5,2023-01-03,NaN,0,2051,...,8.521369,4,1.357569,0,0,M5,0,0,0,0
8044,20230102979023,사골곰탕 한 그릇 100g (1-2인분X5개입),1,8.477412,F,3,2023-01-02,NaN,0,6299,...,8.674647,2,0.693043,0,0,F3,0,0,0,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,product_amt_mean,product_qty_max,net_order_amt_max-min,bottom5,top5,gender+age,order_weekend,order_holiday,holiday_prev7d,week_for_10
0,20230101964282,비비고 스팸부대찌개 460g,1,8.161946,M,4,2023-01-01,NaN,0,9748,...,8.625450,10,2.316721,0,0,M4,1,1,0,0
1,20230101965237,크레잇 블랙페퍼 폭찹 스테이크,1,8.277412,F,4,2023-01-01,NaN,0,9748,...,8.724876,6,1.845584,0,0,F4,1,1,0,0
2,20230102973798,비비고 소고기 미역국 500g,3,9.145375,F,3,2023-01-02,NaN,0,5034,...,8.886063,10,2.422193,0,0,F3,0,0,0,0
3,20230101965633,비비고 소고기 미역국 500g,2,8.613230,F,3,2023-01-01,NaN,0,5034,...,8.886063,10,2.422193,0,0,F3,1,1,0,0
4,20230101963753,고메 치즈크리스피핫도그 340g,1,8.540714,F,4,2023-01-01,NaN,0,9748,...,8.766554,6,1.969910,0,0,F4,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11609,20230102980326,고메 케이준프라이즈 피자 145g,2,8.671115,F,4,2023-01-02,NaN,0,9748,...,8.505393,6,1.878765,0,0,F4,0,0,0,0
11610,20230102983068,고메 빅크리스피핫도그520g,1,8.641179,F,3,2023-01-02,NaN,0,5034,...,8.920214,4,1.599858,0,0,F3,0,0,0,0
11611,20230103992835,비비고 메추리알장조림 170g,7,9.467692,F,4,2023-01-03,NaN,0,9748,...,8.157156,9,2.197164,0,0,F4,0,0,0,0
11612,20230104999357,비비고 언양식 바싹불고기 460g,1,8.594895,M,4,2023-01-04,NaN,0,9748,...,8.762952,4,1.473311,0,0,M4,0,0,0,0


#### 설 상품 구매여부

In [30]:
# train
emp_tr['product_newyear'] = emp_tr.product_name.apply(lambda x : int("23설" in x))
nemp_tr['product_newyear'] = nemp_tr.product_name.apply(lambda x : int("23설" in x))

#test
emp_te['product_newyear'] = emp_te.product_name.apply(lambda x : int("23설" in x))
nemp_te['product_newyear'] = nemp_te.product_name.apply(lambda x : int("23설" in x))

display(emp_tr, nemp_tr, emp_te, nemp_te)

,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,product_qty_max,net_order_amt_max-min,bottom5,top5,gender+age,order_weekend,order_holiday,holiday_prev7d,week_for_10,product_newyear
0,20230124153976,잔칫집 식혜 240ml 30입,1,9.803170,F,2,2023-01-24,0,0,5661,...,2,0.693118,0,0,F2,0,0,0,2,0
1,20230124155563,백설 한입쏙 비엔나 120g*2,1,8.256607,M,3,2023-01-24,0,0,14016,...,10,2.592984,0,0,M3,0,0,0,2,0
2,20230125160921,맥스봉 체다치즈어랏400g (20g*20EA),1,8.974998,F,2,2023-01-25,0,0,5661,...,2,0.705940,0,0,F2,0,0,0,2,0
3,20230125157569,고메 바삭쫄깃한 사천탕수육 450g,1,8.938007,M,4,2023-01-25,1,0,9878,...,3,1.501889,0,0,M4,0,0,0,2,0
4,20230125156655,고메 콘크림수프 180g*4입,1,8.997518,F,3,2023-01-25,0,0,14016,...,4,1.423942,0,0,F3,0,0,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19800,20230102975115,[2023설사원선물신청] 흑삼정 골드클래스 100gx2,1,11.597276,M,5,2023-01-02,0,0,5196,...,9,2.215853,0,0,M5,0,0,0,0,1
19801,20230102972321,고메 바삭쫄깃한 탕수육 450g,1,8.702510,M,5,2023-01-02,1,0,5196,...,5,1.674637,0,0,M5,0,0,0,0,0
19802,20230101969615,리턴업 전립소 쏘팔메토 골드 1000mgX60캡슐(2개월)X2개,1,11.364344,F,2,2023-01-01,1,0,5661,...,3,1.016713,0,0,F2,1,1,0,0,0
19803,20230102972321,고메 거멍 모짜체다핫도그 340g,1,8.648397,M,5,2023-01-02,1,0,5196,...,2,1.042207,0,0,M5,0,0,0,0,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,product_qty_max,net_order_amt_max-min,bottom5,top5,gender+age,order_weekend,order_holiday,holiday_prev7d,week_for_10,product_newyear
0,20230125158386,비비고 왕교자 1.05kg,1,9.348449,F,4,2023-01-25,0,0,20945,...,12,2.914180,0,0,F4,0,0,0,2,0
1,20230126164638,고메 바삭쫄깃한 탕수육 900g,1,9.667259,F,4,2023-01-26,1,0,20945,...,4,1.511256,0,0,F4,0,0,0,2,0
2,20230125159705,햇반 매일잡곡밥210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,20,0.082830,0,0,M4,0,0,0,2,0
3,20230126167696,[UPCYCLE]햇반 흑미밥 210gx24개(1box),1,10.125110,F,3,2023-01-26,0,1,10109,...,5,1.747883,0,0,F3,0,0,0,2,0
4,20230125159705,햇반 매일찰잡곡밥 210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,20,2.037125,0,0,M4,0,0,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26065,20230101966635,고메 순살크리스피 치킨 고추크런치 450g,1,8.659560,M,3,2023-01-01,0,0,10109,...,7,2.108972,0,0,M3,1,1,0,0,0
26066,20230102974273,비비고 사골순댓국 460g,10,10.518646,M,4,2023-01-02,1,0,20945,...,20,3.074324,0,0,M4,0,0,0,0,0
26067,20230102972720,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,3,2023-01-02,0,0,10109,...,3,1.098566,0,0,F3,0,0,0,0,0
26068,20230101964953,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,4,2023-01-01,0,0,20945,...,3,1.098566,0,0,F4,1,1,0,0,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,product_qty_max,net_order_amt_max-min,bottom5,top5,gender+age,order_weekend,order_holiday,holiday_prev7d,week_for_10,product_newyear
0,20230101970142,삼호 생선살어묵 야채 200g,1,8.098947,M,4,2023-01-01,NaN,0,4344,...,1,0.109817,0,0,M4,1,1,0,0,0
1,20230101964512,[앱전용특가] 쁘띠첼밀감90gX6개X3개,1,9.076809,M,4,2023-01-01,NaN,0,4344,...,1,0.086964,0,0,M4,1,1,0,0,0
2,20230102975046,빕스 볶음밥 1인분 X 6종_행사,1,9.928522,F,3,2023-01-02,NaN,1,6299,...,2,0.740922,0,0,F3,0,0,0,0,0
3,20230102973138,백설 진한참기름 500ml,2,10.060534,M,4,2023-01-02,NaN,0,4344,...,2,1.068972,0,0,M4,0,0,0,0,0
4,20230101969192,삼호 가쓰오모둠어묵탕413g,2,9.796181,M,4,2023-01-01,NaN,0,4344,...,3,1.291451,0,0,M4,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8041,20230103994630,고메 콰트로포르마지 피자 310g,1,8.736650,M,5,2023-01-03,NaN,0,2051,...,2,1.096499,0,0,M5,0,0,0,0,0
8042,20230103994130,햇반솥반 전복내장영양밥 200g,2,9.196748,M,3,2023-01-03,NaN,1,6299,...,10,2.493781,0,0,M3,0,0,0,0,0
8043,20230103986821,비비고 찐만두168g,1,7.930566,M,5,2023-01-03,NaN,0,2051,...,4,1.357569,0,0,M5,0,0,0,0,0
8044,20230102979023,사골곰탕 한 그릇 100g (1-2인분X5개입),1,8.477412,F,3,2023-01-02,NaN,0,6299,...,2,0.693043,0,0,F3,0,0,0,0,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,product_qty_max,net_order_amt_max-min,bottom5,top5,gender+age,order_weekend,order_holiday,holiday_prev7d,week_for_10,product_newyear
0,20230101964282,비비고 스팸부대찌개 460g,1,8.161946,M,4,2023-01-01,NaN,0,9748,...,10,2.316721,0,0,M4,1,1,0,0,0
1,20230101965237,크레잇 블랙페퍼 폭찹 스테이크,1,8.277412,F,4,2023-01-01,NaN,0,9748,...,6,1.845584,0,0,F4,1,1,0,0,0
2,20230102973798,비비고 소고기 미역국 500g,3,9.145375,F,3,2023-01-02,NaN,0,5034,...,10,2.422193,0,0,F3,0,0,0,0,0
3,20230101965633,비비고 소고기 미역국 500g,2,8.613230,F,3,2023-01-01,NaN,0,5034,...,10,2.422193,0,0,F3,1,1,0,0,0
4,20230101963753,고메 치즈크리스피핫도그 340g,1,8.540714,F,4,2023-01-01,NaN,0,9748,...,6,1.969910,0,0,F4,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11609,20230102980326,고메 케이준프라이즈 피자 145g,2,8.671115,F,4,2023-01-02,NaN,0,9748,...,6,1.878765,0,0,F4,0,0,0,0,0
11610,20230102983068,고메 빅크리스피핫도그520g,1,8.641179,F,3,2023-01-02,NaN,0,5034,...,4,1.599858,0,0,F3,0,0,0,0,0
11611,20230103992835,비비고 메추리알장조림 170g,7,9.467692,F,4,2023-01-03,NaN,0,9748,...,9,2.197164,0,0,F4,0,0,0,0,0
11612,20230104999357,비비고 언양식 바싹불고기 460g,1,8.594895,M,4,2023-01-04,NaN,0,9748,...,4,1.473311,0,0,M4,0,0,0,0,0


#### 구매물품의 개당 구매 가격

In [31]:
# train
emp_tr['amt_per_qty'] = emp_tr.net_order_amt / emp_tr.net_order_qty
nemp_tr['amt_per_qty'] = nemp_tr.net_order_amt / nemp_tr.net_order_qty

#test
emp_te['amt_per_qty'] = emp_te.net_order_amt / emp_te.net_order_qty
nemp_te['amt_per_qty'] = nemp_te.net_order_amt / nemp_te.net_order_qty

display(emp_tr, nemp_tr, emp_te, nemp_te)

,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,net_order_amt_max-min,bottom5,top5,gender+age,order_weekend,order_holiday,holiday_prev7d,week_for_10,product_newyear,amt_per_qty
0,20230124153976,잔칫집 식혜 240ml 30입,1,9.803170,F,2,2023-01-24,0,0,5661,...,0.693118,0,0,F2,0,0,0,2,0,9.803170
1,20230124155563,백설 한입쏙 비엔나 120g*2,1,8.256607,M,3,2023-01-24,0,0,14016,...,2.592984,0,0,M3,0,0,0,2,0,8.256607
2,20230125160921,맥스봉 체다치즈어랏400g (20g*20EA),1,8.974998,F,2,2023-01-25,0,0,5661,...,0.705940,0,0,F2,0,0,0,2,0,8.974998
3,20230125157569,고메 바삭쫄깃한 사천탕수육 450g,1,8.938007,M,4,2023-01-25,1,0,9878,...,1.501889,0,0,M4,0,0,0,2,0,8.938007
4,20230125156655,고메 콘크림수프 180g*4입,1,8.997518,F,3,2023-01-25,0,0,14016,...,1.423942,0,0,F3,0,0,0,2,0,8.997518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19800,20230102975115,[2023설사원선물신청] 흑삼정 골드클래스 100gx2,1,11.597276,M,5,2023-01-02,0,0,5196,...,2.215853,0,0,M5,0,0,0,0,1,11.597276
19801,20230102972321,고메 바삭쫄깃한 탕수육 450g,1,8.702510,M,5,2023-01-02,1,0,5196,...,1.674637,0,0,M5,0,0,0,0,0,8.702510
19802,20230101969615,리턴업 전립소 쏘팔메토 골드 1000mgX60캡슐(2개월)X2개,1,11.364344,F,2,2023-01-01,1,0,5661,...,1.016713,0,0,F2,1,1,0,0,0,11.364344
19803,20230102972321,고메 거멍 모짜체다핫도그 340g,1,8.648397,M,5,2023-01-02,1,0,5196,...,1.042207,0,0,M5,0,0,0,0,0,8.648397


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,net_order_amt_max-min,bottom5,top5,gender+age,order_weekend,order_holiday,holiday_prev7d,week_for_10,product_newyear,amt_per_qty
0,20230125158386,비비고 왕교자 1.05kg,1,9.348449,F,4,2023-01-25,0,0,20945,...,2.914180,0,0,F4,0,0,0,2,0,9.348449
1,20230126164638,고메 바삭쫄깃한 탕수육 900g,1,9.667259,F,4,2023-01-26,1,0,20945,...,1.511256,0,0,F4,0,0,0,2,0,9.667259
2,20230125159705,햇반 매일잡곡밥210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,0.082830,0,0,M4,0,0,0,2,0,0.499733
3,20230126167696,[UPCYCLE]햇반 흑미밥 210gx24개(1box),1,10.125110,F,3,2023-01-26,0,1,10109,...,1.747883,0,0,F3,0,0,0,2,0,10.125110
4,20230125159705,햇반 매일찰잡곡밥 210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,2.037125,0,0,M4,0,0,0,2,0,0.499733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26065,20230101966635,고메 순살크리스피 치킨 고추크런치 450g,1,8.659560,M,3,2023-01-01,0,0,10109,...,2.108972,0,0,M3,1,1,0,0,0,8.659560
26066,20230102974273,비비고 사골순댓국 460g,10,10.518646,M,4,2023-01-02,1,0,20945,...,3.074324,0,0,M4,0,0,0,0,0,1.051865
26067,20230102972720,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,3,2023-01-02,0,0,10109,...,1.098566,0,0,F3,0,0,0,0,0,9.639327
26068,20230101964953,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,4,2023-01-01,0,0,20945,...,1.098566,0,0,F4,1,1,0,0,0,9.639327


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,net_order_amt_max-min,bottom5,top5,gender+age,order_weekend,order_holiday,holiday_prev7d,week_for_10,product_newyear,amt_per_qty
0,20230101970142,삼호 생선살어묵 야채 200g,1,8.098947,M,4,2023-01-01,NaN,0,4344,...,0.109817,0,0,M4,1,1,0,0,0,8.098947
1,20230101964512,[앱전용특가] 쁘띠첼밀감90gX6개X3개,1,9.076809,M,4,2023-01-01,NaN,0,4344,...,0.086964,0,0,M4,1,1,0,0,0,9.076809
2,20230102975046,빕스 볶음밥 1인분 X 6종_행사,1,9.928522,F,3,2023-01-02,NaN,1,6299,...,0.740922,0,0,F3,0,0,0,0,0,9.928522
3,20230102973138,백설 진한참기름 500ml,2,10.060534,M,4,2023-01-02,NaN,0,4344,...,1.068972,0,0,M4,0,0,0,0,0,5.030267
4,20230101969192,삼호 가쓰오모둠어묵탕413g,2,9.796181,M,4,2023-01-01,NaN,0,4344,...,1.291451,0,0,M4,1,1,0,0,0,4.898090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8041,20230103994630,고메 콰트로포르마지 피자 310g,1,8.736650,M,5,2023-01-03,NaN,0,2051,...,1.096499,0,0,M5,0,0,0,0,0,8.736650
8042,20230103994130,햇반솥반 전복내장영양밥 200g,2,9.196748,M,3,2023-01-03,NaN,1,6299,...,2.493781,0,0,M3,0,0,0,0,0,4.598374
8043,20230103986821,비비고 찐만두168g,1,7.930566,M,5,2023-01-03,NaN,0,2051,...,1.357569,0,0,M5,0,0,0,0,0,7.930566
8044,20230102979023,사골곰탕 한 그릇 100g (1-2인분X5개입),1,8.477412,F,3,2023-01-02,NaN,0,6299,...,0.693043,0,0,F3,0,0,0,0,0,8.477412


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,net_order_amt_max-min,bottom5,top5,gender+age,order_weekend,order_holiday,holiday_prev7d,week_for_10,product_newyear,amt_per_qty
0,20230101964282,비비고 스팸부대찌개 460g,1,8.161946,M,4,2023-01-01,NaN,0,9748,...,2.316721,0,0,M4,1,1,0,0,0,8.161946
1,20230101965237,크레잇 블랙페퍼 폭찹 스테이크,1,8.277412,F,4,2023-01-01,NaN,0,9748,...,1.845584,0,0,F4,1,1,0,0,0,8.277412
2,20230102973798,비비고 소고기 미역국 500g,3,9.145375,F,3,2023-01-02,NaN,0,5034,...,2.422193,0,0,F3,0,0,0,0,0,3.048458
3,20230101965633,비비고 소고기 미역국 500g,2,8.613230,F,3,2023-01-01,NaN,0,5034,...,2.422193,0,0,F3,1,1,0,0,0,4.306615
4,20230101963753,고메 치즈크리스피핫도그 340g,1,8.540714,F,4,2023-01-01,NaN,0,9748,...,1.969910,0,0,F4,1,1,0,0,0,8.540714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11609,20230102980326,고메 케이준프라이즈 피자 145g,2,8.671115,F,4,2023-01-02,NaN,0,9748,...,1.878765,0,0,F4,0,0,0,0,0,4.335558
11610,20230102983068,고메 빅크리스피핫도그520g,1,8.641179,F,3,2023-01-02,NaN,0,5034,...,1.599858,0,0,F3,0,0,0,0,0,8.641179
11611,20230103992835,비비고 메추리알장조림 170g,7,9.467692,F,4,2023-01-03,NaN,0,9748,...,2.197164,0,0,F4,0,0,0,0,0,1.352527
11612,20230104999357,비비고 언양식 바싹불고기 460g,1,8.594895,M,4,2023-01-04,NaN,0,9748,...,1.473311,0,0,M4,0,0,0,0,0,8.594895


#### 고객별 구매품목 다양성

In [32]:
# train
f_tr = emp_tr.groupby('scd')['product_name'].agg([('buy_product_cnt',np.size)]).reset_index()
nf_tr = nemp_tr.groupby('scd')['product_name'].agg([('buy_product_cnt',np.size)]).reset_index()

emp_tr = pd.merge(emp_tr, f_tr, on= 'scd', how= 'left')
nemp_tr = pd.merge(nemp_tr, nf_tr, on= 'scd', how= 'left')

#test
f_te = emp_te.groupby('scd')['product_name'].agg([('buy_product_cnt',np.size)]).reset_index()
nf_te = nemp_te.groupby('scd')['product_name'].agg([('buy_product_cnt',np.size)]).reset_index()

emp_te = pd.merge(emp_te, f_te, on= 'scd', how= 'left')
nemp_te = pd.merge(nemp_te, nf_te, on= 'scd', how= 'left')

display(f_tr, nf_tr, f_te, nf_te)

,scd,buy_product_cnt
0,20230101963226,4
1,20230101963235,2
2,20230101963247,5
3,20230101963251,1
4,20230101963253,5
...,...,...
4708,20230131216567,3
4709,20230131216610,9
4710,20230131216674,4
4711,20230131216734,3


,scd,buy_product_cnt
0,20230101963244,1
1,20230101963277,1
2,20230101963302,10
3,20230101963319,8
4,20230101963336,14
...,...,...
5935,20230131216709,3
5936,20230131216756,2
5937,20230131216771,1
5938,20230131216842,1


,scd,buy_product_cnt
0,20230101963226,4
1,20230101963229,1
2,20230101963246,1
3,20230101963265,5
4,20230101963284,3
...,...,...
2682,20230131215141,2
2683,20230131215302,2
2684,20230131215537,1
2685,20230131215893,1


,scd,buy_product_cnt
0,20230101963221,1
1,20230101963286,1
2,20230101963302,2
3,20230101963306,11
4,20230101963315,1
...,...,...
3636,20230131216228,1
3637,20230131216338,2
3638,20230131216371,2
3639,20230131216446,1


#### 고객별 구매상품의 평균 개당 가격

In [33]:
# train
f_tr = emp_tr.groupby('scd')['amt_per_qty'].agg([('amt_per_qty_mean',np.mean)]).reset_index()
nf_tr = nemp_tr.groupby('scd')['amt_per_qty'].agg([('amt_per_qty_mean',np.mean)]).reset_index()

emp_tr = pd.merge(emp_tr, f_tr, on= 'scd', how= 'left')
nemp_tr = pd.merge(nemp_tr, nf_tr, on= 'scd', how= 'left')

#test
f_te = emp_te.groupby('scd')['amt_per_qty'].agg([('amt_per_qty_mean',np.mean)]).reset_index()
nf_te = nemp_te.groupby('scd')['amt_per_qty'].agg([('amt_per_qty_mean',np.mean)]).reset_index()

emp_te = pd.merge(emp_te, f_te, on= 'scd', how= 'left')
nemp_te = pd.merge(nemp_te, nf_te, on= 'scd', how= 'left')

display(f_tr, nf_tr, f_te, nf_te)

,scd,amt_per_qty_mean
0,20230101963226,7.609629
1,20230101963235,9.617283
2,20230101963247,8.070212
3,20230101963251,5.422811
4,20230101963253,8.445540
...,...,...
4708,20230131216567,5.733350
4709,20230131216610,8.342528
4710,20230131216674,9.329461
4711,20230131216734,9.833438


,scd,amt_per_qty_mean
0,20230101963244,3.411458
1,20230101963277,9.190036
2,20230101963302,7.574948
3,20230101963319,4.093077
4,20230101963336,6.208069
...,...,...
5935,20230131216709,7.037744
5936,20230131216756,6.796144
5937,20230131216771,4.562881
5938,20230131216842,10.176411


,scd,amt_per_qty_mean
0,20230101963226,4.665145
1,20230101963229,9.003808
2,20230101963246,4.787040
3,20230101963265,8.733158
4,20230101963284,8.065759
...,...,...
2682,20230131215141,5.510522
2683,20230131215302,9.289855
2684,20230131215537,8.847216
2685,20230131215893,8.302266


,scd,amt_per_qty_mean
0,20230101963221,7.959975
1,20230101963286,3.285832
2,20230101963302,8.427714
3,20230101963306,7.163753
4,20230101963315,2.546461
...,...,...
3636,20230131216228,4.510980
3637,20230131216338,6.092155
3638,20230131216371,8.009825
3639,20230131216446,8.655214


#### 박스단위 상품 구매여부

In [34]:
# train
emp_tr['1box'] = emp_tr.product_name.apply(lambda x : int("1box" in x))
nemp_tr['1box'] = nemp_tr.product_name.apply(lambda x : int("1box" in x))

#test
emp_te['1box'] = emp_te.product_name.apply(lambda x : int("1box" in x))
nemp_te['1box'] = nemp_te.product_name.apply(lambda x : int("1box" in x))

display(emp_tr, nemp_tr, emp_te, nemp_te)

,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,gender+age,order_weekend,order_holiday,holiday_prev7d,week_for_10,product_newyear,amt_per_qty,buy_product_cnt,amt_per_qty_mean,1box
0,20230124153976,잔칫집 식혜 240ml 30입,1,9.803170,F,2,2023-01-24,0,0,5661,...,F2,0,0,0,2,0,9.803170,10,9.280165,0
1,20230124155563,백설 한입쏙 비엔나 120g*2,1,8.256607,M,3,2023-01-24,0,0,14016,...,M3,0,0,0,2,0,8.256607,17,8.675043,0
2,20230125160921,맥스봉 체다치즈어랏400g (20g*20EA),1,8.974998,F,2,2023-01-25,0,0,5661,...,F2,0,0,0,2,0,8.974998,6,3.940153,0
3,20230125157569,고메 바삭쫄깃한 사천탕수육 450g,1,8.938007,M,4,2023-01-25,1,0,9878,...,M4,0,0,0,2,0,8.938007,16,8.174998,0
4,20230125156655,고메 콘크림수프 180g*4입,1,8.997518,F,3,2023-01-25,0,0,14016,...,F3,0,0,0,2,0,8.997518,9,9.019387,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19800,20230102975115,[2023설사원선물신청] 흑삼정 골드클래스 100gx2,1,11.597276,M,5,2023-01-02,0,0,5196,...,M5,0,0,0,0,1,11.597276,1,11.597276,0
19801,20230102972321,고메 바삭쫄깃한 탕수육 450g,1,8.702510,M,5,2023-01-02,1,0,5196,...,M5,0,0,0,0,0,8.702510,13,7.678905,0
19802,20230101969615,리턴업 전립소 쏘팔메토 골드 1000mgX60캡슐(2개월)X2개,1,11.364344,F,2,2023-01-01,1,0,5661,...,F2,1,1,0,0,0,11.364344,1,11.364344,0
19803,20230102972321,고메 거멍 모짜체다핫도그 340g,1,8.648397,M,5,2023-01-02,1,0,5196,...,M5,0,0,0,0,0,8.648397,13,7.678905,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,gender+age,order_weekend,order_holiday,holiday_prev7d,week_for_10,product_newyear,amt_per_qty,buy_product_cnt,amt_per_qty_mean,1box
0,20230125158386,비비고 왕교자 1.05kg,1,9.348449,F,4,2023-01-25,0,0,20945,...,F4,0,0,0,2,0,9.348449,1,9.348449,0
1,20230126164638,고메 바삭쫄깃한 탕수육 900g,1,9.667259,F,4,2023-01-26,1,0,20945,...,F4,0,0,0,2,0,9.667259,3,7.778245,0
2,20230125159705,햇반 매일잡곡밥210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,M4,0,0,0,2,0,0.499733,8,4.151479,0
3,20230126167696,[UPCYCLE]햇반 흑미밥 210gx24개(1box),1,10.125110,F,3,2023-01-26,0,1,10109,...,F3,0,0,0,2,0,10.125110,1,10.125110,1
4,20230125159705,햇반 매일찰잡곡밥 210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,M4,0,0,0,2,0,0.499733,8,4.151479,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26065,20230101966635,고메 순살크리스피 치킨 고추크런치 450g,1,8.659560,M,3,2023-01-01,0,0,10109,...,M3,1,1,0,0,0,8.659560,5,7.706661,0
26066,20230102974273,비비고 사골순댓국 460g,10,10.518646,M,4,2023-01-02,1,0,20945,...,M4,0,0,0,0,0,1.051865,6,2.517459,0
26067,20230102972720,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,3,2023-01-02,0,0,10109,...,F3,0,0,0,0,0,9.639327,2,9.478408,0
26068,20230101964953,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,4,2023-01-01,0,0,20945,...,F4,1,1,0,0,0,9.639327,1,9.639327,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,gender+age,order_weekend,order_holiday,holiday_prev7d,week_for_10,product_newyear,amt_per_qty,buy_product_cnt,amt_per_qty_mean,1box
0,20230101970142,삼호 생선살어묵 야채 200g,1,8.098947,M,4,2023-01-01,NaN,0,4344,...,M4,1,1,0,0,0,8.098947,6,8.788789,0
1,20230101964512,[앱전용특가] 쁘띠첼밀감90gX6개X3개,1,9.076809,M,4,2023-01-01,NaN,0,4344,...,M4,1,1,0,0,0,9.076809,4,5.861183,0
2,20230102975046,빕스 볶음밥 1인분 X 6종_행사,1,9.928522,F,3,2023-01-02,NaN,1,6299,...,F3,0,0,0,0,0,9.928522,1,9.928522,0
3,20230102973138,백설 진한참기름 500ml,2,10.060534,M,4,2023-01-02,NaN,0,4344,...,M4,0,0,0,0,0,5.030267,1,5.030267,0
4,20230101969192,삼호 가쓰오모둠어묵탕413g,2,9.796181,M,4,2023-01-01,NaN,0,4344,...,M4,1,1,0,0,0,4.898090,8,7.910389,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8041,20230103994630,고메 콰트로포르마지 피자 310g,1,8.736650,M,5,2023-01-03,NaN,0,2051,...,M5,0,0,0,0,0,8.736650,3,6.759689,0
8042,20230103994130,햇반솥반 전복내장영양밥 200g,2,9.196748,M,3,2023-01-03,NaN,1,6299,...,M3,0,0,0,0,0,4.598374,3,6.263244,0
8043,20230103986821,비비고 찐만두168g,1,7.930566,M,5,2023-01-03,NaN,0,2051,...,M5,0,0,0,0,0,7.930566,6,6.987001,0
8044,20230102979023,사골곰탕 한 그릇 100g (1-2인분X5개입),1,8.477412,F,3,2023-01-02,NaN,0,6299,...,F3,0,0,0,0,0,8.477412,2,8.162001,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,gender+age,order_weekend,order_holiday,holiday_prev7d,week_for_10,product_newyear,amt_per_qty,buy_product_cnt,amt_per_qty_mean,1box
0,20230101964282,비비고 스팸부대찌개 460g,1,8.161946,M,4,2023-01-01,NaN,0,9748,...,M4,1,1,0,0,0,8.161946,4,8.164507,0
1,20230101965237,크레잇 블랙페퍼 폭찹 스테이크,1,8.277412,F,4,2023-01-01,NaN,0,9748,...,F4,1,1,0,0,0,8.277412,5,5.942704,0
2,20230102973798,비비고 소고기 미역국 500g,3,9.145375,F,3,2023-01-02,NaN,0,5034,...,F3,0,0,0,0,0,3.048458,2,5.668622,0
3,20230101965633,비비고 소고기 미역국 500g,2,8.613230,F,3,2023-01-01,NaN,0,5034,...,F3,1,1,0,0,0,4.306615,3,4.548013,0
4,20230101963753,고메 치즈크리스피핫도그 340g,1,8.540714,F,4,2023-01-01,NaN,0,9748,...,F4,1,1,0,0,0,8.540714,2,5.876901,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11609,20230102980326,고메 케이준프라이즈 피자 145g,2,8.671115,F,4,2023-01-02,NaN,0,9748,...,F4,0,0,0,0,0,4.335558,5,6.486198,0
11610,20230102983068,고메 빅크리스피핫도그520g,1,8.641179,F,3,2023-01-02,NaN,0,5034,...,F3,0,0,0,0,0,8.641179,2,8.535317,0
11611,20230103992835,비비고 메추리알장조림 170g,7,9.467692,F,4,2023-01-03,NaN,0,9748,...,F4,0,0,0,0,0,1.352527,3,4.743072,0
11612,20230104999357,비비고 언양식 바싹불고기 460g,1,8.594895,M,4,2023-01-04,NaN,0,9748,...,M4,0,0,0,0,0,8.594895,5,7.583406,0


#### 닭가슴살 상품 구매여부

In [35]:
# train
emp_tr['chicken breast'] = emp_tr.product_name.apply(lambda x : int('닭가슴살' in x))
nemp_tr['chicken breast'] = nemp_tr.product_name.apply(lambda x : int('닭가슴살' in x))

#test
emp_te['chicken breast'] = emp_te.product_name.apply(lambda x : int('닭가슴살' in x))
nemp_te['chicken breast'] = nemp_te.product_name.apply(lambda x : int('닭가슴살' in x))

display(emp_tr, nemp_tr, emp_te, nemp_te)

,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,order_weekend,order_holiday,holiday_prev7d,week_for_10,product_newyear,amt_per_qty,buy_product_cnt,amt_per_qty_mean,1box,chicken breast
0,20230124153976,잔칫집 식혜 240ml 30입,1,9.803170,F,2,2023-01-24,0,0,5661,...,0,0,0,2,0,9.803170,10,9.280165,0,0
1,20230124155563,백설 한입쏙 비엔나 120g*2,1,8.256607,M,3,2023-01-24,0,0,14016,...,0,0,0,2,0,8.256607,17,8.675043,0,0
2,20230125160921,맥스봉 체다치즈어랏400g (20g*20EA),1,8.974998,F,2,2023-01-25,0,0,5661,...,0,0,0,2,0,8.974998,6,3.940153,0,0
3,20230125157569,고메 바삭쫄깃한 사천탕수육 450g,1,8.938007,M,4,2023-01-25,1,0,9878,...,0,0,0,2,0,8.938007,16,8.174998,0,0
4,20230125156655,고메 콘크림수프 180g*4입,1,8.997518,F,3,2023-01-25,0,0,14016,...,0,0,0,2,0,8.997518,9,9.019387,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19800,20230102975115,[2023설사원선물신청] 흑삼정 골드클래스 100gx2,1,11.597276,M,5,2023-01-02,0,0,5196,...,0,0,0,0,1,11.597276,1,11.597276,0,0
19801,20230102972321,고메 바삭쫄깃한 탕수육 450g,1,8.702510,M,5,2023-01-02,1,0,5196,...,0,0,0,0,0,8.702510,13,7.678905,0,0
19802,20230101969615,리턴업 전립소 쏘팔메토 골드 1000mgX60캡슐(2개월)X2개,1,11.364344,F,2,2023-01-01,1,0,5661,...,1,1,0,0,0,11.364344,1,11.364344,0,0
19803,20230102972321,고메 거멍 모짜체다핫도그 340g,1,8.648397,M,5,2023-01-02,1,0,5196,...,0,0,0,0,0,8.648397,13,7.678905,0,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,order_weekend,order_holiday,holiday_prev7d,week_for_10,product_newyear,amt_per_qty,buy_product_cnt,amt_per_qty_mean,1box,chicken breast
0,20230125158386,비비고 왕교자 1.05kg,1,9.348449,F,4,2023-01-25,0,0,20945,...,0,0,0,2,0,9.348449,1,9.348449,0,0
1,20230126164638,고메 바삭쫄깃한 탕수육 900g,1,9.667259,F,4,2023-01-26,1,0,20945,...,0,0,0,2,0,9.667259,3,7.778245,0,0
2,20230125159705,햇반 매일잡곡밥210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,0,0,0,2,0,0.499733,8,4.151479,0,0
3,20230126167696,[UPCYCLE]햇반 흑미밥 210gx24개(1box),1,10.125110,F,3,2023-01-26,0,1,10109,...,0,0,0,2,0,10.125110,1,10.125110,1,0
4,20230125159705,햇반 매일찰잡곡밥 210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,0,0,0,2,0,0.499733,8,4.151479,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26065,20230101966635,고메 순살크리스피 치킨 고추크런치 450g,1,8.659560,M,3,2023-01-01,0,0,10109,...,1,1,0,0,0,8.659560,5,7.706661,0,0
26066,20230102974273,비비고 사골순댓국 460g,10,10.518646,M,4,2023-01-02,1,0,20945,...,0,0,0,0,0,1.051865,6,2.517459,0,0
26067,20230102972720,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,3,2023-01-02,0,0,10109,...,0,0,0,0,0,9.639327,2,9.478408,0,0
26068,20230101964953,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,4,2023-01-01,0,0,20945,...,1,1,0,0,0,9.639327,1,9.639327,0,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,order_weekend,order_holiday,holiday_prev7d,week_for_10,product_newyear,amt_per_qty,buy_product_cnt,amt_per_qty_mean,1box,chicken breast
0,20230101970142,삼호 생선살어묵 야채 200g,1,8.098947,M,4,2023-01-01,NaN,0,4344,...,1,1,0,0,0,8.098947,6,8.788789,0,0
1,20230101964512,[앱전용특가] 쁘띠첼밀감90gX6개X3개,1,9.076809,M,4,2023-01-01,NaN,0,4344,...,1,1,0,0,0,9.076809,4,5.861183,0,0
2,20230102975046,빕스 볶음밥 1인분 X 6종_행사,1,9.928522,F,3,2023-01-02,NaN,1,6299,...,0,0,0,0,0,9.928522,1,9.928522,0,0
3,20230102973138,백설 진한참기름 500ml,2,10.060534,M,4,2023-01-02,NaN,0,4344,...,0,0,0,0,0,5.030267,1,5.030267,0,0
4,20230101969192,삼호 가쓰오모둠어묵탕413g,2,9.796181,M,4,2023-01-01,NaN,0,4344,...,1,1,0,0,0,4.898090,8,7.910389,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8041,20230103994630,고메 콰트로포르마지 피자 310g,1,8.736650,M,5,2023-01-03,NaN,0,2051,...,0,0,0,0,0,8.736650,3,6.759689,0,0
8042,20230103994130,햇반솥반 전복내장영양밥 200g,2,9.196748,M,3,2023-01-03,NaN,1,6299,...,0,0,0,0,0,4.598374,3,6.263244,0,0
8043,20230103986821,비비고 찐만두168g,1,7.930566,M,5,2023-01-03,NaN,0,2051,...,0,0,0,0,0,7.930566,6,6.987001,0,0
8044,20230102979023,사골곰탕 한 그릇 100g (1-2인분X5개입),1,8.477412,F,3,2023-01-02,NaN,0,6299,...,0,0,0,0,0,8.477412,2,8.162001,0,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,order_weekend,order_holiday,holiday_prev7d,week_for_10,product_newyear,amt_per_qty,buy_product_cnt,amt_per_qty_mean,1box,chicken breast
0,20230101964282,비비고 스팸부대찌개 460g,1,8.161946,M,4,2023-01-01,NaN,0,9748,...,1,1,0,0,0,8.161946,4,8.164507,0,0
1,20230101965237,크레잇 블랙페퍼 폭찹 스테이크,1,8.277412,F,4,2023-01-01,NaN,0,9748,...,1,1,0,0,0,8.277412,5,5.942704,0,0
2,20230102973798,비비고 소고기 미역국 500g,3,9.145375,F,3,2023-01-02,NaN,0,5034,...,0,0,0,0,0,3.048458,2,5.668622,0,0
3,20230101965633,비비고 소고기 미역국 500g,2,8.613230,F,3,2023-01-01,NaN,0,5034,...,1,1,0,0,0,4.306615,3,4.548013,0,0
4,20230101963753,고메 치즈크리스피핫도그 340g,1,8.540714,F,4,2023-01-01,NaN,0,9748,...,1,1,0,0,0,8.540714,2,5.876901,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11609,20230102980326,고메 케이준프라이즈 피자 145g,2,8.671115,F,4,2023-01-02,NaN,0,9748,...,0,0,0,0,0,4.335558,5,6.486198,0,0
11610,20230102983068,고메 빅크리스피핫도그520g,1,8.641179,F,3,2023-01-02,NaN,0,5034,...,0,0,0,0,0,8.641179,2,8.535317,0,0
11611,20230103992835,비비고 메추리알장조림 170g,7,9.467692,F,4,2023-01-03,NaN,0,9748,...,0,0,0,0,0,1.352527,3,4.743072,0,0
11612,20230104999357,비비고 언양식 바싹불고기 460g,1,8.594895,M,4,2023-01-04,NaN,0,9748,...,0,0,0,0,0,8.594895,5,7.583406,0,0


#### 브랜드 표기 상품 구매여부

In [36]:
brand_list = ['비비고', '햇반', '스팸', '고메', '삼호', '백설', '쿡킷', '더건강한', '다시다', '해찬들',
              'BYO', '한뿌리', '하선정', '행복한콩', '다담', '크레잇', '메티에', '쁘띠첼', '맥스봉', '맛밤']

In [37]:
for brand in brand_list:
    # train
    emp_tr[brand] = emp_tr.product_name.apply(lambda x : int(brand in x))
    nemp_tr[brand] = nemp_tr.product_name.apply(lambda x : int(brand in x))
    
    # test
    emp_te[brand] = emp_te.product_name.apply(lambda x : int(brand in x))
    nemp_te[brand] = nemp_te.product_name.apply(lambda x : int(brand in x))
    
# train
emp_tr['brand'] = emp_tr[brand_list].sum(axis = 1)
nemp_tr['brand'] = nemp_tr[brand_list].sum(axis = 1)
emp_tr = emp_tr.drop(columns = brand_list[6:], axis = 0)
nemp_tr = nemp_tr.drop(columns = brand_list[6:], axis = 0)

# test
emp_te['brand'] = emp_te[brand_list].sum(axis = 1)
nemp_te['brand'] = nemp_te[brand_list].sum(axis = 1)
emp_te = emp_te.drop(columns = brand_list[6:], axis = 0)
nemp_te = nemp_te.drop(columns = brand_list[6:], axis = 0)

display(emp_tr, nemp_tr, emp_te, nemp_te)

,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,amt_per_qty_mean,1box,chicken breast,비비고,햇반,스팸,고메,삼호,백설,brand
0,20230124153976,잔칫집 식혜 240ml 30입,1,9.803170,F,2,2023-01-24,0,0,5661,...,9.280165,0,0,0,0,0,0,0,0,0
1,20230124155563,백설 한입쏙 비엔나 120g*2,1,8.256607,M,3,2023-01-24,0,0,14016,...,8.675043,0,0,0,0,0,0,0,1,1
2,20230125160921,맥스봉 체다치즈어랏400g (20g*20EA),1,8.974998,F,2,2023-01-25,0,0,5661,...,3.940153,0,0,0,0,0,0,0,0,1
3,20230125157569,고메 바삭쫄깃한 사천탕수육 450g,1,8.938007,M,4,2023-01-25,1,0,9878,...,8.174998,0,0,0,0,0,1,0,0,1
4,20230125156655,고메 콘크림수프 180g*4입,1,8.997518,F,3,2023-01-25,0,0,14016,...,9.019387,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19800,20230102975115,[2023설사원선물신청] 흑삼정 골드클래스 100gx2,1,11.597276,M,5,2023-01-02,0,0,5196,...,11.597276,0,0,0,0,0,0,0,0,0
19801,20230102972321,고메 바삭쫄깃한 탕수육 450g,1,8.702510,M,5,2023-01-02,1,0,5196,...,7.678905,0,0,0,0,0,1,0,0,1
19802,20230101969615,리턴업 전립소 쏘팔메토 골드 1000mgX60캡슐(2개월)X2개,1,11.364344,F,2,2023-01-01,1,0,5661,...,11.364344,0,0,0,0,0,0,0,0,0
19803,20230102972321,고메 거멍 모짜체다핫도그 340g,1,8.648397,M,5,2023-01-02,1,0,5196,...,7.678905,0,0,0,0,0,1,0,0,1


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,amt_per_qty_mean,1box,chicken breast,비비고,햇반,스팸,고메,삼호,백설,brand
0,20230125158386,비비고 왕교자 1.05kg,1,9.348449,F,4,2023-01-25,0,0,20945,...,9.348449,0,0,1,0,0,0,0,0,1
1,20230126164638,고메 바삭쫄깃한 탕수육 900g,1,9.667259,F,4,2023-01-26,1,0,20945,...,7.778245,0,0,0,0,0,1,0,0,1
2,20230125159705,햇반 매일잡곡밥210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,4.151479,0,0,0,1,0,0,0,0,1
3,20230126167696,[UPCYCLE]햇반 흑미밥 210gx24개(1box),1,10.125110,F,3,2023-01-26,0,1,10109,...,10.125110,1,0,0,1,0,0,0,0,1
4,20230125159705,햇반 매일찰잡곡밥 210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,4.151479,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26065,20230101966635,고메 순살크리스피 치킨 고추크런치 450g,1,8.659560,M,3,2023-01-01,0,0,10109,...,7.706661,0,0,0,0,0,1,0,0,1
26066,20230102974273,비비고 사골순댓국 460g,10,10.518646,M,4,2023-01-02,1,0,20945,...,2.517459,0,0,1,0,0,0,0,0,1
26067,20230102972720,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,3,2023-01-02,0,0,10109,...,9.478408,0,0,1,0,0,0,0,0,1
26068,20230101964953,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,4,2023-01-01,0,0,20945,...,9.639327,0,0,1,0,0,0,0,0,1


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,amt_per_qty_mean,1box,chicken breast,비비고,햇반,스팸,고메,삼호,백설,brand
0,20230101970142,삼호 생선살어묵 야채 200g,1,8.098947,M,4,2023-01-01,NaN,0,4344,...,8.788789,0,0,0,0,0,0,1,0,1
1,20230101964512,[앱전용특가] 쁘띠첼밀감90gX6개X3개,1,9.076809,M,4,2023-01-01,NaN,0,4344,...,5.861183,0,0,0,0,0,0,0,0,1
2,20230102975046,빕스 볶음밥 1인분 X 6종_행사,1,9.928522,F,3,2023-01-02,NaN,1,6299,...,9.928522,0,0,0,0,0,0,0,0,0
3,20230102973138,백설 진한참기름 500ml,2,10.060534,M,4,2023-01-02,NaN,0,4344,...,5.030267,0,0,0,0,0,0,0,1,1
4,20230101969192,삼호 가쓰오모둠어묵탕413g,2,9.796181,M,4,2023-01-01,NaN,0,4344,...,7.910389,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8041,20230103994630,고메 콰트로포르마지 피자 310g,1,8.736650,M,5,2023-01-03,NaN,0,2051,...,6.759689,0,0,0,0,0,1,0,0,1
8042,20230103994130,햇반솥반 전복내장영양밥 200g,2,9.196748,M,3,2023-01-03,NaN,1,6299,...,6.263244,0,0,0,1,0,0,0,0,1
8043,20230103986821,비비고 찐만두168g,1,7.930566,M,5,2023-01-03,NaN,0,2051,...,6.987001,0,0,1,0,0,0,0,0,1
8044,20230102979023,사골곰탕 한 그릇 100g (1-2인분X5개입),1,8.477412,F,3,2023-01-02,NaN,0,6299,...,8.162001,0,0,0,0,0,0,0,0,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,amt_per_qty_mean,1box,chicken breast,비비고,햇반,스팸,고메,삼호,백설,brand
0,20230101964282,비비고 스팸부대찌개 460g,1,8.161946,M,4,2023-01-01,NaN,0,9748,...,8.164507,0,0,1,0,1,0,0,0,2
1,20230101965237,크레잇 블랙페퍼 폭찹 스테이크,1,8.277412,F,4,2023-01-01,NaN,0,9748,...,5.942704,0,0,0,0,0,0,0,0,1
2,20230102973798,비비고 소고기 미역국 500g,3,9.145375,F,3,2023-01-02,NaN,0,5034,...,5.668622,0,0,1,0,0,0,0,0,1
3,20230101965633,비비고 소고기 미역국 500g,2,8.613230,F,3,2023-01-01,NaN,0,5034,...,4.548013,0,0,1,0,0,0,0,0,1
4,20230101963753,고메 치즈크리스피핫도그 340g,1,8.540714,F,4,2023-01-01,NaN,0,9748,...,5.876901,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11609,20230102980326,고메 케이준프라이즈 피자 145g,2,8.671115,F,4,2023-01-02,NaN,0,9748,...,6.486198,0,0,0,0,0,1,0,0,1
11610,20230102983068,고메 빅크리스피핫도그520g,1,8.641179,F,3,2023-01-02,NaN,0,5034,...,8.535317,0,0,0,0,0,1,0,0,1
11611,20230103992835,비비고 메추리알장조림 170g,7,9.467692,F,4,2023-01-03,NaN,0,9748,...,4.743072,0,0,1,0,0,0,0,0,1
11612,20230104999357,비비고 언양식 바싹불고기 460g,1,8.594895,M,4,2023-01-04,NaN,0,9748,...,7.583406,0,0,1,0,0,0,0,0,1


#### 물 상품 구매여부

In [38]:
water_list = ['아이시스', '삼다수', '에비앙']

In [39]:
for water in water_list:
    # train
    emp_tr[water] = emp_tr.product_name.apply(lambda x : int(water in x))
    nemp_tr[water] = nemp_tr.product_name.apply(lambda x : int(water in x))
    
    # test
    emp_te[water] = emp_te.product_name.apply(lambda x : int(water in x))
    nemp_te[water] = nemp_te.product_name.apply(lambda x : int(water in x))
    
# train
emp_tr['water'] = emp_tr[water_list].sum(axis = 1)
nemp_tr['water'] = nemp_tr[water_list].sum(axis = 1)
emp_tr = emp_tr.drop(columns = water_list, axis = 0)
nemp_tr = nemp_tr.drop(columns = water_list, axis = 0)

# test
emp_te['water'] = emp_te[water_list].sum(axis = 1)
nemp_te['water'] = nemp_te[water_list].sum(axis = 1)
emp_te = emp_te.drop(columns = water_list, axis = 0)
nemp_te = nemp_te.drop(columns = water_list, axis = 0)

display(emp_tr, nemp_tr, emp_te, nemp_te)

,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,1box,chicken breast,비비고,햇반,스팸,고메,삼호,백설,brand,water
0,20230124153976,잔칫집 식혜 240ml 30입,1,9.803170,F,2,2023-01-24,0,0,5661,...,0,0,0,0,0,0,0,0,0,0
1,20230124155563,백설 한입쏙 비엔나 120g*2,1,8.256607,M,3,2023-01-24,0,0,14016,...,0,0,0,0,0,0,0,1,1,0
2,20230125160921,맥스봉 체다치즈어랏400g (20g*20EA),1,8.974998,F,2,2023-01-25,0,0,5661,...,0,0,0,0,0,0,0,0,1,0
3,20230125157569,고메 바삭쫄깃한 사천탕수육 450g,1,8.938007,M,4,2023-01-25,1,0,9878,...,0,0,0,0,0,1,0,0,1,0
4,20230125156655,고메 콘크림수프 180g*4입,1,8.997518,F,3,2023-01-25,0,0,14016,...,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19800,20230102975115,[2023설사원선물신청] 흑삼정 골드클래스 100gx2,1,11.597276,M,5,2023-01-02,0,0,5196,...,0,0,0,0,0,0,0,0,0,0
19801,20230102972321,고메 바삭쫄깃한 탕수육 450g,1,8.702510,M,5,2023-01-02,1,0,5196,...,0,0,0,0,0,1,0,0,1,0
19802,20230101969615,리턴업 전립소 쏘팔메토 골드 1000mgX60캡슐(2개월)X2개,1,11.364344,F,2,2023-01-01,1,0,5661,...,0,0,0,0,0,0,0,0,0,0
19803,20230102972321,고메 거멍 모짜체다핫도그 340g,1,8.648397,M,5,2023-01-02,1,0,5196,...,0,0,0,0,0,1,0,0,1,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,1box,chicken breast,비비고,햇반,스팸,고메,삼호,백설,brand,water
0,20230125158386,비비고 왕교자 1.05kg,1,9.348449,F,4,2023-01-25,0,0,20945,...,0,0,1,0,0,0,0,0,1,0
1,20230126164638,고메 바삭쫄깃한 탕수육 900g,1,9.667259,F,4,2023-01-26,1,0,20945,...,0,0,0,0,0,1,0,0,1,0
2,20230125159705,햇반 매일잡곡밥210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,0,0,0,1,0,0,0,0,1,0
3,20230126167696,[UPCYCLE]햇반 흑미밥 210gx24개(1box),1,10.125110,F,3,2023-01-26,0,1,10109,...,1,0,0,1,0,0,0,0,1,0
4,20230125159705,햇반 매일찰잡곡밥 210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26065,20230101966635,고메 순살크리스피 치킨 고추크런치 450g,1,8.659560,M,3,2023-01-01,0,0,10109,...,0,0,0,0,0,1,0,0,1,0
26066,20230102974273,비비고 사골순댓국 460g,10,10.518646,M,4,2023-01-02,1,0,20945,...,0,0,1,0,0,0,0,0,1,0
26067,20230102972720,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,3,2023-01-02,0,0,10109,...,0,0,1,0,0,0,0,0,1,0
26068,20230101964953,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,4,2023-01-01,0,0,20945,...,0,0,1,0,0,0,0,0,1,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,1box,chicken breast,비비고,햇반,스팸,고메,삼호,백설,brand,water
0,20230101970142,삼호 생선살어묵 야채 200g,1,8.098947,M,4,2023-01-01,NaN,0,4344,...,0,0,0,0,0,0,1,0,1,0
1,20230101964512,[앱전용특가] 쁘띠첼밀감90gX6개X3개,1,9.076809,M,4,2023-01-01,NaN,0,4344,...,0,0,0,0,0,0,0,0,1,0
2,20230102975046,빕스 볶음밥 1인분 X 6종_행사,1,9.928522,F,3,2023-01-02,NaN,1,6299,...,0,0,0,0,0,0,0,0,0,0
3,20230102973138,백설 진한참기름 500ml,2,10.060534,M,4,2023-01-02,NaN,0,4344,...,0,0,0,0,0,0,0,1,1,0
4,20230101969192,삼호 가쓰오모둠어묵탕413g,2,9.796181,M,4,2023-01-01,NaN,0,4344,...,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8041,20230103994630,고메 콰트로포르마지 피자 310g,1,8.736650,M,5,2023-01-03,NaN,0,2051,...,0,0,0,0,0,1,0,0,1,0
8042,20230103994130,햇반솥반 전복내장영양밥 200g,2,9.196748,M,3,2023-01-03,NaN,1,6299,...,0,0,0,1,0,0,0,0,1,0
8043,20230103986821,비비고 찐만두168g,1,7.930566,M,5,2023-01-03,NaN,0,2051,...,0,0,1,0,0,0,0,0,1,0
8044,20230102979023,사골곰탕 한 그릇 100g (1-2인분X5개입),1,8.477412,F,3,2023-01-02,NaN,0,6299,...,0,0,0,0,0,0,0,0,0,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,1box,chicken breast,비비고,햇반,스팸,고메,삼호,백설,brand,water
0,20230101964282,비비고 스팸부대찌개 460g,1,8.161946,M,4,2023-01-01,NaN,0,9748,...,0,0,1,0,1,0,0,0,2,0
1,20230101965237,크레잇 블랙페퍼 폭찹 스테이크,1,8.277412,F,4,2023-01-01,NaN,0,9748,...,0,0,0,0,0,0,0,0,1,0
2,20230102973798,비비고 소고기 미역국 500g,3,9.145375,F,3,2023-01-02,NaN,0,5034,...,0,0,1,0,0,0,0,0,1,0
3,20230101965633,비비고 소고기 미역국 500g,2,8.613230,F,3,2023-01-01,NaN,0,5034,...,0,0,1,0,0,0,0,0,1,0
4,20230101963753,고메 치즈크리스피핫도그 340g,1,8.540714,F,4,2023-01-01,NaN,0,9748,...,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11609,20230102980326,고메 케이준프라이즈 피자 145g,2,8.671115,F,4,2023-01-02,NaN,0,9748,...,0,0,0,0,0,1,0,0,1,0
11610,20230102983068,고메 빅크리스피핫도그520g,1,8.641179,F,3,2023-01-02,NaN,0,5034,...,0,0,0,0,0,1,0,0,1,0
11611,20230103992835,비비고 메추리알장조림 170g,7,9.467692,F,4,2023-01-03,NaN,0,9748,...,0,0,1,0,0,0,0,0,1,0
11612,20230104999357,비비고 언양식 바싹불고기 460g,1,8.594895,M,4,2023-01-04,NaN,0,9748,...,0,0,1,0,0,0,0,0,1,0


#### 식용 기름 상품 구매여부

In [40]:
oil_list = ['식용유', '포도씨유', '올리브유', '요리유', '해바라기씨유', '콩기름', '옥수수유']

In [41]:
for oil in oil_list:
    # train
    emp_tr[oil] = emp_tr.product_name.apply(lambda x : int(oil in x))
    nemp_tr[oil] = nemp_tr.product_name.apply(lambda x : int(oil in x))
    
    # test
    emp_te[oil] = emp_te.product_name.apply(lambda x : int(oil in x))
    nemp_te[oil] = nemp_te.product_name.apply(lambda x : int(oil in x))

# train
emp_tr['oil'] = emp_tr[oil_list].sum(axis = 1)
nemp_tr['oil'] = nemp_tr[oil_list].sum(axis = 1)
emp_tr = emp_tr.drop(columns = oil_list, axis = 0)
nemp_tr = nemp_tr.drop(columns = oil_list, axis = 0)

#test
emp_te['oil'] = emp_te[oil_list].sum(axis = 1)
nemp_te['oil'] = nemp_te[oil_list].sum(axis = 1)
emp_te = emp_te.drop(columns = oil_list, axis = 0)
nemp_te = nemp_te.drop(columns = oil_list, axis = 0)

display(emp_tr, nemp_tr, emp_te, nemp_te)

,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,chicken breast,비비고,햇반,스팸,고메,삼호,백설,brand,water,oil
0,20230124153976,잔칫집 식혜 240ml 30입,1,9.803170,F,2,2023-01-24,0,0,5661,...,0,0,0,0,0,0,0,0,0,0
1,20230124155563,백설 한입쏙 비엔나 120g*2,1,8.256607,M,3,2023-01-24,0,0,14016,...,0,0,0,0,0,0,1,1,0,0
2,20230125160921,맥스봉 체다치즈어랏400g (20g*20EA),1,8.974998,F,2,2023-01-25,0,0,5661,...,0,0,0,0,0,0,0,1,0,0
3,20230125157569,고메 바삭쫄깃한 사천탕수육 450g,1,8.938007,M,4,2023-01-25,1,0,9878,...,0,0,0,0,1,0,0,1,0,0
4,20230125156655,고메 콘크림수프 180g*4입,1,8.997518,F,3,2023-01-25,0,0,14016,...,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19800,20230102975115,[2023설사원선물신청] 흑삼정 골드클래스 100gx2,1,11.597276,M,5,2023-01-02,0,0,5196,...,0,0,0,0,0,0,0,0,0,0
19801,20230102972321,고메 바삭쫄깃한 탕수육 450g,1,8.702510,M,5,2023-01-02,1,0,5196,...,0,0,0,0,1,0,0,1,0,0
19802,20230101969615,리턴업 전립소 쏘팔메토 골드 1000mgX60캡슐(2개월)X2개,1,11.364344,F,2,2023-01-01,1,0,5661,...,0,0,0,0,0,0,0,0,0,0
19803,20230102972321,고메 거멍 모짜체다핫도그 340g,1,8.648397,M,5,2023-01-02,1,0,5196,...,0,0,0,0,1,0,0,1,0,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,chicken breast,비비고,햇반,스팸,고메,삼호,백설,brand,water,oil
0,20230125158386,비비고 왕교자 1.05kg,1,9.348449,F,4,2023-01-25,0,0,20945,...,0,1,0,0,0,0,0,1,0,0
1,20230126164638,고메 바삭쫄깃한 탕수육 900g,1,9.667259,F,4,2023-01-26,1,0,20945,...,0,0,0,0,1,0,0,1,0,0
2,20230125159705,햇반 매일잡곡밥210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,0,0,1,0,0,0,0,1,0,0
3,20230126167696,[UPCYCLE]햇반 흑미밥 210gx24개(1box),1,10.125110,F,3,2023-01-26,0,1,10109,...,0,0,1,0,0,0,0,1,0,0
4,20230125159705,햇반 매일찰잡곡밥 210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26065,20230101966635,고메 순살크리스피 치킨 고추크런치 450g,1,8.659560,M,3,2023-01-01,0,0,10109,...,0,0,0,0,1,0,0,1,0,0
26066,20230102974273,비비고 사골순댓국 460g,10,10.518646,M,4,2023-01-02,1,0,20945,...,0,1,0,0,0,0,0,1,0,0
26067,20230102972720,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,3,2023-01-02,0,0,10109,...,0,1,0,0,0,0,0,1,0,0
26068,20230101964953,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,4,2023-01-01,0,0,20945,...,0,1,0,0,0,0,0,1,0,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,chicken breast,비비고,햇반,스팸,고메,삼호,백설,brand,water,oil
0,20230101970142,삼호 생선살어묵 야채 200g,1,8.098947,M,4,2023-01-01,NaN,0,4344,...,0,0,0,0,0,1,0,1,0,0
1,20230101964512,[앱전용특가] 쁘띠첼밀감90gX6개X3개,1,9.076809,M,4,2023-01-01,NaN,0,4344,...,0,0,0,0,0,0,0,1,0,0
2,20230102975046,빕스 볶음밥 1인분 X 6종_행사,1,9.928522,F,3,2023-01-02,NaN,1,6299,...,0,0,0,0,0,0,0,0,0,0
3,20230102973138,백설 진한참기름 500ml,2,10.060534,M,4,2023-01-02,NaN,0,4344,...,0,0,0,0,0,0,1,1,0,0
4,20230101969192,삼호 가쓰오모둠어묵탕413g,2,9.796181,M,4,2023-01-01,NaN,0,4344,...,0,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8041,20230103994630,고메 콰트로포르마지 피자 310g,1,8.736650,M,5,2023-01-03,NaN,0,2051,...,0,0,0,0,1,0,0,1,0,0
8042,20230103994130,햇반솥반 전복내장영양밥 200g,2,9.196748,M,3,2023-01-03,NaN,1,6299,...,0,0,1,0,0,0,0,1,0,0
8043,20230103986821,비비고 찐만두168g,1,7.930566,M,5,2023-01-03,NaN,0,2051,...,0,1,0,0,0,0,0,1,0,0
8044,20230102979023,사골곰탕 한 그릇 100g (1-2인분X5개입),1,8.477412,F,3,2023-01-02,NaN,0,6299,...,0,0,0,0,0,0,0,0,0,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,chicken breast,비비고,햇반,스팸,고메,삼호,백설,brand,water,oil
0,20230101964282,비비고 스팸부대찌개 460g,1,8.161946,M,4,2023-01-01,NaN,0,9748,...,0,1,0,1,0,0,0,2,0,0
1,20230101965237,크레잇 블랙페퍼 폭찹 스테이크,1,8.277412,F,4,2023-01-01,NaN,0,9748,...,0,0,0,0,0,0,0,1,0,0
2,20230102973798,비비고 소고기 미역국 500g,3,9.145375,F,3,2023-01-02,NaN,0,5034,...,0,1,0,0,0,0,0,1,0,0
3,20230101965633,비비고 소고기 미역국 500g,2,8.613230,F,3,2023-01-01,NaN,0,5034,...,0,1,0,0,0,0,0,1,0,0
4,20230101963753,고메 치즈크리스피핫도그 340g,1,8.540714,F,4,2023-01-01,NaN,0,9748,...,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11609,20230102980326,고메 케이준프라이즈 피자 145g,2,8.671115,F,4,2023-01-02,NaN,0,9748,...,0,0,0,0,1,0,0,1,0,0
11610,20230102983068,고메 빅크리스피핫도그520g,1,8.641179,F,3,2023-01-02,NaN,0,5034,...,0,0,0,0,1,0,0,1,0,0
11611,20230103992835,비비고 메추리알장조림 170g,7,9.467692,F,4,2023-01-03,NaN,0,9748,...,0,1,0,0,0,0,0,1,0,0
11612,20230104999357,비비고 언양식 바싹불고기 460g,1,8.594895,M,4,2023-01-04,NaN,0,9748,...,0,1,0,0,0,0,0,1,0,0


#### 만두 상품 구매여부

In [42]:
mandu_list = ['만두', '교자']

In [43]:
for mandu in mandu_list:
    # train
    emp_tr[mandu] = emp_tr.product_name.apply(lambda x : int(mandu in x))
    nemp_tr[mandu] = nemp_tr.product_name.apply(lambda x : int(mandu in x))
    
    # test
    emp_te[mandu] = emp_te.product_name.apply(lambda x : int(mandu in x))
    nemp_te[mandu] = nemp_te.product_name.apply(lambda x : int(mandu in x))

# train
emp_tr['mandu'] = emp_tr[mandu_list].sum(axis = 1)
nemp_tr['mandu'] = nemp_tr[mandu_list].sum(axis = 1)
emp_tr = emp_tr.drop(columns = mandu_list, axis = 0)
nemp_tr = nemp_tr.drop(columns = mandu_list, axis = 0)

#test
emp_te['mandu'] = emp_te[mandu_list].sum(axis = 1)
nemp_te['mandu'] = nemp_te[mandu_list].sum(axis = 1)
emp_te = emp_te.drop(columns = mandu_list, axis = 0)
nemp_te = nemp_te.drop(columns = mandu_list, axis = 0)

display(emp_tr, nemp_tr, emp_te, nemp_te)

,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,비비고,햇반,스팸,고메,삼호,백설,brand,water,oil,mandu
0,20230124153976,잔칫집 식혜 240ml 30입,1,9.803170,F,2,2023-01-24,0,0,5661,...,0,0,0,0,0,0,0,0,0,0
1,20230124155563,백설 한입쏙 비엔나 120g*2,1,8.256607,M,3,2023-01-24,0,0,14016,...,0,0,0,0,0,1,1,0,0,0
2,20230125160921,맥스봉 체다치즈어랏400g (20g*20EA),1,8.974998,F,2,2023-01-25,0,0,5661,...,0,0,0,0,0,0,1,0,0,0
3,20230125157569,고메 바삭쫄깃한 사천탕수육 450g,1,8.938007,M,4,2023-01-25,1,0,9878,...,0,0,0,1,0,0,1,0,0,0
4,20230125156655,고메 콘크림수프 180g*4입,1,8.997518,F,3,2023-01-25,0,0,14016,...,0,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19800,20230102975115,[2023설사원선물신청] 흑삼정 골드클래스 100gx2,1,11.597276,M,5,2023-01-02,0,0,5196,...,0,0,0,0,0,0,0,0,0,0
19801,20230102972321,고메 바삭쫄깃한 탕수육 450g,1,8.702510,M,5,2023-01-02,1,0,5196,...,0,0,0,1,0,0,1,0,0,0
19802,20230101969615,리턴업 전립소 쏘팔메토 골드 1000mgX60캡슐(2개월)X2개,1,11.364344,F,2,2023-01-01,1,0,5661,...,0,0,0,0,0,0,0,0,0,0
19803,20230102972321,고메 거멍 모짜체다핫도그 340g,1,8.648397,M,5,2023-01-02,1,0,5196,...,0,0,0,1,0,0,1,0,0,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,비비고,햇반,스팸,고메,삼호,백설,brand,water,oil,mandu
0,20230125158386,비비고 왕교자 1.05kg,1,9.348449,F,4,2023-01-25,0,0,20945,...,1,0,0,0,0,0,1,0,0,1
1,20230126164638,고메 바삭쫄깃한 탕수육 900g,1,9.667259,F,4,2023-01-26,1,0,20945,...,0,0,0,1,0,0,1,0,0,0
2,20230125159705,햇반 매일잡곡밥210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,0,1,0,0,0,0,1,0,0,0
3,20230126167696,[UPCYCLE]햇반 흑미밥 210gx24개(1box),1,10.125110,F,3,2023-01-26,0,1,10109,...,0,1,0,0,0,0,1,0,0,0
4,20230125159705,햇반 매일찰잡곡밥 210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,0,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26065,20230101966635,고메 순살크리스피 치킨 고추크런치 450g,1,8.659560,M,3,2023-01-01,0,0,10109,...,0,0,0,1,0,0,1,0,0,0
26066,20230102974273,비비고 사골순댓국 460g,10,10.518646,M,4,2023-01-02,1,0,20945,...,1,0,0,0,0,0,1,0,0,0
26067,20230102972720,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,3,2023-01-02,0,0,10109,...,1,0,0,0,0,0,1,0,0,0
26068,20230101964953,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,4,2023-01-01,0,0,20945,...,1,0,0,0,0,0,1,0,0,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,비비고,햇반,스팸,고메,삼호,백설,brand,water,oil,mandu
0,20230101970142,삼호 생선살어묵 야채 200g,1,8.098947,M,4,2023-01-01,NaN,0,4344,...,0,0,0,0,1,0,1,0,0,0
1,20230101964512,[앱전용특가] 쁘띠첼밀감90gX6개X3개,1,9.076809,M,4,2023-01-01,NaN,0,4344,...,0,0,0,0,0,0,1,0,0,0
2,20230102975046,빕스 볶음밥 1인분 X 6종_행사,1,9.928522,F,3,2023-01-02,NaN,1,6299,...,0,0,0,0,0,0,0,0,0,0
3,20230102973138,백설 진한참기름 500ml,2,10.060534,M,4,2023-01-02,NaN,0,4344,...,0,0,0,0,0,1,1,0,0,0
4,20230101969192,삼호 가쓰오모둠어묵탕413g,2,9.796181,M,4,2023-01-01,NaN,0,4344,...,0,0,0,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8041,20230103994630,고메 콰트로포르마지 피자 310g,1,8.736650,M,5,2023-01-03,NaN,0,2051,...,0,0,0,1,0,0,1,0,0,0
8042,20230103994130,햇반솥반 전복내장영양밥 200g,2,9.196748,M,3,2023-01-03,NaN,1,6299,...,0,1,0,0,0,0,1,0,0,0
8043,20230103986821,비비고 찐만두168g,1,7.930566,M,5,2023-01-03,NaN,0,2051,...,1,0,0,0,0,0,1,0,0,1
8044,20230102979023,사골곰탕 한 그릇 100g (1-2인분X5개입),1,8.477412,F,3,2023-01-02,NaN,0,6299,...,0,0,0,0,0,0,0,0,0,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,비비고,햇반,스팸,고메,삼호,백설,brand,water,oil,mandu
0,20230101964282,비비고 스팸부대찌개 460g,1,8.161946,M,4,2023-01-01,NaN,0,9748,...,1,0,1,0,0,0,2,0,0,0
1,20230101965237,크레잇 블랙페퍼 폭찹 스테이크,1,8.277412,F,4,2023-01-01,NaN,0,9748,...,0,0,0,0,0,0,1,0,0,0
2,20230102973798,비비고 소고기 미역국 500g,3,9.145375,F,3,2023-01-02,NaN,0,5034,...,1,0,0,0,0,0,1,0,0,0
3,20230101965633,비비고 소고기 미역국 500g,2,8.613230,F,3,2023-01-01,NaN,0,5034,...,1,0,0,0,0,0,1,0,0,0
4,20230101963753,고메 치즈크리스피핫도그 340g,1,8.540714,F,4,2023-01-01,NaN,0,9748,...,0,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11609,20230102980326,고메 케이준프라이즈 피자 145g,2,8.671115,F,4,2023-01-02,NaN,0,9748,...,0,0,0,1,0,0,1,0,0,0
11610,20230102983068,고메 빅크리스피핫도그520g,1,8.641179,F,3,2023-01-02,NaN,0,5034,...,0,0,0,1,0,0,1,0,0,0
11611,20230103992835,비비고 메추리알장조림 170g,7,9.467692,F,4,2023-01-03,NaN,0,9748,...,1,0,0,0,0,0,1,0,0,0
11612,20230104999357,비비고 언양식 바싹불고기 460g,1,8.594895,M,4,2023-01-04,NaN,0,9748,...,1,0,0,0,0,0,1,0,0,0


#### 핫도그 상품 구매여부

In [44]:
hotdog_list = ['핫도그', '핫바', '맥스봉']

In [45]:
for hotdog in hotdog_list:
    # train
    emp_tr[hotdog] = emp_tr.product_name.apply(lambda x : int(hotdog in x))
    nemp_tr[hotdog] = nemp_tr.product_name.apply(lambda x : int(hotdog in x))
    
    # test
    emp_te[hotdog] = emp_te.product_name.apply(lambda x : int(hotdog in x))
    nemp_te[hotdog] = nemp_te.product_name.apply(lambda x : int(hotdog in x))

# train
emp_tr['hotdog'] = emp_tr[hotdog_list].sum(axis = 1)
nemp_tr['hotdog'] = nemp_tr[hotdog_list].sum(axis = 1)
emp_tr = emp_tr.drop(columns = hotdog_list, axis = 0)
nemp_tr = nemp_tr.drop(columns = hotdog_list, axis = 0)

#test
emp_te['hotdog'] = emp_te[hotdog_list].sum(axis = 1)
nemp_te['hotdog'] = nemp_te[hotdog_list].sum(axis = 1)
emp_te = emp_te.drop(columns = hotdog_list, axis = 0)
nemp_te = nemp_te.drop(columns = hotdog_list, axis = 0)

display(emp_tr, nemp_tr, emp_te, nemp_te)

,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,햇반,스팸,고메,삼호,백설,brand,water,oil,mandu,hotdog
0,20230124153976,잔칫집 식혜 240ml 30입,1,9.803170,F,2,2023-01-24,0,0,5661,...,0,0,0,0,0,0,0,0,0,0
1,20230124155563,백설 한입쏙 비엔나 120g*2,1,8.256607,M,3,2023-01-24,0,0,14016,...,0,0,0,0,1,1,0,0,0,0
2,20230125160921,맥스봉 체다치즈어랏400g (20g*20EA),1,8.974998,F,2,2023-01-25,0,0,5661,...,0,0,0,0,0,1,0,0,0,1
3,20230125157569,고메 바삭쫄깃한 사천탕수육 450g,1,8.938007,M,4,2023-01-25,1,0,9878,...,0,0,1,0,0,1,0,0,0,0
4,20230125156655,고메 콘크림수프 180g*4입,1,8.997518,F,3,2023-01-25,0,0,14016,...,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19800,20230102975115,[2023설사원선물신청] 흑삼정 골드클래스 100gx2,1,11.597276,M,5,2023-01-02,0,0,5196,...,0,0,0,0,0,0,0,0,0,0
19801,20230102972321,고메 바삭쫄깃한 탕수육 450g,1,8.702510,M,5,2023-01-02,1,0,5196,...,0,0,1,0,0,1,0,0,0,0
19802,20230101969615,리턴업 전립소 쏘팔메토 골드 1000mgX60캡슐(2개월)X2개,1,11.364344,F,2,2023-01-01,1,0,5661,...,0,0,0,0,0,0,0,0,0,0
19803,20230102972321,고메 거멍 모짜체다핫도그 340g,1,8.648397,M,5,2023-01-02,1,0,5196,...,0,0,1,0,0,1,0,0,0,1


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,햇반,스팸,고메,삼호,백설,brand,water,oil,mandu,hotdog
0,20230125158386,비비고 왕교자 1.05kg,1,9.348449,F,4,2023-01-25,0,0,20945,...,0,0,0,0,0,1,0,0,1,0
1,20230126164638,고메 바삭쫄깃한 탕수육 900g,1,9.667259,F,4,2023-01-26,1,0,20945,...,0,0,1,0,0,1,0,0,0,0
2,20230125159705,햇반 매일잡곡밥210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,1,0,0,0,0,1,0,0,0,0
3,20230126167696,[UPCYCLE]햇반 흑미밥 210gx24개(1box),1,10.125110,F,3,2023-01-26,0,1,10109,...,1,0,0,0,0,1,0,0,0,0
4,20230125159705,햇반 매일찰잡곡밥 210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,1,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26065,20230101966635,고메 순살크리스피 치킨 고추크런치 450g,1,8.659560,M,3,2023-01-01,0,0,10109,...,0,0,1,0,0,1,0,0,0,0
26066,20230102974273,비비고 사골순댓국 460g,10,10.518646,M,4,2023-01-02,1,0,20945,...,0,0,0,0,0,1,0,0,0,0
26067,20230102972720,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,3,2023-01-02,0,0,10109,...,0,0,0,0,0,1,0,0,0,0
26068,20230101964953,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,4,2023-01-01,0,0,20945,...,0,0,0,0,0,1,0,0,0,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,햇반,스팸,고메,삼호,백설,brand,water,oil,mandu,hotdog
0,20230101970142,삼호 생선살어묵 야채 200g,1,8.098947,M,4,2023-01-01,NaN,0,4344,...,0,0,0,1,0,1,0,0,0,0
1,20230101964512,[앱전용특가] 쁘띠첼밀감90gX6개X3개,1,9.076809,M,4,2023-01-01,NaN,0,4344,...,0,0,0,0,0,1,0,0,0,0
2,20230102975046,빕스 볶음밥 1인분 X 6종_행사,1,9.928522,F,3,2023-01-02,NaN,1,6299,...,0,0,0,0,0,0,0,0,0,0
3,20230102973138,백설 진한참기름 500ml,2,10.060534,M,4,2023-01-02,NaN,0,4344,...,0,0,0,0,1,1,0,0,0,0
4,20230101969192,삼호 가쓰오모둠어묵탕413g,2,9.796181,M,4,2023-01-01,NaN,0,4344,...,0,0,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8041,20230103994630,고메 콰트로포르마지 피자 310g,1,8.736650,M,5,2023-01-03,NaN,0,2051,...,0,0,1,0,0,1,0,0,0,0
8042,20230103994130,햇반솥반 전복내장영양밥 200g,2,9.196748,M,3,2023-01-03,NaN,1,6299,...,1,0,0,0,0,1,0,0,0,0
8043,20230103986821,비비고 찐만두168g,1,7.930566,M,5,2023-01-03,NaN,0,2051,...,0,0,0,0,0,1,0,0,1,0
8044,20230102979023,사골곰탕 한 그릇 100g (1-2인분X5개입),1,8.477412,F,3,2023-01-02,NaN,0,6299,...,0,0,0,0,0,0,0,0,0,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,햇반,스팸,고메,삼호,백설,brand,water,oil,mandu,hotdog
0,20230101964282,비비고 스팸부대찌개 460g,1,8.161946,M,4,2023-01-01,NaN,0,9748,...,0,1,0,0,0,2,0,0,0,0
1,20230101965237,크레잇 블랙페퍼 폭찹 스테이크,1,8.277412,F,4,2023-01-01,NaN,0,9748,...,0,0,0,0,0,1,0,0,0,0
2,20230102973798,비비고 소고기 미역국 500g,3,9.145375,F,3,2023-01-02,NaN,0,5034,...,0,0,0,0,0,1,0,0,0,0
3,20230101965633,비비고 소고기 미역국 500g,2,8.613230,F,3,2023-01-01,NaN,0,5034,...,0,0,0,0,0,1,0,0,0,0
4,20230101963753,고메 치즈크리스피핫도그 340g,1,8.540714,F,4,2023-01-01,NaN,0,9748,...,0,0,1,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11609,20230102980326,고메 케이준프라이즈 피자 145g,2,8.671115,F,4,2023-01-02,NaN,0,9748,...,0,0,1,0,0,1,0,0,0,0
11610,20230102983068,고메 빅크리스피핫도그520g,1,8.641179,F,3,2023-01-02,NaN,0,5034,...,0,0,1,0,0,1,0,0,0,1
11611,20230103992835,비비고 메추리알장조림 170g,7,9.467692,F,4,2023-01-03,NaN,0,9748,...,0,0,0,0,0,1,0,0,0,0
11612,20230104999357,비비고 언양식 바싹불고기 460g,1,8.594895,M,4,2023-01-04,NaN,0,9748,...,0,0,0,0,0,1,0,0,0,0


#### 찌개 상품 구매여부

In [46]:
soup_list = ['갈비탕', '감자탕', '곰탕', '사골곰탕', '삼계탕', '설렁탕', '양지곰탕',
             '추어탕', '육개장', '찌개', '된장국', '뭇국', '미역국', '장터국', '황태국']

In [47]:
for soup in soup_list:
    # train
    emp_tr[soup] = emp_tr.product_name.apply(lambda x : int(soup in x))
    nemp_tr[soup] = nemp_tr.product_name.apply(lambda x : int(soup in x))
    
    # test
    emp_te[soup] = emp_te.product_name.apply(lambda x : int(soup in x))
    nemp_te[soup] = nemp_te.product_name.apply(lambda x : int(soup in x))

# train
emp_tr['soup'] = emp_tr[soup_list].sum(axis = 1)
nemp_tr['soup'] = nemp_tr[soup_list].sum(axis = 1)
emp_tr = emp_tr.drop(columns = soup_list, axis = 0)
nemp_tr = nemp_tr.drop(columns = soup_list, axis = 0)

#test
emp_te['soup'] = emp_te[soup_list].sum(axis = 1)
nemp_te['soup'] = nemp_te[soup_list].sum(axis = 1)
emp_te = emp_te.drop(columns = soup_list, axis = 0)
nemp_te = nemp_te.drop(columns = soup_list, axis = 0)

display(emp_tr, nemp_tr, emp_te, nemp_te)

,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,스팸,고메,삼호,백설,brand,water,oil,mandu,hotdog,soup
0,20230124153976,잔칫집 식혜 240ml 30입,1,9.803170,F,2,2023-01-24,0,0,5661,...,0,0,0,0,0,0,0,0,0,0
1,20230124155563,백설 한입쏙 비엔나 120g*2,1,8.256607,M,3,2023-01-24,0,0,14016,...,0,0,0,1,1,0,0,0,0,0
2,20230125160921,맥스봉 체다치즈어랏400g (20g*20EA),1,8.974998,F,2,2023-01-25,0,0,5661,...,0,0,0,0,1,0,0,0,1,0
3,20230125157569,고메 바삭쫄깃한 사천탕수육 450g,1,8.938007,M,4,2023-01-25,1,0,9878,...,0,1,0,0,1,0,0,0,0,0
4,20230125156655,고메 콘크림수프 180g*4입,1,8.997518,F,3,2023-01-25,0,0,14016,...,0,1,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19800,20230102975115,[2023설사원선물신청] 흑삼정 골드클래스 100gx2,1,11.597276,M,5,2023-01-02,0,0,5196,...,0,0,0,0,0,0,0,0,0,0
19801,20230102972321,고메 바삭쫄깃한 탕수육 450g,1,8.702510,M,5,2023-01-02,1,0,5196,...,0,1,0,0,1,0,0,0,0,0
19802,20230101969615,리턴업 전립소 쏘팔메토 골드 1000mgX60캡슐(2개월)X2개,1,11.364344,F,2,2023-01-01,1,0,5661,...,0,0,0,0,0,0,0,0,0,0
19803,20230102972321,고메 거멍 모짜체다핫도그 340g,1,8.648397,M,5,2023-01-02,1,0,5196,...,0,1,0,0,1,0,0,0,1,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,스팸,고메,삼호,백설,brand,water,oil,mandu,hotdog,soup
0,20230125158386,비비고 왕교자 1.05kg,1,9.348449,F,4,2023-01-25,0,0,20945,...,0,0,0,0,1,0,0,1,0,0
1,20230126164638,고메 바삭쫄깃한 탕수육 900g,1,9.667259,F,4,2023-01-26,1,0,20945,...,0,1,0,0,1,0,0,0,0,0
2,20230125159705,햇반 매일잡곡밥210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,0,0,0,0,1,0,0,0,0,0
3,20230126167696,[UPCYCLE]햇반 흑미밥 210gx24개(1box),1,10.125110,F,3,2023-01-26,0,1,10109,...,0,0,0,0,1,0,0,0,0,0
4,20230125159705,햇반 매일찰잡곡밥 210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26065,20230101966635,고메 순살크리스피 치킨 고추크런치 450g,1,8.659560,M,3,2023-01-01,0,0,10109,...,0,1,0,0,1,0,0,0,0,0
26066,20230102974273,비비고 사골순댓국 460g,10,10.518646,M,4,2023-01-02,1,0,20945,...,0,0,0,0,1,0,0,0,0,0
26067,20230102972720,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,3,2023-01-02,0,0,10109,...,0,0,0,0,1,0,0,0,0,1
26068,20230101964953,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,4,2023-01-01,0,0,20945,...,0,0,0,0,1,0,0,0,0,1


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,스팸,고메,삼호,백설,brand,water,oil,mandu,hotdog,soup
0,20230101970142,삼호 생선살어묵 야채 200g,1,8.098947,M,4,2023-01-01,NaN,0,4344,...,0,0,1,0,1,0,0,0,0,0
1,20230101964512,[앱전용특가] 쁘띠첼밀감90gX6개X3개,1,9.076809,M,4,2023-01-01,NaN,0,4344,...,0,0,0,0,1,0,0,0,0,0
2,20230102975046,빕스 볶음밥 1인분 X 6종_행사,1,9.928522,F,3,2023-01-02,NaN,1,6299,...,0,0,0,0,0,0,0,0,0,0
3,20230102973138,백설 진한참기름 500ml,2,10.060534,M,4,2023-01-02,NaN,0,4344,...,0,0,0,1,1,0,0,0,0,0
4,20230101969192,삼호 가쓰오모둠어묵탕413g,2,9.796181,M,4,2023-01-01,NaN,0,4344,...,0,0,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8041,20230103994630,고메 콰트로포르마지 피자 310g,1,8.736650,M,5,2023-01-03,NaN,0,2051,...,0,1,0,0,1,0,0,0,0,0
8042,20230103994130,햇반솥반 전복내장영양밥 200g,2,9.196748,M,3,2023-01-03,NaN,1,6299,...,0,0,0,0,1,0,0,0,0,0
8043,20230103986821,비비고 찐만두168g,1,7.930566,M,5,2023-01-03,NaN,0,2051,...,0,0,0,0,1,0,0,1,0,0
8044,20230102979023,사골곰탕 한 그릇 100g (1-2인분X5개입),1,8.477412,F,3,2023-01-02,NaN,0,6299,...,0,0,0,0,0,0,0,0,0,2


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,스팸,고메,삼호,백설,brand,water,oil,mandu,hotdog,soup
0,20230101964282,비비고 스팸부대찌개 460g,1,8.161946,M,4,2023-01-01,NaN,0,9748,...,1,0,0,0,2,0,0,0,0,1
1,20230101965237,크레잇 블랙페퍼 폭찹 스테이크,1,8.277412,F,4,2023-01-01,NaN,0,9748,...,0,0,0,0,1,0,0,0,0,0
2,20230102973798,비비고 소고기 미역국 500g,3,9.145375,F,3,2023-01-02,NaN,0,5034,...,0,0,0,0,1,0,0,0,0,1
3,20230101965633,비비고 소고기 미역국 500g,2,8.613230,F,3,2023-01-01,NaN,0,5034,...,0,0,0,0,1,0,0,0,0,1
4,20230101963753,고메 치즈크리스피핫도그 340g,1,8.540714,F,4,2023-01-01,NaN,0,9748,...,0,1,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11609,20230102980326,고메 케이준프라이즈 피자 145g,2,8.671115,F,4,2023-01-02,NaN,0,9748,...,0,1,0,0,1,0,0,0,0,0
11610,20230102983068,고메 빅크리스피핫도그520g,1,8.641179,F,3,2023-01-02,NaN,0,5034,...,0,1,0,0,1,0,0,0,1,0
11611,20230103992835,비비고 메추리알장조림 170g,7,9.467692,F,4,2023-01-03,NaN,0,9748,...,0,0,0,0,1,0,0,0,0,0
11612,20230104999357,비비고 언양식 바싹불고기 460g,1,8.594895,M,4,2023-01-04,NaN,0,9748,...,0,0,0,0,1,0,0,0,0,0


#### 가공밥 상품 구매여부

In [48]:
processed_rice_list = ['볶음밥', '주먹밥', '비빔밥']

In [49]:
for processed_rice in processed_rice_list:
    # train
    emp_tr[processed_rice] = emp_tr.product_name.apply(lambda x : int(processed_rice in x))
    nemp_tr[processed_rice] = nemp_tr.product_name.apply(lambda x : int(processed_rice in x))
    
    # test
    emp_te[processed_rice] = emp_te.product_name.apply(lambda x : int(processed_rice in x))
    nemp_te[processed_rice] = nemp_te.product_name.apply(lambda x : int(processed_rice in x))

# train
emp_tr['processed_rice'] = emp_tr[processed_rice_list].sum(axis = 1)
nemp_tr['processed_rice'] = nemp_tr[processed_rice_list].sum(axis = 1)
emp_tr = emp_tr.drop(columns = processed_rice_list, axis = 0)
nemp_tr = nemp_tr.drop(columns = processed_rice_list, axis = 0)

#test
emp_te['processed_rice'] = emp_te[processed_rice_list].sum(axis = 1)
nemp_te['processed_rice'] = nemp_te[processed_rice_list].sum(axis = 1)
emp_te = emp_te.drop(columns = processed_rice_list, axis = 0)
nemp_te = nemp_te.drop(columns = processed_rice_list, axis = 0)

display(emp_tr, nemp_tr, emp_te, nemp_te)

,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,고메,삼호,백설,brand,water,oil,mandu,hotdog,soup,processed_rice
0,20230124153976,잔칫집 식혜 240ml 30입,1,9.803170,F,2,2023-01-24,0,0,5661,...,0,0,0,0,0,0,0,0,0,0
1,20230124155563,백설 한입쏙 비엔나 120g*2,1,8.256607,M,3,2023-01-24,0,0,14016,...,0,0,1,1,0,0,0,0,0,0
2,20230125160921,맥스봉 체다치즈어랏400g (20g*20EA),1,8.974998,F,2,2023-01-25,0,0,5661,...,0,0,0,1,0,0,0,1,0,0
3,20230125157569,고메 바삭쫄깃한 사천탕수육 450g,1,8.938007,M,4,2023-01-25,1,0,9878,...,1,0,0,1,0,0,0,0,0,0
4,20230125156655,고메 콘크림수프 180g*4입,1,8.997518,F,3,2023-01-25,0,0,14016,...,1,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19800,20230102975115,[2023설사원선물신청] 흑삼정 골드클래스 100gx2,1,11.597276,M,5,2023-01-02,0,0,5196,...,0,0,0,0,0,0,0,0,0,0
19801,20230102972321,고메 바삭쫄깃한 탕수육 450g,1,8.702510,M,5,2023-01-02,1,0,5196,...,1,0,0,1,0,0,0,0,0,0
19802,20230101969615,리턴업 전립소 쏘팔메토 골드 1000mgX60캡슐(2개월)X2개,1,11.364344,F,2,2023-01-01,1,0,5661,...,0,0,0,0,0,0,0,0,0,0
19803,20230102972321,고메 거멍 모짜체다핫도그 340g,1,8.648397,M,5,2023-01-02,1,0,5196,...,1,0,0,1,0,0,0,1,0,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,고메,삼호,백설,brand,water,oil,mandu,hotdog,soup,processed_rice
0,20230125158386,비비고 왕교자 1.05kg,1,9.348449,F,4,2023-01-25,0,0,20945,...,0,0,0,1,0,0,1,0,0,0
1,20230126164638,고메 바삭쫄깃한 탕수육 900g,1,9.667259,F,4,2023-01-26,1,0,20945,...,1,0,0,1,0,0,0,0,0,0
2,20230125159705,햇반 매일잡곡밥210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,0,0,0,1,0,0,0,0,0,0
3,20230126167696,[UPCYCLE]햇반 흑미밥 210gx24개(1box),1,10.125110,F,3,2023-01-26,0,1,10109,...,0,0,0,1,0,0,0,0,0,0
4,20230125159705,햇반 매일찰잡곡밥 210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26065,20230101966635,고메 순살크리스피 치킨 고추크런치 450g,1,8.659560,M,3,2023-01-01,0,0,10109,...,1,0,0,1,0,0,0,0,0,0
26066,20230102974273,비비고 사골순댓국 460g,10,10.518646,M,4,2023-01-02,1,0,20945,...,0,0,0,1,0,0,0,0,0,0
26067,20230102972720,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,3,2023-01-02,0,0,10109,...,0,0,0,1,0,0,0,0,1,0
26068,20230101964953,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,4,2023-01-01,0,0,20945,...,0,0,0,1,0,0,0,0,1,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,고메,삼호,백설,brand,water,oil,mandu,hotdog,soup,processed_rice
0,20230101970142,삼호 생선살어묵 야채 200g,1,8.098947,M,4,2023-01-01,NaN,0,4344,...,0,1,0,1,0,0,0,0,0,0
1,20230101964512,[앱전용특가] 쁘띠첼밀감90gX6개X3개,1,9.076809,M,4,2023-01-01,NaN,0,4344,...,0,0,0,1,0,0,0,0,0,0
2,20230102975046,빕스 볶음밥 1인분 X 6종_행사,1,9.928522,F,3,2023-01-02,NaN,1,6299,...,0,0,0,0,0,0,0,0,0,1
3,20230102973138,백설 진한참기름 500ml,2,10.060534,M,4,2023-01-02,NaN,0,4344,...,0,0,1,1,0,0,0,0,0,0
4,20230101969192,삼호 가쓰오모둠어묵탕413g,2,9.796181,M,4,2023-01-01,NaN,0,4344,...,0,1,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8041,20230103994630,고메 콰트로포르마지 피자 310g,1,8.736650,M,5,2023-01-03,NaN,0,2051,...,1,0,0,1,0,0,0,0,0,0
8042,20230103994130,햇반솥반 전복내장영양밥 200g,2,9.196748,M,3,2023-01-03,NaN,1,6299,...,0,0,0,1,0,0,0,0,0,0
8043,20230103986821,비비고 찐만두168g,1,7.930566,M,5,2023-01-03,NaN,0,2051,...,0,0,0,1,0,0,1,0,0,0
8044,20230102979023,사골곰탕 한 그릇 100g (1-2인분X5개입),1,8.477412,F,3,2023-01-02,NaN,0,6299,...,0,0,0,0,0,0,0,0,2,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,고메,삼호,백설,brand,water,oil,mandu,hotdog,soup,processed_rice
0,20230101964282,비비고 스팸부대찌개 460g,1,8.161946,M,4,2023-01-01,NaN,0,9748,...,0,0,0,2,0,0,0,0,1,0
1,20230101965237,크레잇 블랙페퍼 폭찹 스테이크,1,8.277412,F,4,2023-01-01,NaN,0,9748,...,0,0,0,1,0,0,0,0,0,0
2,20230102973798,비비고 소고기 미역국 500g,3,9.145375,F,3,2023-01-02,NaN,0,5034,...,0,0,0,1,0,0,0,0,1,0
3,20230101965633,비비고 소고기 미역국 500g,2,8.613230,F,3,2023-01-01,NaN,0,5034,...,0,0,0,1,0,0,0,0,1,0
4,20230101963753,고메 치즈크리스피핫도그 340g,1,8.540714,F,4,2023-01-01,NaN,0,9748,...,1,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11609,20230102980326,고메 케이준프라이즈 피자 145g,2,8.671115,F,4,2023-01-02,NaN,0,9748,...,1,0,0,1,0,0,0,0,0,0
11610,20230102983068,고메 빅크리스피핫도그520g,1,8.641179,F,3,2023-01-02,NaN,0,5034,...,1,0,0,1,0,0,0,1,0,0
11611,20230103992835,비비고 메추리알장조림 170g,7,9.467692,F,4,2023-01-03,NaN,0,9748,...,0,0,0,1,0,0,0,0,0,0
11612,20230104999357,비비고 언양식 바싹불고기 460g,1,8.594895,M,4,2023-01-04,NaN,0,9748,...,0,0,0,1,0,0,0,0,0,0


#### 튀김 상품 구매여부

In [50]:
fried_list = ['치킨', '너겟', '닭강정', '까스', '카츠', '탕수육', '깐풍기']

In [51]:
for fried in fried_list:
    # train
    emp_tr[fried] = emp_tr.product_name.apply(lambda x : int(fried in x))
    nemp_tr[fried] = nemp_tr.product_name.apply(lambda x : int(fried in x))
    
    # test
    emp_te[fried] = emp_te.product_name.apply(lambda x : int(fried in x))
    nemp_te[fried] = nemp_te.product_name.apply(lambda x : int(fried in x))

# train
emp_tr['fried'] = emp_tr[fried_list].sum(axis = 1)
nemp_tr['fried'] = nemp_tr[fried_list].sum(axis = 1)
emp_tr = emp_tr.drop(columns = fried_list, axis = 0)
nemp_tr = nemp_tr.drop(columns = fried_list, axis = 0)

#test
emp_te['fried'] = emp_te[fried_list].sum(axis = 1)
nemp_te['fried'] = nemp_te[fried_list].sum(axis = 1)
emp_te = emp_te.drop(columns = fried_list, axis = 0)
nemp_te = nemp_te.drop(columns = fried_list, axis = 0)

display(emp_tr, nemp_tr, emp_te, nemp_te)

,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,삼호,백설,brand,water,oil,mandu,hotdog,soup,processed_rice,fried
0,20230124153976,잔칫집 식혜 240ml 30입,1,9.803170,F,2,2023-01-24,0,0,5661,...,0,0,0,0,0,0,0,0,0,0
1,20230124155563,백설 한입쏙 비엔나 120g*2,1,8.256607,M,3,2023-01-24,0,0,14016,...,0,1,1,0,0,0,0,0,0,0
2,20230125160921,맥스봉 체다치즈어랏400g (20g*20EA),1,8.974998,F,2,2023-01-25,0,0,5661,...,0,0,1,0,0,0,1,0,0,0
3,20230125157569,고메 바삭쫄깃한 사천탕수육 450g,1,8.938007,M,4,2023-01-25,1,0,9878,...,0,0,1,0,0,0,0,0,0,1
4,20230125156655,고메 콘크림수프 180g*4입,1,8.997518,F,3,2023-01-25,0,0,14016,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19800,20230102975115,[2023설사원선물신청] 흑삼정 골드클래스 100gx2,1,11.597276,M,5,2023-01-02,0,0,5196,...,0,0,0,0,0,0,0,0,0,0
19801,20230102972321,고메 바삭쫄깃한 탕수육 450g,1,8.702510,M,5,2023-01-02,1,0,5196,...,0,0,1,0,0,0,0,0,0,1
19802,20230101969615,리턴업 전립소 쏘팔메토 골드 1000mgX60캡슐(2개월)X2개,1,11.364344,F,2,2023-01-01,1,0,5661,...,0,0,0,0,0,0,0,0,0,0
19803,20230102972321,고메 거멍 모짜체다핫도그 340g,1,8.648397,M,5,2023-01-02,1,0,5196,...,0,0,1,0,0,0,1,0,0,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,삼호,백설,brand,water,oil,mandu,hotdog,soup,processed_rice,fried
0,20230125158386,비비고 왕교자 1.05kg,1,9.348449,F,4,2023-01-25,0,0,20945,...,0,0,1,0,0,1,0,0,0,0
1,20230126164638,고메 바삭쫄깃한 탕수육 900g,1,9.667259,F,4,2023-01-26,1,0,20945,...,0,0,1,0,0,0,0,0,0,1
2,20230125159705,햇반 매일잡곡밥210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,0,0,1,0,0,0,0,0,0,0
3,20230126167696,[UPCYCLE]햇반 흑미밥 210gx24개(1box),1,10.125110,F,3,2023-01-26,0,1,10109,...,0,0,1,0,0,0,0,0,0,0
4,20230125159705,햇반 매일찰잡곡밥 210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26065,20230101966635,고메 순살크리스피 치킨 고추크런치 450g,1,8.659560,M,3,2023-01-01,0,0,10109,...,0,0,1,0,0,0,0,0,0,1
26066,20230102974273,비비고 사골순댓국 460g,10,10.518646,M,4,2023-01-02,1,0,20945,...,0,0,1,0,0,0,0,0,0,0
26067,20230102972720,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,3,2023-01-02,0,0,10109,...,0,0,1,0,0,0,0,1,0,0
26068,20230101964953,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,4,2023-01-01,0,0,20945,...,0,0,1,0,0,0,0,1,0,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,삼호,백설,brand,water,oil,mandu,hotdog,soup,processed_rice,fried
0,20230101970142,삼호 생선살어묵 야채 200g,1,8.098947,M,4,2023-01-01,NaN,0,4344,...,1,0,1,0,0,0,0,0,0,0
1,20230101964512,[앱전용특가] 쁘띠첼밀감90gX6개X3개,1,9.076809,M,4,2023-01-01,NaN,0,4344,...,0,0,1,0,0,0,0,0,0,0
2,20230102975046,빕스 볶음밥 1인분 X 6종_행사,1,9.928522,F,3,2023-01-02,NaN,1,6299,...,0,0,0,0,0,0,0,0,1,0
3,20230102973138,백설 진한참기름 500ml,2,10.060534,M,4,2023-01-02,NaN,0,4344,...,0,1,1,0,0,0,0,0,0,0
4,20230101969192,삼호 가쓰오모둠어묵탕413g,2,9.796181,M,4,2023-01-01,NaN,0,4344,...,1,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8041,20230103994630,고메 콰트로포르마지 피자 310g,1,8.736650,M,5,2023-01-03,NaN,0,2051,...,0,0,1,0,0,0,0,0,0,0
8042,20230103994130,햇반솥반 전복내장영양밥 200g,2,9.196748,M,3,2023-01-03,NaN,1,6299,...,0,0,1,0,0,0,0,0,0,0
8043,20230103986821,비비고 찐만두168g,1,7.930566,M,5,2023-01-03,NaN,0,2051,...,0,0,1,0,0,1,0,0,0,0
8044,20230102979023,사골곰탕 한 그릇 100g (1-2인분X5개입),1,8.477412,F,3,2023-01-02,NaN,0,6299,...,0,0,0,0,0,0,0,2,0,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,삼호,백설,brand,water,oil,mandu,hotdog,soup,processed_rice,fried
0,20230101964282,비비고 스팸부대찌개 460g,1,8.161946,M,4,2023-01-01,NaN,0,9748,...,0,0,2,0,0,0,0,1,0,0
1,20230101965237,크레잇 블랙페퍼 폭찹 스테이크,1,8.277412,F,4,2023-01-01,NaN,0,9748,...,0,0,1,0,0,0,0,0,0,0
2,20230102973798,비비고 소고기 미역국 500g,3,9.145375,F,3,2023-01-02,NaN,0,5034,...,0,0,1,0,0,0,0,1,0,0
3,20230101965633,비비고 소고기 미역국 500g,2,8.613230,F,3,2023-01-01,NaN,0,5034,...,0,0,1,0,0,0,0,1,0,0
4,20230101963753,고메 치즈크리스피핫도그 340g,1,8.540714,F,4,2023-01-01,NaN,0,9748,...,0,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11609,20230102980326,고메 케이준프라이즈 피자 145g,2,8.671115,F,4,2023-01-02,NaN,0,9748,...,0,0,1,0,0,0,0,0,0,0
11610,20230102983068,고메 빅크리스피핫도그520g,1,8.641179,F,3,2023-01-02,NaN,0,5034,...,0,0,1,0,0,0,1,0,0,0
11611,20230103992835,비비고 메추리알장조림 170g,7,9.467692,F,4,2023-01-03,NaN,0,9748,...,0,0,1,0,0,0,0,0,0,0
11612,20230104999357,비비고 언양식 바싹불고기 460g,1,8.594895,M,4,2023-01-04,NaN,0,9748,...,0,0,1,0,0,0,0,0,0,0


#### 가공김 상품 구매여부

In [52]:
processed_kim_list = ['김밥김', '구이김', '재래김', '김자반', '파래김', '감태김', '바삭한김', '토종김', '구운김', '돌김', '간장김']

In [53]:
for processed_kim in processed_kim_list:
    # train
    emp_tr[processed_kim] = emp_tr.product_name.apply(lambda x : int(processed_kim in x))
    nemp_tr[processed_kim] = nemp_tr.product_name.apply(lambda x : int(processed_kim in x))
    
    # test
    emp_te[processed_kim] = emp_te.product_name.apply(lambda x : int(processed_kim in x))
    nemp_te[processed_kim] = nemp_te.product_name.apply(lambda x : int(processed_kim in x))

# train
emp_tr['processed_kim'] = emp_tr[processed_kim_list].sum(axis = 1)
nemp_tr['processed_kim'] = nemp_tr[processed_kim_list].sum(axis = 1)
emp_tr = emp_tr.drop(columns = processed_kim_list, axis = 0)
nemp_tr = nemp_tr.drop(columns = processed_kim_list, axis = 0)

#test
emp_te['processed_kim'] = emp_te[processed_kim_list].sum(axis = 1)
nemp_te['processed_kim'] = nemp_te[processed_kim_list].sum(axis = 1)
emp_te = emp_te.drop(columns = processed_kim_list, axis = 0)
nemp_te = nemp_te.drop(columns = processed_kim_list, axis = 0)

display(emp_tr, nemp_tr, emp_te, nemp_te)

,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,백설,brand,water,oil,mandu,hotdog,soup,processed_rice,fried,processed_kim
0,20230124153976,잔칫집 식혜 240ml 30입,1,9.803170,F,2,2023-01-24,0,0,5661,...,0,0,0,0,0,0,0,0,0,0
1,20230124155563,백설 한입쏙 비엔나 120g*2,1,8.256607,M,3,2023-01-24,0,0,14016,...,1,1,0,0,0,0,0,0,0,0
2,20230125160921,맥스봉 체다치즈어랏400g (20g*20EA),1,8.974998,F,2,2023-01-25,0,0,5661,...,0,1,0,0,0,1,0,0,0,0
3,20230125157569,고메 바삭쫄깃한 사천탕수육 450g,1,8.938007,M,4,2023-01-25,1,0,9878,...,0,1,0,0,0,0,0,0,1,0
4,20230125156655,고메 콘크림수프 180g*4입,1,8.997518,F,3,2023-01-25,0,0,14016,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19800,20230102975115,[2023설사원선물신청] 흑삼정 골드클래스 100gx2,1,11.597276,M,5,2023-01-02,0,0,5196,...,0,0,0,0,0,0,0,0,0,0
19801,20230102972321,고메 바삭쫄깃한 탕수육 450g,1,8.702510,M,5,2023-01-02,1,0,5196,...,0,1,0,0,0,0,0,0,1,0
19802,20230101969615,리턴업 전립소 쏘팔메토 골드 1000mgX60캡슐(2개월)X2개,1,11.364344,F,2,2023-01-01,1,0,5661,...,0,0,0,0,0,0,0,0,0,0
19803,20230102972321,고메 거멍 모짜체다핫도그 340g,1,8.648397,M,5,2023-01-02,1,0,5196,...,0,1,0,0,0,1,0,0,0,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,백설,brand,water,oil,mandu,hotdog,soup,processed_rice,fried,processed_kim
0,20230125158386,비비고 왕교자 1.05kg,1,9.348449,F,4,2023-01-25,0,0,20945,...,0,1,0,0,1,0,0,0,0,0
1,20230126164638,고메 바삭쫄깃한 탕수육 900g,1,9.667259,F,4,2023-01-26,1,0,20945,...,0,1,0,0,0,0,0,0,1,0
2,20230125159705,햇반 매일잡곡밥210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,0,1,0,0,0,0,0,0,0,0
3,20230126167696,[UPCYCLE]햇반 흑미밥 210gx24개(1box),1,10.125110,F,3,2023-01-26,0,1,10109,...,0,1,0,0,0,0,0,0,0,0
4,20230125159705,햇반 매일찰잡곡밥 210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26065,20230101966635,고메 순살크리스피 치킨 고추크런치 450g,1,8.659560,M,3,2023-01-01,0,0,10109,...,0,1,0,0,0,0,0,0,1,0
26066,20230102974273,비비고 사골순댓국 460g,10,10.518646,M,4,2023-01-02,1,0,20945,...,0,1,0,0,0,0,0,0,0,0
26067,20230102972720,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,3,2023-01-02,0,0,10109,...,0,1,0,0,0,0,1,0,0,0
26068,20230101964953,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,4,2023-01-01,0,0,20945,...,0,1,0,0,0,0,1,0,0,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,백설,brand,water,oil,mandu,hotdog,soup,processed_rice,fried,processed_kim
0,20230101970142,삼호 생선살어묵 야채 200g,1,8.098947,M,4,2023-01-01,NaN,0,4344,...,0,1,0,0,0,0,0,0,0,0
1,20230101964512,[앱전용특가] 쁘띠첼밀감90gX6개X3개,1,9.076809,M,4,2023-01-01,NaN,0,4344,...,0,1,0,0,0,0,0,0,0,0
2,20230102975046,빕스 볶음밥 1인분 X 6종_행사,1,9.928522,F,3,2023-01-02,NaN,1,6299,...,0,0,0,0,0,0,0,1,0,0
3,20230102973138,백설 진한참기름 500ml,2,10.060534,M,4,2023-01-02,NaN,0,4344,...,1,1,0,0,0,0,0,0,0,0
4,20230101969192,삼호 가쓰오모둠어묵탕413g,2,9.796181,M,4,2023-01-01,NaN,0,4344,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8041,20230103994630,고메 콰트로포르마지 피자 310g,1,8.736650,M,5,2023-01-03,NaN,0,2051,...,0,1,0,0,0,0,0,0,0,0
8042,20230103994130,햇반솥반 전복내장영양밥 200g,2,9.196748,M,3,2023-01-03,NaN,1,6299,...,0,1,0,0,0,0,0,0,0,0
8043,20230103986821,비비고 찐만두168g,1,7.930566,M,5,2023-01-03,NaN,0,2051,...,0,1,0,0,1,0,0,0,0,0
8044,20230102979023,사골곰탕 한 그릇 100g (1-2인분X5개입),1,8.477412,F,3,2023-01-02,NaN,0,6299,...,0,0,0,0,0,0,2,0,0,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,백설,brand,water,oil,mandu,hotdog,soup,processed_rice,fried,processed_kim
0,20230101964282,비비고 스팸부대찌개 460g,1,8.161946,M,4,2023-01-01,NaN,0,9748,...,0,2,0,0,0,0,1,0,0,0
1,20230101965237,크레잇 블랙페퍼 폭찹 스테이크,1,8.277412,F,4,2023-01-01,NaN,0,9748,...,0,1,0,0,0,0,0,0,0,0
2,20230102973798,비비고 소고기 미역국 500g,3,9.145375,F,3,2023-01-02,NaN,0,5034,...,0,1,0,0,0,0,1,0,0,0
3,20230101965633,비비고 소고기 미역국 500g,2,8.613230,F,3,2023-01-01,NaN,0,5034,...,0,1,0,0,0,0,1,0,0,0
4,20230101963753,고메 치즈크리스피핫도그 340g,1,8.540714,F,4,2023-01-01,NaN,0,9748,...,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11609,20230102980326,고메 케이준프라이즈 피자 145g,2,8.671115,F,4,2023-01-02,NaN,0,9748,...,0,1,0,0,0,0,0,0,0,0
11610,20230102983068,고메 빅크리스피핫도그520g,1,8.641179,F,3,2023-01-02,NaN,0,5034,...,0,1,0,0,0,1,0,0,0,0
11611,20230103992835,비비고 메추리알장조림 170g,7,9.467692,F,4,2023-01-03,NaN,0,9748,...,0,1,0,0,0,0,0,0,0,0
11612,20230104999357,비비고 언양식 바싹불고기 460g,1,8.594895,M,4,2023-01-04,NaN,0,9748,...,0,1,0,0,0,0,0,0,0,0


#### 자동배송 상위 10개 상품 구매여부

In [54]:
auto_top10_list = ['고메 치즈크리스피핫도그 340g','고메 바삭촉촉한 깐풍기 450g','햇반쿡반 김치치즈주먹밥 500g',
                   '비비고 수제진한김치만두 400gx2','고메 바삭탱글한 칠리새우 300g','스팸라이트 200gx3번들(노캡)',
                   '고메 프리미엄 삼선해물짬뽕 (2인분)','고메 통모짜 핫도그 340g','고메 오리지널 핫도그 400g',
                   '고메 크리스피너겟 450g']

In [55]:
# train
emp_tr['auto_delivery'] = emp_tr['product_name'].apply(lambda x : int(x in auto_top10_list))
nemp_tr['auto_delivery'] = nemp_tr['product_name'].apply(lambda x : int(x in auto_top10_list))

# test
emp_te['auto_delivery'] = emp_te['product_name'].apply(lambda x : int(x in auto_top10_list))
nemp_te['auto_delivery'] = nemp_te['product_name'].apply(lambda x : int(x in auto_top10_list))

display(emp_tr, nemp_tr, emp_te, nemp_te)

,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,brand,water,oil,mandu,hotdog,soup,processed_rice,fried,processed_kim,auto_delivery
0,20230124153976,잔칫집 식혜 240ml 30입,1,9.803170,F,2,2023-01-24,0,0,5661,...,0,0,0,0,0,0,0,0,0,0
1,20230124155563,백설 한입쏙 비엔나 120g*2,1,8.256607,M,3,2023-01-24,0,0,14016,...,1,0,0,0,0,0,0,0,0,0
2,20230125160921,맥스봉 체다치즈어랏400g (20g*20EA),1,8.974998,F,2,2023-01-25,0,0,5661,...,1,0,0,0,1,0,0,0,0,0
3,20230125157569,고메 바삭쫄깃한 사천탕수육 450g,1,8.938007,M,4,2023-01-25,1,0,9878,...,1,0,0,0,0,0,0,1,0,0
4,20230125156655,고메 콘크림수프 180g*4입,1,8.997518,F,3,2023-01-25,0,0,14016,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19800,20230102975115,[2023설사원선물신청] 흑삼정 골드클래스 100gx2,1,11.597276,M,5,2023-01-02,0,0,5196,...,0,0,0,0,0,0,0,0,0,0
19801,20230102972321,고메 바삭쫄깃한 탕수육 450g,1,8.702510,M,5,2023-01-02,1,0,5196,...,1,0,0,0,0,0,0,1,0,0
19802,20230101969615,리턴업 전립소 쏘팔메토 골드 1000mgX60캡슐(2개월)X2개,1,11.364344,F,2,2023-01-01,1,0,5661,...,0,0,0,0,0,0,0,0,0,0
19803,20230102972321,고메 거멍 모짜체다핫도그 340g,1,8.648397,M,5,2023-01-02,1,0,5196,...,1,0,0,0,1,0,0,0,0,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,brand,water,oil,mandu,hotdog,soup,processed_rice,fried,processed_kim,auto_delivery
0,20230125158386,비비고 왕교자 1.05kg,1,9.348449,F,4,2023-01-25,0,0,20945,...,1,0,0,1,0,0,0,0,0,0
1,20230126164638,고메 바삭쫄깃한 탕수육 900g,1,9.667259,F,4,2023-01-26,1,0,20945,...,1,0,0,0,0,0,0,1,0,0
2,20230125159705,햇반 매일잡곡밥210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,1,0,0,0,0,0,0,0,0,0
3,20230126167696,[UPCYCLE]햇반 흑미밥 210gx24개(1box),1,10.125110,F,3,2023-01-26,0,1,10109,...,1,0,0,0,0,0,0,0,0,0
4,20230125159705,햇반 매일찰잡곡밥 210g,20,9.994653,M,4,2023-01-25,1,1,20945,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26065,20230101966635,고메 순살크리스피 치킨 고추크런치 450g,1,8.659560,M,3,2023-01-01,0,0,10109,...,1,0,0,0,0,0,0,1,0,0
26066,20230102974273,비비고 사골순댓국 460g,10,10.518646,M,4,2023-01-02,1,0,20945,...,1,0,0,0,0,0,0,0,0,0
26067,20230102972720,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,3,2023-01-02,0,0,10109,...,1,0,0,0,0,1,0,0,0,0
26068,20230101964953,[앱전용특가]비비고 차돌된장찌개 460gX4개,1,9.639327,F,4,2023-01-01,0,0,20945,...,1,0,0,0,0,1,0,0,0,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,brand,water,oil,mandu,hotdog,soup,processed_rice,fried,processed_kim,auto_delivery
0,20230101970142,삼호 생선살어묵 야채 200g,1,8.098947,M,4,2023-01-01,NaN,0,4344,...,1,0,0,0,0,0,0,0,0,0
1,20230101964512,[앱전용특가] 쁘띠첼밀감90gX6개X3개,1,9.076809,M,4,2023-01-01,NaN,0,4344,...,1,0,0,0,0,0,0,0,0,0
2,20230102975046,빕스 볶음밥 1인분 X 6종_행사,1,9.928522,F,3,2023-01-02,NaN,1,6299,...,0,0,0,0,0,0,1,0,0,0
3,20230102973138,백설 진한참기름 500ml,2,10.060534,M,4,2023-01-02,NaN,0,4344,...,1,0,0,0,0,0,0,0,0,0
4,20230101969192,삼호 가쓰오모둠어묵탕413g,2,9.796181,M,4,2023-01-01,NaN,0,4344,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8041,20230103994630,고메 콰트로포르마지 피자 310g,1,8.736650,M,5,2023-01-03,NaN,0,2051,...,1,0,0,0,0,0,0,0,0,0
8042,20230103994130,햇반솥반 전복내장영양밥 200g,2,9.196748,M,3,2023-01-03,NaN,1,6299,...,1,0,0,0,0,0,0,0,0,0
8043,20230103986821,비비고 찐만두168g,1,7.930566,M,5,2023-01-03,NaN,0,2051,...,1,0,0,1,0,0,0,0,0,0
8044,20230102979023,사골곰탕 한 그릇 100g (1-2인분X5개입),1,8.477412,F,3,2023-01-02,NaN,0,6299,...,0,0,0,0,0,2,0,0,0,0


,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,order_date,prime_yn,whether_rice,age_qty_sum,...,brand,water,oil,mandu,hotdog,soup,processed_rice,fried,processed_kim,auto_delivery
0,20230101964282,비비고 스팸부대찌개 460g,1,8.161946,M,4,2023-01-01,NaN,0,9748,...,2,0,0,0,0,1,0,0,0,0
1,20230101965237,크레잇 블랙페퍼 폭찹 스테이크,1,8.277412,F,4,2023-01-01,NaN,0,9748,...,1,0,0,0,0,0,0,0,0,0
2,20230102973798,비비고 소고기 미역국 500g,3,9.145375,F,3,2023-01-02,NaN,0,5034,...,1,0,0,0,0,1,0,0,0,0
3,20230101965633,비비고 소고기 미역국 500g,2,8.613230,F,3,2023-01-01,NaN,0,5034,...,1,0,0,0,0,1,0,0,0,0
4,20230101963753,고메 치즈크리스피핫도그 340g,1,8.540714,F,4,2023-01-01,NaN,0,9748,...,1,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11609,20230102980326,고메 케이준프라이즈 피자 145g,2,8.671115,F,4,2023-01-02,NaN,0,9748,...,1,0,0,0,0,0,0,0,0,0
11610,20230102983068,고메 빅크리스피핫도그520g,1,8.641179,F,3,2023-01-02,NaN,0,5034,...,1,0,0,0,1,0,0,0,0,0
11611,20230103992835,비비고 메추리알장조림 170g,7,9.467692,F,4,2023-01-03,NaN,0,9748,...,1,0,0,0,0,0,0,0,0,0
11612,20230104999357,비비고 언양식 바싹불고기 460g,1,8.594895,M,4,2023-01-04,NaN,0,9748,...,1,0,0,0,0,0,0,0,0,0


# Data Saving

In [56]:
# train date save
emp_tr.to_csv('data/employee_feature_train.csv',index=False)
nemp_tr.to_csv('data/nemployee_feature_train.csv',index=False)

# test data save
emp_te.to_csv('data/employee_feature_test.csv',index=False)
nemp_te.to_csv('data/nemployee_feature_test.csv',index=False)